<a href="https://colab.research.google.com/github/Umariqbal777/pestdetection/blob/main/majorproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install flask flask-ngrok tensorflow==2.19 tensorflow-io tensorflow-hub pillow --quiet


In [7]:
# Colab-ready: Full Modern UI Flask app (Version 2)
# Paste and run this full cell in Google Colab.

# 0) Install dependencies
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# 1) Imports
import os, time, sqlite3, json, shutil
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, send_file
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import numpy as np
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as keras_image
from deep_translator import GoogleTranslator

# 2) Paths & config
MODEL_CANDIDATES = ["/mnt/data/best_crop_model.h5", "/mnt/data/crop_disease_model.h5"]
MODEL_PATH = None
for p in MODEL_CANDIDATES:
    if os.path.exists(p):
        MODEL_PATH = p
        break

CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot',
    'Pepper__bell___healthy',
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy',
    'Tomato_Bacterial_spot',
    'Tomato_Early_blight',
    'Tomato_Late_blight',
    'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot',
    'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot',
    'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus',
    'Tomato_healthy'
]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]

BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)

DB_PATH = os.path.join(BASE_DIR, "database.db")
ALLOWED_EXT = {"png","jpg","jpeg"}
IMAGE_SIZE = (128,128)   # model input size
PREPROCESS_METHOD = "simple_rescale"  # we normalize by /255.0 (matches previous fixed app)

# 3) Initialize DB
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL
                   )''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     timestamp TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id)
                   )''')
    conn.commit()
    conn.close()
init_db()

# 4) Ask for ngrok authtoken (hidden)
print("🔑 Paste your ngrok authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken). It will be hidden.")
_ngrok_token = getpass("ngrok authtoken: ")
if _ngrok_token and _ngrok_token.strip():
    try:
        ngrok.set_auth_token(_ngrok_token.strip())
    except Exception as e:
        print("Warning: failed to set ngrok token:", e)

# 5) Create premium UI templates (Tailwind) --------------------------------
# NOTE: templates below are intentionally comprehensive but concise enough for Colab.
layout_html = """<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <script src="https://cdn.tailwindcss.com"></script>
  <title>{{ title or 'AgroScan AI' }}</title>
</head>
<body class="bg-gray-50 text-gray-800">
<nav class="bg-white/80 backdrop-blur fixed w-full z-30 shadow">
  <div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between">
    <a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg"><path d="M2 12l5-5 5 5-5 5-5-5z" fill="currentColor"/></svg><span class="font-bold text-xl">AgroScan AI</span></a>
    <div class="space-x-4">
      {% if current_user.is_authenticated %}
        <span class="hidden md:inline text-gray-700">Welcome, {{ current_user.username }}!</span>
        <a href="{{ url_for('dashboard') }}" class="text-gray-600 hover:text-green-600">Dashboard</a>
        <a href="{{ url_for('analyze_page') }}" class="text-gray-600 hover:text-green-600">Analyze</a>
        <a href="{{ url_for('history') }}" class="text-gray-600 hover:text-green-600">History</a>
        <a href="{{ url_for('metrics') }}" class="text-gray-600 hover:text-green-600">Metrics</a>
        <a href="{{ url_for('logout') }}" class="text-blue-600 font-semibold">Logout</a>
      {% else %}
        <a href="{{ url_for('login') }}" class="text-gray-600 hover:text-green-600">Login</a>
        <a href="{{ url_for('register') }}" class="text-gray-600 hover:text-green-600">Register</a>
      {% endif %}
      <a href="{{ url_for('set_language', lang='en') }}" class="text-sm">EN</a>
      <a href="{{ url_for('set_language', lang='hi') }}" class="text-sm">HI</a>
      <a href="{{ url_for('set_language', lang='ur') }}" class="text-sm">UR</a>
    </div>
  </div>
</nav>
<main class="pt-20 max-w-6xl mx-auto p-6">
  {% with messages = get_flashed_messages() %}
    {% if messages %}
      <div class="mb-4 p-3 bg-red-100 text-red-700 rounded">{{ messages[0] }}</div>
    {% endif %}
  {% endwith %}
  {% block content %}{% endblock %}
</main>
<footer class="max-w-6xl mx-auto text-center text-gray-500 py-6">
  © 2025 AgroScan AI — Student Project
</footer>
</body>
</html>
"""
with open(os.path.join(TEMPLATES_DIR, "layout.html"), "w", encoding="utf-8") as f: f.write(layout_html)

# Home
home_html = """{% extends "layout.html" %}
{% block content %}
<div class="grid md:grid-cols-2 gap-8 items-center">
  <div>
    <h1 class="text-4xl font-bold">Revolutionize Your Farming with AI</h1>
    <p class="mt-4 text-gray-600">Upload a leaf image and get instant disease detection, confidence score and remedies.</p>
    <div class="mt-6">
      {% if current_user.is_authenticated %}
        <a href="{{ url_for('analyze_page') }}" class="bg-green-600 text-white px-4 py-2 rounded">Get Started</a>
        <a href="{{ url_for('history') }}" class="ml-3 text-gray-600">History</a>
      {% else %}
        <a href="{{ url_for('login') }}" class="bg-blue-600 text-white px-4 py-2 rounded">Login</a>
      {% endif %}
    </div>
  </div>
  <div>
    <img src="{{ url_for('static', filename='uploads/sample_leaf.png') }}" class="rounded-lg shadow" alt="sample">
  </div>
</div>
{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR, "home.html"), "w", encoding="utf-8") as f: f.write(home_html)

# Analyze (drag & drop)
index_html = """{% extends "layout.html" %}
{% block content %}
<div class="max-w-3xl mx-auto bg-white p-6 rounded-xl shadow">
  <h2 class="text-2xl font-bold mb-4">Upload a leaf image</h2>
  <form id="upload-form" method="post" action="{{ url_for('predict') }}" enctype="multipart/form-data">
    <label for="file-upload" class="block border-2 border-dashed p-8 rounded cursor-pointer hover:border-green-500">
      <input id="file-upload" name="file" type="file" accept="image/*" class="hidden">
      <div class="text-center">
        <svg class="mx-auto h-12 w-12 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor"><path d="M7 16l-4-4m0 0l4-4M3 12h18"/></svg>
        <p class="mt-2 text-gray-600">Click to select or drag & drop an image (JPG/PNG)</p>
        <p id="file-info" class="mt-2 text-sm text-gray-500"></p>
      </div>
    </label>
    <div class="mt-4 flex space-x-3">
      <button id="submit-button" class="bg-green-600 text-white px-4 py-2 rounded">Detect Disease</button>
      <div id="loader" class="hidden items-center"><div class="animate-spin w-5 h-5 border-2 border-green-600 rounded-full mr-2"></div><span>Analyzing...</span></div>
    </div>
  </form>
</div>
<script>
const fileUpload = document.getElementById('file-upload'), form=document.getElementById('upload-form'), fileInfo=document.getElementById('file-info'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader');
fileUpload.addEventListener('change', e=>{ const f=e.target.files[0]; if(f) fileInfo.textContent = `File: ${f.name}`; });
form.addEventListener('submit', e=>{ if(!fileUpload.files.length){ e.preventDefault(); alert('Please select an image to upload.'); } submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });
</script>
{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR, "index.html"), "w", encoding="utf-8") as f: f.write(index_html)

# Result (animated confidence bar + remedies)
result_html = """{% extends "layout.html" %}
{% block content %}
<div class="max-w-4xl mx-auto bg-white p-6 rounded-xl shadow">
  <div class="md:flex md:gap-6">
    <div class="md:w-1/2">
      <img src="{{ image_path }}" class="rounded w-full object-cover" alt="uploaded">
    </div>
    <div class="md:w-1/2">
      <h2 class="text-3xl font-bold text-green-700">{{ prediction }}</h2>
      <p class="mt-2 text-gray-600">Confidence: <strong>{{ confidence }}%</strong></p>
      <div class="w-full bg-gray-200 rounded-full h-6 mt-3 overflow-hidden" title="{{ confidence }}%">
        {% if confidence_level == 'High' %}
          <div class="h-6 bg-green-500 text-white flex items-center justify-center font-bold" style="width: {{ confidence }}%;">{{ confidence }}%</div>
        {% elif confidence_level == 'Medium' %}
          <div class="h-6 bg-yellow-500 text-white flex items-center justify-center font-bold" style="width: {{ confidence }}%;">{{ confidence }}%</div>
        {% else %}
          <div class="h-6 bg-red-500 text-white flex items-center justify-center font-bold" style="width: {{ confidence }}%;">{{ confidence }}%</div>
        {% endif %}
      </div>
      <div class="mt-4">
        <h3 class="font-semibold">About this condition</h3>
        <p class="text-gray-600">{{ description }}</p>
      </div>
      {% if remedies %}
      <div class="mt-4 bg-green-50 p-4 rounded">
        <h4 class="font-semibold">Suggested Actions</h4>
        <ul class="list-disc pl-6">
          {% for r in remedies %}<li>{{ r }}</li>{% endfor %}
        </ul>
      </div>
      {% endif %}
      <div class="mt-4">
        <button id="correct-btn" class="bg-green-600 text-white px-3 py-1 rounded">👍 Correct</button>
        <button id="incorrect-btn" class="bg-red-600 text-white px-3 py-1 rounded">👎 Incorrect</button>
        <p id="feedback-thanks" class="hidden text-green-700 mt-2 font-semibold">Thank you for your feedback!</p>
      </div>
    </div>
  </div>
</div>
<script>
const correctBtn=document.getElementById('correct-btn'), incorrectBtn=document.getElementById('incorrect-btn'), thanks=document.getElementById('feedback-thanks');
function handle(){ correctBtn.disabled=true; incorrectBtn.disabled=true; thanks.classList.remove('hidden'); }
correctBtn.addEventListener('click', handle); incorrectBtn.addEventListener('click', handle);
</script>
{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR, "result.html"), "w", encoding="utf-8") as f: f.write(result_html)

# History, dashboard, login/register, metrics
history_html = """{% extends "layout.html" %}
{% block content %}
<h2 class="text-2xl font-bold mb-4">Prediction History</h2>
{% if history %}
  <div class="grid md:grid-cols-3 gap-4">
    {% for p in history %}
      <div class="bg-white rounded shadow overflow-hidden">
        <img src="{{ p.image_path }}" class="w-full h-40 object-cover">
        <div class="p-3">
          <div class="text-sm text-gray-500">{{ p.timestamp }}</div>
          <div class="font-semibold">{{ p.prediction }}</div>
          <div class="text-sm text-gray-600">Confidence: {{ p.confidence }}%</div>
        </div>
      </div>
    {% endfor %}
  </div>
{% else %}
  <p>No saved predictions yet.</p>
{% endif %}
{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR, "history.html"), "w", encoding="utf-8") as f: f.write(history_html)

dashboard_html = """{% extends "layout.html" %}
{% block content %}
<h2 class="text-2xl font-bold mb-4">Dashboard</h2>
<div class="bg-white p-4 rounded shadow">
  {% if disease_data.labels %}
    <ul>
      {% for i in range(disease_data.labels|length) %}
        <li>{{ disease_data.labels[i] }} — {{ disease_data.values[i] }}</li>
      {% endfor %}
    </ul>
  {% else %}
    <p>No detections yet. Analyze to populate your dashboard.</p>
  {% endif %}
</div>
{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR, "dashboard.html"), "w", encoding="utf-8") as f: f.write(dashboard_html)

login_html = """{% extends "layout.html" %}
{% block content %}
<div class="max-w-md mx-auto bg-white p-6 rounded shadow">
  <h2 class="text-2xl font-bold mb-4">Login</h2>
  <form method="post" action="{{ url_for('login') }}">
    <label class="block">Username<input name="username" class="w-full p-2 border rounded mt-1" required></label>
    <label class="block mt-3">Password<input name="password" type="password" class="w-full p-2 border rounded mt-1" required></label>
    <div class="mt-4"><button class="bg-green-600 text-white px-4 py-2 rounded">Sign in</button></div>
  </form>
  <p class="mt-3 text-sm">Not a member? <a href="{{ url_for('register') }}" class="text-green-600">Register here</a></p>
</div>
{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR, "login.html"), "w", encoding="utf-8") as f: f.write(login_html)

register_html = """{% extends "layout.html" %}
{% block content %}
<div class="max-w-md mx-auto bg-white p-6 rounded shadow">
  <h2 class="text-2xl font-bold mb-4">Register</h2>
  <form method="post" action="{{ url_for('register') }}">
    <label class="block">Username<input name="username" class="w-full p-2 border rounded mt-1" required></label>
    <label class="block mt-3">Password<input name="password" type="password" class="w-full p-2 border rounded mt-1" required></label>
    <div class="mt-4"><button class="bg-green-600 text-white px-4 py-2 rounded">Register</button></div>
  </form>
</div>
{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR, "register.html"), "w", encoding="utf-8") as f: f.write(register_html)

metrics_html = """{% extends "layout.html" %}
{% block content %}
<h2 class="text-2xl font-bold mb-4">Training Metrics</h2>
<div class="bg-white p-4 rounded shadow">
  <p>If you upload training_results.json, confusion_matrix.png or classification_report.txt into the Colab root, they will be shown here.</p>
  {% if training_results %}
    <pre class="whitespace-pre-wrap bg-gray-50 p-2 rounded">{{ training_results | tojson(indent=2) }}</pre>
  {% endif %}
  {% if confusion_exists %}
    <h3 class="mt-3">Confusion Matrix</h3>
    <img src="{{ url_for('static', filename='confusion_matrix.png') }}" class="max-w-full">
  {% endif %}
  {% if report_exists %}
    <h3 class="mt-3">Classification Report</h3>
    <pre class="bg-gray-50 p-2 rounded">{{ report_text }}</pre>
  {% endif %}
</div>
{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR, "metrics.html"), "w", encoding="utf-8") as f: f.write(metrics_html)

# create sample static image for hero
sample_image = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_image):
    img = Image.new("RGB", (900,600), (235,251,230))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_image)

# 6) Minimal disease info (extendable)
disease_info = {
    'Pepper__bell___Bacterial_spot': {
        'description':'A bacterial disease causing dark water-soaked spots on leaves and fruit.',
        'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']
    },
    'Pepper__bell___healthy': {'description':'The plant appears healthy.','remedies':[]},
    'Potato___Early_blight': {'description':'Target-like rings on leaves.', 'remedies':['Use certified seed','Apply fungicides','Rotate crops']},
    'Potato___Late_blight': {'description':'Large dark lesions, can rapidly destroy crops.', 'remedies':['Remove infected plants','Use protective fungicides']},
    'Tomato_healthy': {'description':'Healthy tomato plant','remedies':[]},
    'default': {'description':'Information not available.','remedies':['Consult local extension services']}
}

# translation cache
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out
        return out
    except Exception as e:
        print("Translation error:", e)
        return text

# 7) Load model
MODEL = None
if MODEL_PATH:
    try:
        MODEL = load_model(MODEL_PATH, compile=False)
        print("✔ Model loaded from", MODEL_PATH)
    except Exception as e:
        print("⚠️ Failed to load model:", e)
else:
    print("⚠️ No model file found in expected paths. Upload best_crop_model.h5 to /mnt/data and re-run.")

# 8) Flask app and auth setup
app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.urandom(24)
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'

class User(UserMixin): pass

@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    row = cur.fetchone(); conn.close()
    if row:
        u = User(); u.id = row[0]; u.username = row[1]; return u
    return None

@app.context_processor
def inject_trans():
    def _(text):
        lang = session.get('language','en')
        return get_translation(text, lang)
    return dict(_=_)

# 9) Routes
@app.route('/language/<lang>')
def set_language(lang):
    session['language']=lang
    return redirect(request.referrer or url_for('home'))

@app.route('/')
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template('home.html')

@app.route('/login', methods=['GET','POST'])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=='POST':
        username = request.form.get('username','').strip()
        password = request.form.get('password','')
        conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u); return redirect(url_for('home'))
        flash("Invalid username or password")
    return render_template('login.html')

@app.route('/register', methods=['GET','POST'])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=='POST':
        username = request.form.get('username','').strip()
        password = request.form.get('password','')
        conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close()
            flash("Registration successful. Please login.")
            return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close()
            flash("Username already taken")
    return render_template('register.html')

@app.route('/logout')
@login_required
def logout():
    logout_user(); return redirect(url_for('login'))

@app.route('/analyze')
@login_required
def analyze_page():
    return render_template('index.html')

@app.route('/dashboard')
@login_required
def dashboard():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    rows = cur.fetchall(); conn.close()
    disease_data = {'labels':[r[0] for r in rows], 'values':[r[1] for r in rows]}
    return render_template('dashboard.html', disease_data=disease_data)

@app.route('/metrics')
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f:
            training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

@app.route('/history')
@login_required
def history():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    cur.execute("SELECT * FROM predictions WHERE user_id=? ORDER BY timestamp DESC", (current_user.id,))
    rows = cur.fetchall(); conn.close()
    out=[]
    for r in rows:
        ts=r['timestamp']
        try:
            ts_dt = datetime.fromisoformat(ts)
        except:
            try:
                ts_dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S.%f")
            except:
                ts_dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + timedelta(hours=5, minutes=30)
        out.append({'image_path':r['image_path'], 'prediction':r['prediction'], 'confidence':r['confidence'], 'timestamp':ist.strftime("%b %d, %Y %I:%M %p")})
    return render_template('history.html', history=out)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route('/predict', methods=['POST'])
@login_required
def predict():
    global MODEL
    if MODEL is None:
        return "Model not loaded.", 500
    if 'file' not in request.files or request.files['file'].filename == '':
        return "No image uploaded.", 400
    f = request.files['file']
    if not allowed_file(f.filename):
        return "Invalid file type.", 400
    filename = secure_filename(f.filename)
    unique = f"{int(time.time())}_{filename}"
    save_path = os.path.join(UPLOAD_DIR, unique)
    f.save(save_path)

    # Preprocess image (resize, arr, normalize)
    try:
        img = keras_image.load_img(save_path, target_size=IMAGE_SIZE)
        arr = keras_image.img_to_array(img).astype('float32')
        if PREPROCESS_METHOD == "simple_rescale":
            arr = arr / 255.0
        else:
            arr = arr / 255.0
        arr = np.expand_dims(arr, 0)
        preds = MODEL.predict(arr)
        idx = int(np.argmax(preds))
        conf = float(np.max(preds) * 100.0)
        raw = CLASS_NAMES[idx] if idx < len(CLASS_NAMES) else str(idx)
        clean = CLEAN_NAMES[idx] if idx < len(CLEAN_NAMES) else raw

        # store in DB
        url = url_for('static', filename=f'uploads/{unique}')
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, timestamp) VALUES (?,?,?,?,?)",
                    (current_user.id, url, clean, conf, datetime.utcnow().isoformat()))
        conn.commit(); conn.close()

        info = disease_info.get(raw, disease_info['default'])
        lang = session.get('language','en')
        description = get_translation(info.get('description',''), lang)
        remedies = [get_translation(r, lang) for r in info.get('remedies', [])]
        if conf >= 85: conf_level='High'
        elif conf >=60: conf_level='Medium'
        else: conf_level='Low'

    except Exception as e:
        return f"Prediction error: {e}", 500

    return render_template('result.html', prediction=clean, confidence=round(conf,2), image_path=url, description=description, remedies=remedies, confidence_level=conf_level)

# 10) Run ngrok + flask
if __name__ == "__main__":
    # create public tunnel
    try:
        public_url = ngrok.connect(5000)
        print("🔗 ngrok tunnel:", public_url)
    except Exception as e:
        print("⚠️ ngrok error:", e)
    # run app
    app.run(host="0.0.0.0", port=5000, debug=True)


🔑 Paste your ngrok authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken). It will be hidden.
ngrok authtoken: ··········
⚠️ No model file found in expected paths. Upload best_crop_model.h5 to /mnt/data and re-run.
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [8]:
# --- FAST MODE FOR NGROK + FLASK ---
import time, threading
from pyngrok import ngrok

def start_ngrok_async(port=5000):
    ngrok.kill()  # clean previous tunnels
    time.sleep(1)
    print("🚀 Starting ngrok tunnel…")
    url = ngrok.connect(port, proto="http")
    print("🌍 Public URL:", url)
    return url

threading.Thread(target=start_ngrok_async, daemon=True).start()

print("⚡ Fast mode enabled — continue running your Flask app normally.")


⚡ Fast mode enabled — continue running your Flask app normally.


In [19]:
# Colab-ready: Full Modern UI Flask app with dual-model support & fast ngrok
# Run this entire cell in Google Colab.

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, time, threading, sqlite3, json, shutil, atexit
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, send_file
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import numpy as np
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as keras_image
from deep_translator import GoogleTranslator

# -------------------
# 2) Disable pyngrok update-check for faster startup
# -------------------
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# -------------------
# 3) Model candidate paths (you uploaded these earlier)
# -------------------
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}

# -------------------
# 4) Class names (keeps identical order used during training)
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot',
    'Pepper__bell___healthy',
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy',
    'Tomato_Bacterial_spot',
    'Tomato_Early_blight',
    'Tomato_Late_blight',
    'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot',
    'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot',
    'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus',
    'Tomato_healthy'
]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]

# -------------------
# 5) File system setup
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)

DB_PATH = os.path.join(BASE_DIR, "database.db")
ALLOWED_EXT = {"png","jpg","jpeg"}
IMAGE_SIZE = (128,128)   # model expected input
PREPROCESS = "rescale"   # simple normalization /255.0

# -------------------
# 6) Initialize SQLite DB
# -------------------
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL
                   )''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     timestamp TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id)
                   )''')
    conn.commit(); conn.close()
init_db()

# -------------------
# 7) Minimal modern templates (full UI)
# -------------------
layout_html = """<!doctype html><html lang="en"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><title>{{ title or 'AgroScan AI' }}</title></head><body class="bg-gray-50 text-gray-800"><nav class="bg-white/80 fixed w-full z-30 shadow"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24"><path d="M2 12l5-5 5 5-5 5-5-5z" fill="currentColor"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="space-x-4">{% if current_user.is_authenticated %}<span class="hidden md:inline text-gray-700">Welcome, {{ current_user.username }}!</span><a href='{{ url_for(\"dashboard\") }}' class='text-gray-600 hover:text-green-600'>Dashboard</a><a href='{{ url_for(\"analyze_page\") }}' class='text-gray-600 hover:text-green-600'>Analyze</a><a href='{{ url_for(\"history\") }}' class='text-gray-600 hover:text-green-600'>History</a><a href='{{ url_for(\"metrics\") }}' class='text-gray-600 hover:text-green-600'>Metrics</a><a href='{{ url_for(\"logout\") }}' class='text-blue-600 font-semibold'>Logout</a>{% else %}<a href='{{ url_for(\"login\") }}' class='text-gray-600 hover:text-green-600'>Login</a><a href='{{ url_for(\"register\") }}' class='text-gray-600 hover:text-green-600'>Register</a>{% endif %}<a href='{{ url_for(\"set_language\", lang=\"en\") }}' class='text-sm'>EN</a><a href='{{ url_for(\"set_language\", lang=\"hi\") }}' class='text-sm'>HI</a><a href='{{ url_for(\"set_language\", lang=\"ur\") }}' class='text-sm'>UR</a></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6'>© 2025 AgroScan AI</footer></body></html>"""
with open(os.path.join(TEMPLATES_DIR,"layout.html"), "w", encoding="utf-8") as f: f.write(layout_html)

home_html = """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-8 items-center'><div><h1 class='text-4xl font-bold'>Revolutionize Your Farming with AI</h1><p class='mt-4 text-gray-600'>Upload a leaf image and get instant disease detection, confidence score and remedies.</p><div class='mt-6'>{% if current_user.is_authenticated %}<a href='{{ url_for(\"analyze_page\") }}' class='bg-green-600 text-white px-4 py-2 rounded'>Get Started</a><a href='{{ url_for(\"history\") }}' class='ml-3 text-gray-600'>History</a>{% else %}<a href='{{ url_for(\"login\") }}' class='bg-blue-600 text-white px-4 py-2 rounded'>Login</a>{% endif %}</div></div><div><img src='{{ url_for(\"static\", filename=\"uploads/sample_leaf.png\") }}' class='rounded-lg shadow'></div></div>{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR,"home.html"), "w", encoding="utf-8") as f: f.write(home_html)

index_html = """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto bg-white p-6 rounded-xl shadow'><h2 class='text-2xl font-bold mb-4'>Upload a leaf image</h2><form id='upload-form' method='post' action='{{ url_for(\"predict\") }}' enctype='multipart/form-data'><label for='file-upload' class='block border-2 border-dashed p-8 rounded cursor-pointer hover:border-green-500'><input id='file-upload' name='file' type='file' accept='image/*' class='hidden'><div class='text-center'><p class='mt-2 text-gray-600'>Click to select or drag & drop an image (JPG/PNG)</p><p id='file-info' class='mt-2 text-sm text-gray-500'></p></div></label><div class='mt-4 flex space-x-3'><button id='submit-button' class='bg-green-600 text-white px-4 py-2 rounded'>Detect Disease</button><div id='loader' class='hidden items-center'><div class='animate-spin w-5 h-5 border-2 border-green-600 rounded-full mr-2'></div><span>Analyzing...</span></div></div></form></div><script>const fileUpload=document.getElementById('file-upload'), form=document.getElementById('upload-form'), fileInfo=document.getElementById('file-info'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); fileUpload.addEventListener('change', e=>{ const f=e.target.files[0]; if(f) fileInfo.textContent = `File: ${f.name}`; }); form.addEventListener('submit', e=>{ if(!fileUpload.files.length){ e.preventDefault(); alert('Please select an image to upload.'); } submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR,"index.html"), "w", encoding="utf-8") as f: f.write(index_html)

result_html = """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto bg-white p-6 rounded-xl shadow'><div class='md:flex md:gap-6'><div class='md:w-1/2'><img src='{{ image_path }}' class='rounded w-full object-cover' alt='uploaded'></div><div class='md:w-1/2'><h2 class='text-3xl font-bold text-green-700'>{{ prediction }}</h2><p class='mt-2 text-gray-600'>Confidence: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-6 mt-3 overflow-hidden' title='{{ confidence }}%'>{% if confidence_level == 'High' %}<div class='h-6 bg-green-500 text-white flex items-center justify-center font-bold' style='width: {{ confidence }}%;'>{{ confidence }}%</div>{% elif confidence_level == 'Medium' %}<div class='h-6 bg-yellow-500 text-white flex items-center justify-center font-bold' style='width: {{ confidence }}%;'>{{ confidence }}%</div>{% else %}<div class='h-6 bg-red-500 text-white flex items-center justify-center font-bold' style='width: {{ confidence }}%;'>{{ confidence }}%</div>{% endif %}</div><div class='mt-4'><h3 class='font-semibold'>About this condition</h3><p class='text-gray-600'>{{ description }}</p></div>{% if remedies %}<div class='mt-4 bg-green-50 p-4 rounded'><h4 class='font-semibold'>Suggested Actions</h4><ul class='list-disc pl-6'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}<div class='mt-4'><button id='correct-btn' class='bg-green-600 text-white px-3 py-1 rounded'>👍 Correct</button><button id='incorrect-btn' class='bg-red-600 text-white px-3 py-1 rounded'>👎 Incorrect</button><p id='feedback-thanks' class='hidden text-green-700 mt-2 font-semibold'>Thank you for your feedback!</p></div></div></div></div><script>const correctBtn=document.getElementById('correct-btn'), incorrectBtn=document.getElementById('incorrect-btn'), thanks=document.getElementById('feedback-thanks'); function handle(){ correctBtn.disabled=true; incorrectBtn.disabled=true; thanks.classList.remove('hidden'); } correctBtn.addEventListener('click', handle); incorrectBtn.addEventListener('click', handle);</script>{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR,"result.html"), "w", encoding="utf-8") as f: f.write(result_html)

history_html = """{% extends 'layout.html' %}{% block content %}<h2 class='text-2xl font-bold mb-4'>Prediction History</h2>{% if history %}<div class='grid md:grid-cols-3 gap-4'>{% for p in history %}<div class='bg-white rounded shadow overflow-hidden'><img src='{{ p.image_path }}' class='w-full h-40 object-cover'><div class='p-3'><div class='text-sm text-gray-500'>{{ p.timestamp }}</div><div class='font-semibold'>{{ p.prediction }}</div><div class='text-sm text-gray-600'>Confidence: {{ p.confidence }}%</div><div class='text-xs text-gray-500'>Model: {{ p.model_used }}</div></div></div>{% endfor %}</div>{% else %}<p>No saved predictions yet.</p>{% endif %}{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR,"history.html"), "w", encoding="utf-8") as f: f.write(history_html)

dashboard_html = """{% extends 'layout.html' %}{% block content %}<h2 class='text-2xl font-bold mb-4'>Dashboard</h2><div class='bg-white p-4 rounded shadow'>{% if disease_data.labels %}<ul>{% for i in range(disease_data.labels|length) %}<li>{{ disease_data.labels[i] }} — {{ disease_data.values[i] }}</li>{% endfor %}</ul>{% else %}<p>No detections yet. Analyze to populate your dashboard.</p>{% endif %}<div class='mt-4'><form method='post' action='{{ url_for(\"switch_model\") }}'><label class='block'>Select active model<select name='model' class='mt-2 p-2 border rounded'> <option value='best' {% if active_model=='best' %}selected{% endif %}>Best model (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>Fallback model (crop_disease_model.h5)</option></select></label><div class='mt-2'><button class='bg-blue-600 text-white px-3 py-1 rounded'>Switch Model</button></div></form></div></div>{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR,"dashboard.html"), "w", encoding="utf-8") as f: f.write(dashboard_html)

login_html = """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto bg-white p-6 rounded shadow'><h2 class='text-2xl font-bold mb-4'>Login</h2><form method='post' action='{{ url_for(\"login\") }}'><label class='block'>Username<input name='username' class='w-full p-2 border rounded mt-1' required></label><label class='block mt-3'>Password<input name='password' type='password' class='w-full p-2 border rounded mt-1' required></label><div class='mt-4'><button class='bg-green-600 text-white px-4 py-2 rounded'>Sign in</button></div></form><p class='mt-3 text-sm'>Not a member? <a href='{{ url_for(\"register\") }}' class='text-green-600'>Register here</a></p></div>{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR,"login.html"), "w", encoding="utf-8") as f: f.write(login_html)

register_html = """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto bg-white p-6 rounded shadow'><h2 class='text-2xl font-bold mb-4'>Register</h2><form method='post' action='{{ url_for(\"register\") }}'><label class='block'>Username<input name='username' class='w-full p-2 border rounded mt-1' required></label><label class='block mt-3'>Password<input name='password' type='password' class='w-full p-2 border rounded mt-1' required></label><div class='mt-4'><button class='bg-green-600 text-white px-4 py-2 rounded'>Register</button></div></form></div>{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR,"register.html"), "w", encoding="utf-8") as f: f.write(register_html)

metrics_html = """{% extends 'layout.html' %}{% block content %}<h2 class='text-2xl font-bold mb-4'>Training Metrics</h2><div class='bg-white p-4 rounded shadow'><p>If you upload training_results.json, confusion_matrix.png or classification_report.txt into the Colab root, they will be shown here.</p>{% if training_results %}<pre class='whitespace-pre-wrap bg-gray-50 p-2 rounded'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}{% if confusion_exists %}<h3 class='mt-3'>Confusion Matrix</h3><img src='{{ url_for('static', filename='confusion_matrix.png') }}' class='max-w-full'>{% endif %}{% if report_exists %}<h3 class='mt-3'>Classification Report</h3><pre class='bg-gray-50 p-2 rounded'>{{ report_text }}</pre>{% endif %}</div>{% endblock %}"""
with open(os.path.join(TEMPLATES_DIR,"metrics.html"), "w", encoding="utf-8") as f: f.write(metrics_html)

# Sample static image for hero
sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (900,600), (235,251,230))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# -------------------
# 8) Disease info (extendable)
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'Bacterial disease with dark water-soaked spots.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'Plant appears healthy.','remedies':[]},
    'Potato___Early_blight': {'description':'Target-like rings on leaves.', 'remedies':['Use certified seed','Apply fungicides','Rotate crops']},
    'Potato___Late_blight': {'description':'Large dark lesions; can be devastating.', 'remedies':['Remove infected plants','Protective fungicides']},
    'Tomato_healthy': {'description':'Healthy tomato plant','remedies':[]},
    'default': {'description':'Information not available.','remedies':['Consult local extension services']}
}

translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception as e:
        print("Translation failed:", e); return text

# -------------------
# 9) Flask app & login
# -------------------
app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass

@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_trans():
    def _(text):
        lang = session.get('language','en')
        return get_translation(text, lang)
    return dict(_=_)

# -------------------
# 10) Model management (lazy load both, support switching)
# -------------------
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"  # default; can be switched via dashboard

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

# Try to pre-load quickly in background thread (non-blocking)
def _background_preload():
    for k in ["best","fallback"]:
        if os.path.exists(MODEL_PATHS.get(k,"")):
            try:
                load_model_lazy(k)
            except Exception as e:
                print("Background preload error:", e)
threading.Thread(target=_background_preload, daemon=True).start()

# -------------------
# 11) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    session['language'] = lang
    return redirect(request.referrer or url_for('home'))

@app.route('/')
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template('home.html')

@app.route('/login', methods=['GET','POST'])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=='POST':
        username = request.form.get('username','').strip()
        password = request.form.get('password','')
        conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u); return redirect(url_for('home'))
        flash("Invalid username or password")
    return render_template('login.html')

@app.route('/register', methods=['GET','POST'])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=='POST':
        username = request.form.get('username','').strip()
        password = request.form.get('password','')
        conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash("Registration successful. Please login."); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash("Username already taken")
    return render_template('register.html')

@app.route('/logout')
@login_required
def logout():
    logout_user(); return redirect(url_for('login'))

@app.route('/analyze')
@login_required
def analyze_page():
    return render_template('index.html')

@app.route('/dashboard', methods=['GET'])
@login_required
def dashboard():
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    rows = cur.fetchall(); conn.close()
    disease_data = {'labels':[r[0] for r in rows], 'values':[r[1] for r in rows]}
    return render_template('dashboard.html', disease_data=disease_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        # lazy load chosen model
        load_model_lazy(m)
        flash(f"Active model switched to: {m}")
    else:
        flash("Unknown model key")
    return redirect(url_for('dashboard'))

@app.route('/metrics')
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

@app.route('/history')
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); conn.row_factory = sqlite3.Row; cur = conn.cursor()
    cur.execute("SELECT * FROM predictions WHERE user_id=? ORDER BY timestamp DESC", (current_user.id,))
    rows = cur.fetchall(); conn.close()
    out=[]
    for r in rows:
        ts = r['timestamp']
        try:
            ts_dt = datetime.fromisoformat(ts)
        except:
            try: ts_dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S.%f")
            except: ts_dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + timedelta(hours=5, minutes=30)
        out.append({'image_path':r['image_path'], 'prediction':r['prediction'], 'confidence':r['confidence'], 'timestamp':ist.strftime("%b %d, %Y %I:%M %p"), 'model_used': r['model_used']})
    return render_template('history.html', history=out)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route('/predict', methods=['POST'])
@login_required
def predict():
    global ACTIVE_MODEL_KEY
    # pick active model (lazy load if necessary)
    model = load_model_lazy(ACTIVE_MODEL_KEY)
    if model is None:
        return "Model not loaded. Upload model to /mnt/data and re-run.", 500

    if 'file' not in request.files or request.files['file'].filename == '':
        return "No image uploaded.", 400
    f = request.files['file']
    if not allowed_file(f.filename):
        return "Invalid file type.", 400

    filename = secure_filename(f.filename)
    unique = f"{int(time.time())}_{filename}"
    save_path = os.path.join(UPLOAD_DIR, unique)
    f.save(save_path)

    try:
        img = keras_image.load_img(save_path, target_size=IMAGE_SIZE)
        arr = keras_image.img_to_array(img).astype('float32')
        if PREPROCESS == "rescale":
            arr = arr / 255.0
        arr = np.expand_dims(arr, 0)

        preds = model.predict(arr)
        idx = int(np.argmax(preds))
        conf = float(np.max(preds) * 100.0)
        raw = CLASS_NAMES[idx] if idx < len(CLASS_NAMES) else str(idx)
        clean = CLEAN_NAMES[idx] if idx < len(CLEAN_NAMES) else raw

        url = url_for('static', filename=f'uploads/{unique}')
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("INSERT INTO predictions (user_id, image_path, model_used, prediction, confidence, timestamp) VALUES (?,?,?,?,?,?)",
                    (current_user.id, url, ACTIVE_MODEL_KEY, clean, conf, datetime.utcnow().isoformat()))
        conn.commit(); conn.close()

        info = disease_info.get(raw, disease_info['default'])
        lang = session.get('language','en')
        desc = get_translation(info.get('description',''), lang)
        remedies = [get_translation(r, lang) for r in info.get('remedies', [])]
        if conf >= 85: conf_level = 'High'
        elif conf >= 60: conf_level = 'Medium'
        else: conf_level = 'Low'

    except Exception as e:
        return f"Prediction error: {e}", 500

    return render_template('result.html', prediction=clean, confidence=round(conf,2), image_path=url, description=desc, remedies=remedies, confidence_level=conf_level)

# -------------------
# 12) ngrok fast-start utility
# -------------------
def start_ngrok_in_background(port=5000):
    try:
        ngrok.kill()
    except Exception:
        pass
    def _start():
        try:
            public_url = ngrok.connect(port, proto="http")
            print("🔗 ngrok tunnel:", public_url)
        except Exception as e:
            print("⚠️ ngrok error:", e)
    t = threading.Thread(target=_start, daemon=True); t.start()

# Ensure ngrok is killed at exit
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

# -------------------
# 13) Start (prints instructions)
# -------------------
if __name__ == "__main__":
    print("Colab Flask app (dual-model) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("Uploading additional files? put them into /mnt/data and re-run this cell.")
    print("You will be asked for your ngrok authtoken in the prompt below (paste and press Enter).")
    token = getpass("ngrok authtoken (hidden): ")
    if token and token.strip():
        try:
            ngrok.set_auth_token(token.strip())
        except Exception as e:
            print("ngrok token set error:", e)
    # start ngrok quickly in background
    start_ngrok_in_background(5000)
    # run flask
    app.run(host="0.0.0.0", port=5000, debug=True)


Colab Flask app (dual-model) starting...
Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)
Uploading additional files? put them into /mnt/data and re-run this cell.
You will be asked for your ngrok authtoken in the prompt below (paste and press Enter).
[model] Loading 'best' from best_crop_model.h5 ...
[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ngrok authtoken (hidden): ··········
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"


In [1]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINALIZED SCRIPT
# =========================================================================
#  -- Version 18: FIX APPLIED: EfficientNet Preprocessing Replication --
# =========================================================================

# --------------------------------------------------------------------
# Part 1: Setup and File Creation
# --------------------------------------------------------------------

# Step 1.1: Install necessary libraries
print("⏳ Installing required libraries...")
# Added Pillow explicitly for image loading stability
!pip install -q pyngrok tensorflow flask deep-translator flask-login Pillow
print("✅ Libraries installed.")

# Step 1.2: Import all dependencies
import os
import numpy as np
import getpass
import time
import sqlite3
import json
from flask import Flask, request, render_template, url_for, session, redirect, flash
from werkzeug.security import generate_password_hash, check_password_hash
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from pyngrok import ngrok
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from deep_translator import GoogleTranslator
from datetime import datetime, timedelta
# IMPORT THE CRITICAL PREPROCESSING FUNCTION
from tensorflow.keras.applications.efficientnet import preprocess_input

# --------------------------------------------------------------------
# Part 2: Database and User Authentication Setup
# --------------------------------------------------------------------

# Step 2.1: Initialize the database
def init_db():
    conn = sqlite3.connect('database.db')
    print("Opened database successfully")
    # User table
    conn.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            password_hash TEXT NOT NULL
        );
    ''')
    # Predictions history table
    conn.execute('''
        CREATE TABLE IF NOT EXISTS predictions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            image_path TEXT NOT NULL,
            prediction TEXT NOT NULL,
            confidence REAL NOT NULL,
            timestamp DATETIME NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users (id)
        );
    ''')
    print("Tables created successfully")
    conn.close()

# Call the function to ensure DB is ready
init_db()

# Step 2.2: Ngrok Authentication
print("\n🔑 Please provide your ngrok authtoken.")
print("You can get it from https://dashboard.ngrok.com/get-started/your-authtoken")
try:
    authtoken = getpass.getpass()
    ngrok.set_auth_token(authtoken)
    print("✅ ngrok authtoken configured successfully.")
except:
    print("⚠️ Ngrok setup skipped (no authtoken provided).")

# Step 2.3: Create necessary folders
os.makedirs('templates', exist_ok=True)
os.makedirs('static/uploads', exist_ok=True)

# --------------------------------------------------------------------
# Part 3: Define HTML Content (Using the fixed history template)
# --------------------------------------------------------------------

# --- Layout ---
layout_html_content = """
<!DOCTYPE html>
{% set lang = session.get('language', 'en') %}
<html lang="{{ lang }}" {% if lang == 'ur' %}dir="rtl"{% endif %} class="scroll-smooth">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{{ _(title) }} - AgroScan AI</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;700&family=Noto+Nastaliq+Urdu:wght@400;700&family=Noto+Sans+Devanagari:wght@400;700&family=Noto+Sans+Tamil:wght@400;700&family=Noto+Sans+Bengali:wght@400;700&display=swap" rel="stylesheet">
    <style>
        body { font-family: 'Inter', sans-serif; }
        html[lang="ur"] body { font-family: 'Noto Nastaliq Urdu', sans-serif; }
        html[lang="hi"] body, html[lang="mr"] body, html[lang="gu"] body { font-family: 'Noto Sans Devanagari', sans-serif; }
        html[lang="ta"] body { font-family: 'Noto Sans Tamil', sans-serif; }
        html[lang="bn"] body { font-family: 'Noto Sans Bengali', sans-serif; }
        .transition-all { transition: all 0.3s ease-in-out; }
        .fade-in { animation: fadeIn 0.5s ease-in-out; }
        @keyframes fadeIn { from { opacity: 0; transform: translateY(10px); } to { opacity: 1; transform: translateY(0); } }
    </style>
</head>
<body class="bg-gray-50 text-gray-800">
    <nav class="bg-white/80 backdrop-blur-lg fixed top-0 left-0 right-0 z-20 shadow-md">
        <div class="max-w-6xl mx-auto px-4">
            <div class="flex justify-between items-center h-16">
                <a href="{{ url_for('home') }}" class="flex items-center space-x-2">
                     <svg class="h-8 w-8 text-green-600" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24" stroke-width="1.5" stroke="currentColor"><path stroke-linecap="round" stroke-linejoin="round" d="M2.25 15.75l5.159-5.159a2.25 2.25 0 013.182 0l5.159 5.159m-1.5-1.5l1.409-1.409a2.25 2.25 0 013.182 0l2.909 2.909m-18 3.75h16.5a1.5 1.5 0 001.5-1.5V6a1.5 1.5 0 00-1.5-1.5H3.75A1.5 1.5 0 002.25 6v12a1.5 1.5 0 001.5 1.5zm10.5-11.25h.008v.008h-.008V8.25zm.375 0a.375.375 0 11-.75 0 .375.375 0 01.75 0z" /></svg>
                    <span class="text-2xl font-bold text-gray-800">AgroScan AI</span>
                </a>
                <div class="flex items-center space-x-4">
                    <div class="hidden md:flex items-center space-x-6">
                        {% if current_user.is_authenticated %}
                            <span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span>
                            <a href="{{ url_for('dashboard') }}" class="text-gray-600 hover:text-green-600 font-medium transition-all">{{ _('Dashboard') }}</a>
                            <a href="{{ url_for('analyze_page') }}" class="text-gray-600 hover:text-green-600 font-medium transition-all">{{ _('Analyze') }}</a>
                            <a href="{{ url_for('history') }}" class="text-gray-600 hover:text-green-600 font-medium transition-all">{{ _('History') }}</a>
                            <a href="{{ url_for('resources') }}" class="text-gray-600 hover:text-green-600 font-medium transition-all">{{ _('Resources') }}</a>
                            <a href="{{ url_for('logout') }}" class="text-blue-600 hover:text-blue-800 font-bold transition-all">{{ _('Logout') }}</a>
                        {% else %}
                             <a href="{{ url_for('login') }}" class="text-gray-600 hover:text-green-600 font-medium transition-all">{{ _('Login') }}</a>
                             <a href="{{ url_for('register') }}" class="text-gray-600 hover:text-green-600 font-medium transition-all">{{ _('Register') }}</a>
                        {% endif %}
                    </div>
                    <div class="relative" x-data="{ open: false }" @click.away="open = false">
                        <button @click="open = !open" class="flex items-center p-2 rounded-lg hover:bg-gray-200">
                           <svg class="h-5 w-5" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 20 20" fill="currentColor"><path d="M7.75 2.75a.75.75 0 00-1.5 0v1.258a32.987 32.987 0 00-3.599.278.75.75 0 10.198 1.487A31.545 31.545 0 018.7 5.545 19.38 19.38 0 017 9.762V11.5a.75.75 0 001.5 0V9.762c0-.683.085-1.35.25-1.994.169-.65.4-1.28.68-1.86A7.94 7.94 0 0112 6.561v4.939a.75.75 0 101.5 0V6.561a7.94 7.94 0 012.07 1.407c.28.58.512 1.21.68 1.86.165.643.25 1.311.25 1.994V11.5a.75.75 0 001.5 0V9.762a19.38 19.38 0 01-1.7 4.217 31.545 31.545 0 014.101-.987.75.75 0 00.198-1.487 32.987 32.987 0 00-3.599-.278V2.75a.75.75 0 00-1.5 0v1.439a7.94 7.94 0 01-3.12 1.548A7.94 7.94 0 019.25 4.19V2.75zM8.5 13.25a.75.75 0 00-1.5 0v2a.75.75 0 001.5 0v-2z" /></svg>
                        </button>
                        <div x-show="open" class="absolute right-0 mt-2 w-48 bg-white rounded-md shadow-lg z-30" style="display: none;">
                            <a href="/language/en" class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">English</a>
                            <a href="/language/hi" class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">हिन्दी (Hindi)</a>
                            <a href="/language/mr" class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">मराठी (Marathi)</a>
                            <a href="/language/bn" class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">বাংলা (Bengali)</a>
                            <a href="/language/ta" class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">தமிழ் (Tamil)</a>
                            <a href="/language/te" class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">తెలుగు (Telugu)</a>
                            <a href="/language/gu" class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">ગુજરાતી (Gujarati)</a>
                            <a href="/language/ur" class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">اردو (Urdu)</a>
                        </div>
                    </div>
                </div>
            </div>
        </div>
    </nav>
    <main class="pt-16">
        {% block content %}{% endblock %}
    </main>
    <footer class="bg-white border-t border-gray-200 mt-12">
        <div class="max-w-6xl mx-auto py-6 px-4 text-center text-gray-500">
            <p>{{ _('© 2025 AgroScan AI. A Student Project powered by Google AI.') }}</p>
        </div>
    </footer>
    <script src="https://cdn.jsdelivr.net/gh/alpinejs/alpine@v2.x.x/dist/alpine.min.js" defer></script>
</body>
</html>
"""
# --- History page (with the definitive confidence score fix) ---
history_html_content = """
{% extends "layout.html" %}
{% block content %}
<div class="max-w-6xl mx-auto px-4 py-8 fade-in">
    <h1 class="text-4xl font-bold mb-6">{{ _('Prediction History') }}</h1>

    {% if history %}
        <div class="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-8">
            {% for p in history %}
                <div class="bg-white rounded-lg shadow-md overflow-hidden">
                    <img src="{{ p.image_path }}" alt="Uploaded leaf" class="w-full h-48 object-cover">
                    <div class="p-4">
                        <p class="text-sm text-gray-500">{{ p.timestamp.strftime('%B %d, %Y %I:%M %p') }} (IST)</p>
                        <h3 class="text-xl font-bold mt-1">{{ p.prediction }}</h3>
                        <p class="text-md text-gray-600 mt-1">{{ _('Confidence') }}: <span class="font-semibold">{{ p.confidence | int }}%</span></p>
                    </div>
                </div>
            {% endfor %}
        </div>
    {% else %}
        <div class="text-center py-16">
            <p class="text-xl text-gray-500">{{ _('No predictions have been saved yet.') }}</p>
            <a href="{{ url_for('analyze_page') }}" class="mt-4 inline-block bg-green-600 text-white font-bold py-3 px-6 rounded-lg hover:bg-green-700 transition-all">{{ _('Make your first analysis!') }}</a>
        </div>
    {% endif %}
</div>
{% endblock %}
"""
# --- Other HTML templates remain the same for brevity ---
home_html_content="""{% extends "layout.html" %}{% block content %}<div class="fade-in"><div class="bg-gradient-to-br from-green-100 to-cyan-100"><div class="max-w-6xl mx-auto px-4 py-20 text-center"><h1 class="text-5xl md:text-6xl font-bold text-gray-800">{{ _('Revolutionize Your Farming with AI') }}</h1><p class="mt-4 text-lg text-gray-600 max-w-2xl mx-auto">{{ _('Instantly detect crop diseases with a simple photo. Get actionable advice to protect your harvest and increase your yield.') }}</p><a href="{{ url_for('analyze_page') }}" class="mt-8 inline-block bg-green-600 text-white font-bold py-3 px-8 rounded-lg hover:bg-green-700 transition-all shadow-lg">{{ _('Get Started Now') }}</a></div></div><div class="py-16 bg-gray-50"><div class="max-w-6xl mx-auto px-4 text-center"><h2 class="text-4xl font-bold text-gray-800">{{ _('How It Works') }}</h2><p class="mt-2 text-gray-600">{{ _('A simple, three-step process to safeguard your crops.') }}</p><div class="mt-12 grid md:grid-cols-3 gap-12"><div class="bg-white p-8 rounded-xl shadow-md"><div class="text-5xl mb-4">📸</div><h3 class="text-2xl font-bold mb-2">{{ _('1. Upload Image') }}</h3><p>{{ _('Take a clear picture of the plant leaf showing symptoms and upload it to our tool.') }}</p></div><div class="bg-white p-8 rounded-xl shadow-md"><div class="text-5xl mb-4">🧠</div><h3 class="text-2xl font-bold mb-2">{{ _('2. AI Analysis') }}</h3><p>{{ _('Our advanced AI model analyzes the image against a vast database of plant diseases.') }}</p></div><div class="bg-white p-8 rounded-xl shadow-md"><div class="text-5xl mb-4">📋</div><h3 class="text-2xl font-bold mb-2">{{ _('3. Get Results') }}</h3><p>{{ _('Receive an instant diagnosis, confidence score, and suggested actions to take.') }}</p></div></div></div></div></div>{% endblock %}"""
about_html_content="""{% extends "layout.html" %}{% block content %}<div class="bg-white/50 fade-in"><div class="max-w-4xl mx-auto px-4 py-16"><h1 class="text-4xl font-bold text-center mb-8">{{ _('About AgroScan AI') }}</h1><div class="text-lg text-gray-700 space-y-6"><p>{{ _('AgroScan AI is an innovative student project designed to empower farmers and gardening enthusiasts by leveraging the power of Artificial Intelligence. Our mission is to make crop disease detection accessible, affordable, and instantaneous.') }}</p><p>{{ _('By using a sophisticated Convolutional Neural Network (CNN) trained on the extensive PlantVillage dataset, our tool can identify various diseases across multiple crops like tomatoes, potatoes, and peppers with a high degree of accuracy.') }}</p><h2 class="text-2xl font-bold pt-4">{{ _('Our Technology') }}</h2><p>{{ _('This application is built using a modern tech stack:') }}</p><ul class="list-disc list-inside space-y-2"><li><strong>{{ _('Backend') }}:</strong> Flask (a lightweight Python web framework)</li><li><strong>{{ _('Machine Learning') }}:</strong> TensorFlow and Keras for building and deploying the CNN model.</li><li><strong>{{ _('Frontend') }}:</strong> TailwindCSS for a responsive and modern user interface.</li><li><strong>{{ _('Deployment') }}:</strong> Served via Ngrok for easy access from a Google Colab environment.</li></ul></div></div></div>{% endblock %}"""
index_html_content="""{% extends "layout.html" %}{% block content %}<div class="bg-gradient-to-br from-green-50 to-cyan-100 flex items-center justify-center min-h-screen -mt-16"><div class="bg-white rounded-2xl shadow-xl p-8 max-w-lg w-full fade-in"><div class="text-center mb-8"><h1 class="text-4xl font-bold">{{ _('Analysis Tool') }}</h1><p class="text-gray-600 mt-2">{{ _('Upload a leaf image to begin diagnosis.') }}</p></div><form action="{{ url_for('predict') }}" method="post" enctype="multipart/form-data" id="upload-form"><div class="mb-6"><label for="file-upload" class="cursor-pointer group"><div id="upload-area" class="w-full px-4 py-12 border-2 border-dashed border-gray-300 rounded-lg text-gray-500 hover:border-green-500 hover:text-green-600 transition-all text-center"><img id="image-preview" class="hidden mx-auto max-h-48 rounded-lg mb-4" /><div id="upload-text"><svg class="mx-auto h-12 w-12" stroke="currentColor" fill="none" viewBox="0 0 48 48"><path d="M28 8H12a4 4 0 00-4 4v20m32-12v8m0 0v8a4 4 0 01-4 4H12a4 4 0 01-4-4v-4m32-4l-3.172-3.172a4 4 0 00-5.656 0L28 28M8 32l9.172-9.172a4 4 0 015.656 0L28 28m0 0l4 4m4-24h8m-4-4v8" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"></path></svg><span class="mt-2 block text-sm font-medium">{{ _('Click to upload or drag & drop') }}</span></div><p id="file-info" class="mt-2 text-sm font-medium"></p></div></label><input id="file-upload" name="file" type="file" class="sr-only" accept="image/png, image/jpeg, image/jpg"></div><button type="submit" id="submit-button" class="w-full flex items-center justify-center bg-green-600 text-white font-bold py-3 px-4 rounded-lg hover:bg-green-700 transition-all">{{ _('Detect Disease') }}</button><div id="loader" class="hidden w-full flex items-center justify-center"><div class="animate-spin rounded-full h-8 w-8 border-b-2 border-green-600"></div><span class="ml-4 font-medium">{{ _('Analyzing...') }}</span></div></form></div></div><script>const form=document.getElementById("upload-form"),fileUpload=document.getElementById("file-upload"),imagePreview=document.getElementById("image-preview"),uploadText=document.getElementById("upload-text"),fileInfo=document.getElementById("file-info"),submitButton=document.getElementById("submit-button"),loader=document.getElementById("loader");fileUpload.addEventListener("change",function(e){const t=e.target.files[0];if(t){if(!t.type.startsWith("image/")){return alert("{{ _('Invalid file type. Please upload an image (JPG, PNG).') }}"),void(fileUpload.value="");}const e=new FileReader;e.onload=function(e){imagePreview.src=e.target.result,imagePreview.classList.remove("hidden"),uploadText.classList.add("hidden"),fileInfo.textContent=`{{ _('File') }}: ${t.name}`},e.readAsDataURL(t)}}),form.addEventListener("submit",function(e){0===fileUpload.files.length&&(e.preventDefault(),alert("{{ _('Please select an image to upload.') }}")),submitButton.classList.add("hidden"),loader.classList.remove("hidden")});</script>{% endblock %}"""
result_html_content="""{% extends "layout.html" %}{% block content %}<div class="bg-gradient-to-br from-green-50 to-cyan-100 flex items-center justify-center min-h-screen -mt-16 py-12"><div class="bg-white rounded-2xl shadow-xl p-8 max-w-4xl w-full fade-in"><h1 class="text-4xl font-bold text-center mb-8">{{ _('Detection Result') }}</h1><div class="grid md:grid-cols-2 gap-8 items-start"><div class="space-y-6"><img src="{{ image_path }}" alt="Uploaded Leaf Image" class="rounded-lg w-full shadow-md border-4 border-white"><div class="bg-gray-50 rounded-lg p-6 space-y-4 text-center"><div><p class="text-lg text-gray-600 mb-1">{{ _('Predicted Condition') }}:</p><h2 class="text-3xl font-bold text-green-700">{{ prediction }}</h2></div><div><p class="text-md text-gray-600 mb-2">{{ _('Confidence Score') }}:</p><div class="w-full bg-gray-200 rounded-full h-6" title="{{ confidence_level }} {{ _('Confidence') }}">{% if confidence_level == 'High' %}<div class="bg-green-500 h-6 rounded-full text-center text-white font-bold flex items-center justify-center" style="width: {{ confidence }}%;"><span>{{ confidence }}%</span></div>{% elif confidence_level == 'Medium' %}<div class="bg-yellow-500 h-6 rounded-full text-center text-white font-bold flex items-center justify-center" style="width: {{ confidence }}%;"><span>{{ confidence }}%</span></div>{% else %}<div class="bg-red-500 h-6 rounded-full text-center text-white font-bold flex items-center justify-center" style="width: {{ confidence }}%;"><span>{{ confidence }}%</span></div>{% endif %}</div>{% if confidence_level == 'Medium' %}<p class="text-xs text-yellow-600 mt-2">{{ _('Prediction confidence is medium. For best results, consider re-taking the photo in better light.') }}</p>{% elif confidence_level == 'Low' %}<p class="text-xs text-red-600 mt-2"><b>{{ _('Warning') }}:</b> {{ _('Low confidence. The model is uncertain. Please consult a professional for an accurate diagnosis.') }}</p>{% endif %}</div></div></div><div class="space-y-6"><div class="text-left bg-blue-50 p-6 rounded-lg"><h3 class="text-2xl font-bold mb-3">{{ _('About this Condition') }}</h3><p>{{ description }}</p></div>{% if remedies %}<div class="text-left bg-green-50 p-6 rounded-lg"><h3 class="text-2xl font-bold mb-3">{{ _('Suggested Actions') }}</h3><ul class="list-disc list-inside space-y-2">{% for remedy in remedies %}<li>{{ remedy }}</li>{% endfor %}</ul></div>{% endif %}<div id="feedback-section" class="text-center bg-gray-100 p-4 rounded-lg"><p class="font-medium mb-2">{{ _('Was this diagnosis helpful?') }}</p><div class="flex justify-center space-x-4"><button id="correct-btn" class="px-4 py-2 bg-green-500 text-white rounded-lg hover:bg-green-600 transition-colors">👍 {{ _('Correct') }}</button><button id="incorrect-btn" class="px-4 py-2 bg-red-500 text-white rounded-lg hover:bg-red-600 transition-colors">👎 {{ _('Incorrect') }}</button></div><p id="feedback-thanks" class="hidden mt-2 text-green-700 font-semibold">{{ _('Thank you for your feedback!') }}</p></div></div></div><div class="mt-8 text-center"><a href="{{ url_for('analyze_page') }}" class="inline-block bg-gray-600 text-white font-bold py-3 px-6 rounded-lg hover:bg-gray-700 transition-all">{{ _('Analyze Another Image') }}</a></div></div></div><script>const correctBtn=document.getElementById("correct-btn"),incorrectBtn=document.getElementById("incorrect-btn"),thanksMsg=document.getElementById("feedback-thanks");function handleFeedback(){correctBtn.disabled=!0,incorrectBtn.disabled=!0,correctBtn.classList.add("opacity-50","cursor-not-allowed"),incorrectBtn.classList.add("opacity-50","cursor-not-allowed"),thanksMsg.classList.remove("hidden")}correctBtn.addEventListener("click",handleFeedback),incorrectBtn.addEventListener("click",handleFeedback);</script>{% endblock %}"""
resources_html_content="""{% extends "layout.html" %}{% block content %}<div class="max-w-4xl mx-auto px-4 py-16 fade-in"><h1 class="text-4xl font-bold text-center mb-8">{{ _('Agricultural Resources') }}</h1><div class="space-y-8"><div class="bg-white p-6 rounded-lg shadow-md"><h2 class="text-2xl font-bold mb-3">{{ _('Government Portals & Schemes (India)') }}</h2><ul class="list-disc list-inside space-y-2 text-blue-600"><li><a href="https://farmer.gov.in/" target="_blank" class="hover:underline">{{ _('Farmer Portal of India') }}</a></li><li><a href="https://kvk.icar.gov.in/" target="_blank" class="hover:underline">{{ _('Krishi Vigyan Kendra (KVK) Portal') }}</a></li><li><a href="https://mkisan.gov.in/" target="_blank" class="hover:underline">{{ _('mKisan Portal') }}</a></li><li><a href="https://www.e-nam.gov.in/" target="_blank" class="hover:underline">{{ _('e-National Agriculture Market (eNAM)') }}</a></li></ul></div><div class="bg-white p-6 rounded-lg shadow-md"><h2 class="text-2xl font-bold mb-3">{{ _('Knowledge & Research Institutions') }}</h2><ul class="list-disc list-inside space-y-2 text-blue-600"><li><a href="https://www.icar.org.in/" target="_blank" class="hover:underline">{{ _('Indian Council of Agricultural Research (ICAR)') }}</a></li><li><a href="https://iari.res.in/" target="_blank" class="hover:underline">{{ _('Indian Agricultural Research Institute (IARI)') }}</a></li></ul></div><div class="bg-white p-6 rounded-lg shadow-md"><h2 class="text-2xl font-bold mb-3">{{ _('Weather & Forecast') }}</h2><ul class="list-disc list-inside space-y-2 text-blue-600"><li><a href="https://mausam.imd.gov.in/" target="_blank" class="hover:underline">{{ _('India Meteorological Department (IMD)') }}</a></li></ul></div></div></div>{% endblock %}"""
login_html_content="""{% extends "layout.html" %}{% block content %}<div class="min-h-screen flex items-center justify-center bg-gray-50 -mt-16"><div class="max-w-md w-full bg-white p-8 rounded-lg shadow-md"><h2 class="text-3xl font-bold text-center text-gray-900">{{ _('Login to Your Account') }}</h2>{% with messages = get_flashed_messages() %}{% if messages %}<div class="bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative mt-4" role="alert"><strong class="font-bold">{{ messages[0] }}</strong></div>{% endif %}{% endwith %}<form method="post" class="mt-8 space-y-6"><div><label for="username" class="sr-only">{{ _('Username') }}</label><input id="username" name="username" type="text" required class="appearance-none rounded-md relative block w-full px-3 py-2 border border-gray-300 placeholder-gray-500 text-gray-900 focus:outline-none focus:ring-green-500 focus:border-green-500" placeholder="{{ _('Username') }}"></div><div><label for="password" class="sr-only">{{ _('Password') }}</label><input id="password" name="password" type="password" required class="appearance-none rounded-md relative block w-full px-3 py-2 border border-gray-300 placeholder-gray-500 text-gray-900 focus:outline-none focus:ring-green-500 focus:border-green-500" placeholder="{{ _('Password') }}"></div><div><button type="submit" class="group relative w-full flex justify-center py-2 px-4 border border-transparent text-sm font-medium rounded-md text-white bg-green-600 hover:bg-green-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-green-500">{{ _('Sign in') }}</button></div></form><p class="mt-4 text-center text-sm text-gray-600">{{ _('Not a member?') }} <a href="{{ url_for('register') }}" class="font-medium text-green-600 hover:text-green-500">{{ _('Register here') }}</a></p></div></div>{% endblock %}"""
register_html_content="""{% extends "layout.html" %}{% block content %}<div class="min-h-screen flex items-center justify-center bg-gray-50 -mt-16"><div class="max-w-md w-full bg-white p-8 rounded-lg shadow-md"><h2 class="text-3xl font-bold text-center text-gray-900">{{ _('Create an Account') }}</h2>{% with messages = get_flashed_messages() %}{% if messages %}<div class="bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative mt-4" role="alert"><strong class="font-bold">{{ messages[0] }}</strong></div>{% endif %}{% endwith %}<form method="post" class="mt-8 space-y-6"><div><label for="username" class="sr-only">{{ _('Username') }}</label><input id="username" name="username" type="text" required class="appearance-none rounded-md relative block w-full px-3 py-2 border border-gray-300 placeholder-gray-500 text-gray-900 focus:outline-none focus:ring-green-500 focus:border-green-500" placeholder="{{ _('Choose a username') }}"></div><div><label for="password" class="sr-only">{{ _('Password') }}</label><input id="password" name="password" type="password" required class="appearance-none rounded-md relative block w-full px-3 py-2 border border-gray-300 placeholder-gray-500 text-gray-900 focus:outline-none focus:ring-green-500 focus:border-green-500" placeholder="{{ _('Create a password') }}"></div><div><button type="submit" class="group relative w-full flex justify-center py-2 px-4 border border-transparent text-sm font-medium rounded-md text-white bg-green-600 hover:bg-green-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-green-500">{{ _('Register') }}</button></div></form><p class="mt-4 text-center text-sm text-gray-600">{{ _('Already a member?') }} <a href="{{ url_for('login') }}" class="font-medium text-green-600 hover:text-green-500">{{ _('Login here') }}</a></p></div></div>{% endblock %}"""
dashboard_html_content="""{% extends "layout.html" %}{% block content %}<div class="max-w-6xl mx-auto px-4 py-8 fade-in"><h1 class="text-4xl font-bold mb-8">{{ _('Your Personal Dashboard') }}</h1>{% if disease_data.labels %}<div class="grid grid-cols-1 lg:grid-cols-2 gap-8"><div class="bg-white p-6 rounded-lg shadow-md"><h2 class="text-2xl font-bold mb-4 text-center">{{ _('Disease Detections') }}</h2><div class="h-80 w-full"><canvas id="diseaseChart"></canvas></div></div><div class="bg-white p-6 rounded-lg shadow-md"><h2 class="text-2xl font-bold mb-4 text-center">{{ _('Monthly Scan Activity') }}</h2><div class="h-80 w-full"><canvas id="monthlyChart"></canvas></div></div></div>{% else %}<div class="text-center py-16"><p class="text-xl text-gray-500">{{ _('No data available to display. Start analyzing to see your dashboard!') }}</p><a href="{{ url_for('analyze_page') }}" class="mt-4 inline-block bg-green-600 text-white font-bold py-3 px-6 rounded-lg hover:bg-green-700 transition-all">{{ _('Make your first analysis!') }}</a></div>{% endif %}</div><script src="https://cdn.jsdelivr.net/npm/chart.js"></script><script>const diseaseData={{ disease_data | tojson }},monthlyData={{ monthly_data | tojson }};if(diseaseData.labels.length>0){const e=document.getElementById("diseaseChart").getContext("2d");new Chart(e,{type:"pie",data:{labels:diseaseData.labels.map(e=>e.replace(/_|-/g," ")),datasets:[{label:"{{ _('Detections') }}",data:diseaseData.values,backgroundColor:["rgba(34, 197, 94, 0.6)","rgba(239, 68, 68, 0.6)","rgba(234, 179, 8, 0.6)","rgba(59, 130, 246, 0.6)","rgba(168, 85, 247, 0.6)","rgba(236, 72, 153, 0.6)"],borderColor:["rgba(34, 197, 94, 1)","rgba(239, 68, 68, 1)","rgba(234, 179, 8, 1)","rgba(59, 130, 246, 1)","rgba(168, 85, 247, 1)","rgba(236, 72, 153, 1)"],borderWidth:1}]},options:{responsive:!0,maintainAspectRatio:!1,plugins:{legend:{position:"top"}}}})}if(monthlyData.labels.length>0){const e=document.getElementById("monthlyChart").getContext("2d");new Chart(e,{type:"bar",data:{labels:monthlyData.labels,datasets:[{label:"{{ _('Number of Scans') }}",data:monthlyData.values,backgroundColor:"rgba(16, 185, 129, 0.5)",borderColor:"rgba(16, 185, 129, 1)",borderWidth:1}]},options:{responsive:!0,maintainAspectRatio:!1,scales:{y:{beginAtZero:!0,ticks:{stepSize:1}}}}})}</script>{% endblock %}"""

with open("templates/layout.html", "w") as f: f.write(layout_html_content)
with open("templates/home.html", "w") as f: f.write(home_html_content)
with open("templates/about.html", "w") as f: f.write(about_html_content)
with open("templates/index.html", "w") as f: f.write(index_html_content)
with open("templates/result.html", "w") as f: f.write(result_html_content)
with open("templates/history.html", "w") as f: f.write(history_html_content)
with open("templates/resources.html", "w") as f: f.write(resources_html_content)
with open("templates/login.html", "w") as f: f.write(login_html_content)
with open("templates/register.html", "w") as f: f.write(register_html_content)
with open("templates/dashboard.html", "w") as f: f.write(dashboard_html_content)
print("✅ All 10 HTML template files created successfully.")


# --------------------------------------------------------------------
# Part 4: Flask App Backend
# --------------------------------------------------------------------

# Step 4.1: App Initialization
app = Flask(__name__)
app.secret_key = os.urandom(24)

# Step 4.2: Flask-Login Configuration
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
class User(UserMixin):
    def __init__(self, id, username):
        self.id = id
        self.username = username
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect('database.db')
    curs = conn.cursor()
    curs.execute("SELECT * FROM users WHERE id = ?", (user_id,))
    user_data = curs.fetchone()
    conn.close()
    if user_data:
        return User(id=user_data[0], username=user_data[1])
    return None

# --- Disease Information & Translation Setup ---
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description': 'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies': ['Apply copper-based fungicides preventatively.', 'Ensure good air circulation and avoid overhead watering.', 'Practice crop rotation with non-host plants.', 'Remove and destroy infected plant debris.']},
    'Pepper__bell___healthy': {'description': 'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.', 'remedies': []},
    'Potato___Early_blight': {'description': 'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies': ['Use certified disease-free potato seeds.', 'Apply fungicides containing mancozeb or chlorothalonil.', 'Practice crop rotation and remove volunteer potato plants.', 'Maintain adequate plant nutrition, especially nitrogen.']},
    'Potato___Late_blight': {'description': 'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside. It can rapidly destroy entire crops.', 'remedies': ['Apply fungicides proactively, especially during cool, moist weather.', 'Destroy infected plants and cull piles immediately.', 'Ensure good drainage and air circulation.', 'Use resistant potato varieties if available.']},
    'Potato___healthy': {'description': 'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.', 'remedies': []},
    'Tomato_Bacterial_spot': {'description': 'Causes small, water-soaked spots on tomato leaves and fruit. The spots can merge, causing leaves to yellow and die, and fruit to become unmarketable.', 'remedies': ['Avoid working in the garden when plants are wet.', 'Use copper-based bactericides.', 'Mulch around the base of plants to prevent soil splash.', 'Sanitize garden tools between uses.']},
    'Tomato_Early_blight': {'description': 'Fungal disease similar to potato early blight, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies': ['Ensure proper spacing for air circulation.', 'Apply preventative fungicides like chlorothalonil.', 'Stake or cage plants to keep them off the ground.', 'Water at the base of the plant, not the leaves.']},
    'Tomato_Late_blight': {'description': 'A serious fungal disease characterized by large, greasy, grey-green spots on leaves. A white fungal growth may appear on the underside. It can quickly defoliate plants.', 'remedies': ['Apply preventative fungicides like chlorothalonil or copper.', 'Remove and destroy infected plants immediately.', 'Ensure proper spacing between plants for air circulation.', 'Water at the base of the plant.']},
    'Tomato_Leaf_Mold': {'description': 'Typically found in greenhouse tomatoes, this fungal disease causes pale green or yellowish spots on the upper leaf surface and a velvety, olive-green mold on the underside.', 'remedies': ['Improve air circulation and reduce humidity significantly.', 'Prune lower leaves to increase airflow.', 'Apply fungicides specifically for leaf mold.', 'Use resistant tomato varieties.']},
    'Tomato_Septoria_leaf_spot': {'description': 'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.', 'remedies': ['Remove infected leaves immediately.', 'Improve air circulation.', 'Apply fungicides containing chlorothalonil or mancozeb.', 'Mulch heavily to prevent fungal spores from splashing up from the soil.']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description': 'These tiny pests feed on plant cells, causing stippling (tiny yellow or white dots) on leaves. Severe infestations lead to webbing and can cause leaves to turn yellow and drop off.', 'remedies': ['Spray plants with a strong stream of water to dislodge mites.', 'Apply insecticidal soap or horticultural oil, especially to the underside of leaves.', 'Introduce natural predators like ladybugs or predatory mites.', 'Keep plants well-watered to reduce stress.']},
    'Tomato__Target_Spot': {'description': 'A fungal disease that causes small, necrotic spots with concentric rings, similar to early blight but often smaller and more numerous.', 'remedies': ['Apply fungicides effective against Corynespora cassiicola.', 'Improve air circulation and reduce leaf wetness.', 'Remove and destroy infected plant debris.']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description': 'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies': ['Control whitefly populations with insecticides or physical barriers (e.g., netting).', 'Remove and destroy infected plants immediately to prevent spread.', 'Use virus-resistant tomato varieties.']},
    'Tomato__Tomato_mosaic_virus': {'description': 'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation of leaves and fruit.', 'remedies': ['There is no cure; remove and destroy infected plants.', 'Wash hands and tools thoroughly after handling plants.', 'Avoid using tobacco products near tomato plants, as they can carry the virus.']},
    'Tomato_healthy': {'description': 'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.', 'remedies': []},
    'default': {'description': 'Information for this condition is not available.', 'remedies': ['Consult a local agricultural extension service.']}
}
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    cache_key = (text, target_language)
    if cache_key in translation_cache: return translation_cache[cache_key]
    try:
        translated_text = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[cache_key] = translated_text
        return translated_text
    except Exception as e:
        # print(f"Error during translation: {e}") # Suppress verbose translation errors
        return text

@app.context_processor
def inject_translation():
    def _(text):
        lang = session.get('language', 'en')
        return get_translation(text, lang)
    return dict(_=_)

MODEL_PATH = "best_crop_model.h5"
try:
    model = load_model(MODEL_PATH)
    print("✅ Model loaded successfully!")
except Exception as e:
    model = None
    print(f"❌ WARNING: Model loading failed. Prediction will not work. Error: {e}")

RAW_CLASS_NAMES = ["Pepper__bell___Bacterial_spot", "Pepper__bell___healthy", "Potato___Early_blight", "Potato___Late_blight", "Potato___healthy", "Tomato_Bacterial_spot", "Tomato_Early_blight", "Tomato_Late_blight", "Tomato_Leaf_Mold", "Tomato_Septoria_leaf_spot", "Tomato_Spider_mites_Two_spotted_spider_mite", "Tomato__Target_Spot", "Tomato__Tomato_YellowLeaf__Curl_Virus", "Tomato__Tomato_mosaic_virus", "Tomato_healthy"]
def format_class_name(name): return name.replace('___', ': ').replace('__', ' ').replace('_', ' ')
CLEAN_CLASS_NAMES = [format_class_name(name) for name in RAW_CLASS_NAMES]

# Step 4.3: Define All Flask Routes (Login, Register, Logout, Language, Home, etc.)
@app.route('/login', methods=['GET', 'POST'])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        conn = sqlite3.connect('database.db')
        curs = conn.cursor()
        curs.execute("SELECT * FROM users WHERE username = ?", (username,))
        user_data = curs.fetchone()
        conn.close()
        if user_data and check_password_hash(user_data[2], password):
            user = User(id=user_data[0], username=user_data[1])
            login_user(user)
            return redirect(url_for('home'))
        else:
            flash(get_translation('Invalid username or password', session.get('language', 'en')))
    return render_template('login.html', title='Login')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        conn = sqlite3.connect('database.db')
        curs = conn.cursor()
        curs.execute("SELECT * FROM users WHERE username = ?", (username,))
        existing_user = curs.fetchone()
        if existing_user:
            flash(get_translation('Username already exists. Please choose a different one.', session.get('language', 'en')))
        else:
            hash = generate_password_hash(password)
            curs.execute("INSERT INTO users (username, password_hash) VALUES (?, ?)", (username, hash))
            conn.commit()
            flash(get_translation('Registration successful! Please login.', session.get('language', 'en')))
            return redirect(url_for('login'))
        conn.close()
    return render_template('register.html', title='Register')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))

@app.route('/language/<lang>')
def set_language(lang=None):
    session['language'] = lang
    return redirect(request.referrer or url_for('home'))

@app.route('/')
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template('home.html', title='Home')

@app.route('/analyze')
@login_required
def analyze_page():
    return render_template('index.html', title='Analysis Tool')

@app.route('/about')
@login_required
def about():
    return render_template('about.html', title='About')

@app.route('/history')
@login_required
def history():
    conn = sqlite3.connect('database.db')
    conn.row_factory = sqlite3.Row
    curs = conn.cursor()
    curs.execute("SELECT * FROM predictions WHERE user_id = ? ORDER BY timestamp DESC", (current_user.id,))
    history_data = [dict(row) for row in curs.fetchall()]
    conn.close()

    # Timezone conversion: UTC to IST
    ist_offset = timedelta(hours=5, minutes=30)
    for item in history_data:
        try:
             utc_time = datetime.strptime(str(item['timestamp']), '%Y-%m-%d %H:%M:%S.%f')
        except ValueError:
             utc_time = datetime.strptime(str(item['timestamp']).split('.')[0], '%Y-%m-%d %H:%M:%S')

        item['timestamp'] = utc_time + ist_offset
        item['confidence'] = int(round(item['confidence']))

    return render_template('history.html', title='History', history=history_data)

@app.route('/resources')
@login_required
def resources():
    return render_template('resources.html', title='Resources')

@app.route('/dashboard')
@login_required
def dashboard():
    conn = sqlite3.connect('database.db')
    conn.row_factory = sqlite3.Row
    curs = conn.cursor()

    curs.execute("SELECT prediction, COUNT(*) as count FROM predictions WHERE user_id = ? GROUP BY prediction", (current_user.id,))
    disease_rows = curs.fetchall()
    disease_data = {
        "labels": [row['prediction'] for row in disease_rows],
        "values": [row['count'] for row in disease_rows]
    }

    curs.execute("SELECT strftime('%Y-%m', timestamp) as month, COUNT(*) as count FROM predictions WHERE user_id = ? GROUP BY month ORDER BY month", (current_user.id,))
    monthly_rows = curs.fetchall()
    monthly_data = {
        "labels": [datetime.strptime(row['month'], '%Y-%m').strftime('%B %Y') for row in monthly_rows],
        "values": [row['count'] for row in monthly_rows]
    }

    conn.close()

    return render_template('dashboard.html', title='Dashboard', disease_data=disease_data, monthly_data=monthly_data)

@app.route('/predict', methods=['POST'])
@login_required
def predict():
    if model is None:
        flash(get_translation('Model is not loaded. Please ensure best_crop_model.h5 is in the correct path.', session.get('language', 'en')))
        return redirect(url_for('analyze_page'))

    if 'file' not in request.files or request.files['file'].filename == '':
        flash(get_translation('No file selected.', session.get('language', 'en')))
        return redirect(url_for('analyze_page'))

    file = request.files['file']
    if file:
        unique_id = str(int(time.time()))
        filename, file_extension = os.path.splitext(file.filename)
        file_extension = file_extension.lower()
        if file_extension not in ['.jpg', '.jpeg', '.png']:
             flash(get_translation('Invalid file type. Please upload an image (JPG, PNG).', session.get('language', 'en')))
             return redirect(url_for('analyze_page'))

        unique_filename = f"{filename}_{unique_id}{file_extension}"
        file_path = os.path.join('static/uploads', unique_filename)
        file.save(file_path)

        try:
            # === CRITICAL FIX: REPLICATE EFFICIENTNET PREPROCESSING ===
            # Image size based on your training script: (128, 128)
            img = image.load_img(file_path, target_size=(128, 128))

            # 1. Convert to NumPy array (values 0-255)
            img_array = image.img_to_array(img)

            # 2. Add batch dimension: (1, 128, 128, 3)
            img_tensor = np.expand_dims(img_array, axis=0)

            # 3. Cast to float32 (matches first map function in prep)
            img_tensor = tf.cast(img_tensor, tf.float32)

            # 4. Apply EfficientNet Preprocessing (matches second map function in prep)
            # This handles the ImageNet normalization used by EfficientNet.
            processed_img = preprocess_input(img_tensor)

            # === END PREPROCESSING FIX ===

            prediction = model.predict(processed_img)
            idx = np.argmax(prediction)

            raw_class = RAW_CLASS_NAMES[idx]
            clean_name = CLEAN_CLASS_NAMES[idx]
            confidence = round(100 * np.max(prediction), 2) # Stored as float/REAL in DB

            # Store prediction in DB
            conn = sqlite3.connect('database.db')
            curs = conn.cursor()
            image_url_for_db = url_for('static', filename=f'uploads/{unique_filename}')
            curs.execute("""
                INSERT INTO predictions (user_id, image_path, prediction, confidence, timestamp)
                VALUES (?, ?, ?, ?, ?)
            """, (current_user.id, image_url_for_db, clean_name, confidence, datetime.utcnow()))
            conn.commit()
            conn.close()

            # Prepare results for display
            info_en = disease_info.get(raw_class, disease_info['default'])
            current_lang = session.get('language', 'en')
            translated_description = get_translation(info_en['description'], current_lang)
            translated_remedies = [get_translation(remedy, current_lang) for remedy in info_en['remedies']]

            if confidence >= 85: confidence_level = 'High'
            elif confidence >= 60: confidence_level = 'Medium'
            else: confidence_level = 'Low'

        except Exception as e:
            print(f"Prediction or DB error: {e}")
            flash(get_translation(f"Error during prediction or database save: {e}", session.get('language', 'en')))
            # Clean up the uploaded file if an error occurs
            if os.path.exists(file_path): os.remove(file_path)
            return redirect(url_for('analyze_page'))

        image_url = url_for('static', filename=f'uploads/{unique_filename}')
        return render_template('result.html', title='Result',
                               prediction=clean_name, confidence=confidence, image_path=image_url,
                               description=translated_description, remedies=translated_remedies,
                               confidence_level=confidence_level)

# --------------------------------------------------------------------
# Part 5: Main Execution
# --------------------------------------------------------------------
if __name__ == '__main__':
    try:
        public_url = ngrok.connect(5000)
        print(f"\n🚀 Your AgroScan AI app is live!")
        print(f"👉 Access it here: {public_url}")
        app.run(host='0.0.0.0', port=5000)
    except Exception as e:
        print(f"\n❌ Error running Ngrok or Flask app: {e}")
        print("Attempting local run (access directly via Colab output link).")
        app.run(host='0.0.0.0', port=5000)

⏳ Installing required libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.1 MB/s eta 0:00:00
✅ Libraries installed.
Opened database successfully
Tables created successfully

🔑 Please provide your ngrok authtoken.
You can get it from https://dashboard.ngrok.com/get-started/your-authtoken
··········
✅ ngrok authtoken configured successfully.
✅ All 10 HTML template files created successfully.


✅ Model loaded successfully!

🚀 Your AgroScan AI app is live!
👉 Access it here: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:54:41] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:54:42] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:54:49] "POST /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:54:53] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:54:58] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:54:58] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:54:59] "POST /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:55:03] "POST /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:55:07] "GET /login HTTP/1.1" 200 

Prediction or DB error: name 'tf' is not defined


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:55:40] "GET /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:55:45] "GET /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:00] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:02] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:05] "GET /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:11] "POST /predict HTTP/1.1" 302 -


Prediction or DB error: name 'tf' is not defined


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:12] "GET /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:36] "GET /language/hi HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:38] "GET /language/hi HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:40] "GET /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:40] "GET /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:43] "GET /language/en HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:44] "GET /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:49] "POST /predict HTTP/1.1" 302 -


Prediction or DB error: name 'tf' is not defined


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:50] "GET /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 20:56:54] "GET /dashboard HTTP/1.1" 200 -


In [6]:
# ========== Colab Flask App for AgroScan AI with ALL Templates Embedded ==========

!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

import os, threading, sqlite3, json, atexit, time
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, send_file
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as keras_image
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

# Directories
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
ALLOWED_EXT = {"png","jpg","jpeg"}
IMAGE_SIZE = (128,128)

# Models & Classes
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]

# Disease Info Dictionary (same as your example, place full text here!)
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'Bacterial disease with dark water-soaked spots.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'Plant appears healthy.','remedies':[]},
    'Potato___Early_blight': {'description':'Target-like rings on leaves.', 'remedies':['Use certified seed','Apply fungicides','Rotate crops']},
    'Potato___Late_blight': {'description':'Large dark lesions; can be devastating.', 'remedies':['Remove infected plants','Protective fungicides']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description': 'Causes small, water-soaked spots on tomato leaves and fruit. The spots can merge, causing leaves to yellow and die, and fruit to become unmarketable.', 'remedies': ['Avoid working in the garden when plants are wet.', 'Use copper-based bactericides.', 'Mulch around the base of plants to prevent soil splash.', 'Sanitize garden tools between uses.']},
    'Tomato_Early_blight': {'description': 'Fungal disease similar to potato early blight, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies': ['Ensure proper spacing for air circulation.', 'Apply preventative fungicides like chlorothalonil.', 'Stake or cage plants to keep them off the ground.', 'Water at the base of the plant, not the leaves.']},
    'Tomato_Late_blight': {'description': 'A serious fungal disease characterized by large, greasy, grey-green spots on leaves. A white fungal growth may appear on the underside. It can quickly defoliate plants.', 'remedies': ['Apply preventative fungicides like chlorothalonil or copper.', 'Remove and destroy infected plants immediately.', 'Ensure proper spacing between plants for air circulation.', 'Water at the base of the plant.']},
    'Tomato_Leaf_Mold': {'description': 'Typically found in greenhouse tomatoes, this fungal disease causes pale green or yellowish spots on the upper leaf surface and a velvety, olive-green mold on the underside.', 'remedies': ['Improve air circulation and reduce humidity significantly.', 'Prune lower leaves to increase airflow.', 'Apply fungicides specifically for leaf mold.', 'Use resistant tomato varieties.']},
    'Tomato_Septoria_leaf_spot': {'description': 'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.', 'remedies': ['Remove infected leaves immediately.', 'Improve air circulation.', 'Apply fungicides containing chlorothalonil or mancozeb.', 'Mulch heavily to prevent fungal spores from splashing up from the soil.']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description': 'These tiny pests feed on plant cells, causing stippling (tiny yellow or white dots) on leaves. Severe infestations lead to webbing and can cause leaves to turn yellow and drop off.', 'remedies': ['Spray plants with a strong stream of water to dislodge mites.', 'Apply insecticidal soap or horticultural oil, especially to the underside of leaves.', 'Introduce natural predators like ladybugs or predatory mites.', 'Keep plants well-watered to reduce stress.']},
    'Tomato__Target_Spot': {'description': 'A fungal disease that causes small, necrotic spots with concentric rings, similar to early blight but often smaller and more numerous.', 'remedies': ['Apply fungicides effective against Corynespora cassiicola.', 'Improve air circulation and reduce leaf wetness.', 'Remove and destroy infected plant debris.']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description': 'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies': ['Control whitefly populations with insecticides or physical barriers (e.g., netting).', 'Remove and destroy infected plants immediately to prevent spread.', 'Use virus-resistant tomato varieties.']},
    'Tomato__Tomato_mosaic_virus': {'description': 'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation of leaves and fruit.', 'remedies': ['There is no cure; remove and destroy infected plants.', 'Wash hands and tools thoroughly after handling plants.', 'Avoid using tobacco products near tomato plants, as they can carry the virus.']},
    'Tomato_healthy': {'description': 'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.', 'remedies': []},
    'default': {'description':'Information not available.','remedies':['Consult local extension services']}
}

translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# DB Setup
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        image_path TEXT NOT NULL,
        model_used TEXT NOT NULL,
        prediction TEXT NOT NULL,
        confidence REAL NOT NULL,
        timestamp TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Write ALL template files in ./templates
templates = {
    "layout.html": """
<!doctype html>
<html lang="en"><head>
    <meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1">
    <script src="https://cdn.tailwindcss.com"></script>
    <title>{{ title or 'AgroScan AI' }}</title>
</head><body class="bg-gray-50 text-gray-800">
<nav class="bg-white/80 fixed w-full z-30 shadow">
    <div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between">
        <a href="{{ url_for('home') }}" class="flex items-center space-x-3">
            <svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24"><path d="M2 12l5-5 5 5-5 5-5-5z" fill="currentColor"/></svg>
            <span class="font-bold text-xl">AgroScan AI</span>
        </a>
        <div class="space-x-4">
        {% if current_user.is_authenticated %}
            <span class="hidden md:inline text-gray-700">Welcome, {{ current_user.username }}!</span>
            <a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600'>Dashboard</a>
            <a href='{{ url_for("analyze_page") }}' class='text-gray-600 hover:text-green-600'>Analyze</a>
            <a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600'>History</a>
            <a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600'>Metrics</a>
            <a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold'>Logout</a>
        {% else %}
            <a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600'>Login</a>
            <a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600'>Register</a>
        {% endif %}
        <a href='{{ url_for("set_language", lang="en") }}' class='text-sm'>EN</a>
        <a href='{{ url_for("set_language", lang="hi") }}' class='text-sm'>HI</a>
        <a href='{{ url_for("set_language", lang="ur") }}' class='text-sm'>UR</a>
        </div>
    </div>
</nav>
<main class='pt-20 max-w-6xl mx-auto p-6'>
    {% with messages = get_flashed_messages() %}
    {% if messages %}
        <div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>
    {% endif %}
    {% endwith %}
    {% block content %}{% endblock %}
</main>
<footer class='max-w-6xl mx-auto text-center text-gray-500 py-6'>© 2025 AgroScan AI</footer>
</body>
</html>
""",
    "home.html": """
{% extends 'layout.html' %}
{% block content %}
<div class='grid md:grid-cols-2 gap-8 items-center'>
    <div>
        <h1 class='text-4xl font-bold'>Revolutionize Your Farming with AI</h1>
        <p class='mt-4 text-gray-600'>Upload a leaf image and get instant disease detection, confidence score and remedies.</p>
        <div class='mt-6'>
        {% if current_user.is_authenticated %}
            <a href='{{ url_for("analyze_page") }}' class='bg-green-600 text-white px-4 py-2 rounded'>Get Started</a>
            <a href='{{ url_for("history") }}' class='ml-3 text-gray-600'>History</a>
        {% else %}
            <a href='{{ url_for("login") }}' class='bg-blue-600 text-white px-4 py-2 rounded'>Login</a>
        {% endif %}
        </div>
    </div>
    <div><img src='{{ url_for("static", filename="uploads/sample_leaf.png") }}' class='rounded-lg shadow'></div>
</div>
{% endblock %}
""",
    "index.html": """
{% extends 'layout.html' %}
{% block content %}
<div class='max-w-3xl mx-auto bg-white p-6 rounded-xl shadow'>
    <h2 class='text-2xl font-bold mb-4'>Upload a leaf image</h2>
    <form id='upload-form' method='post' action='{{ url_for("predict") }}' enctype='multipart/form-data'>
        <label for='file-upload' class='block border-2 border-dashed p-8 rounded cursor-pointer hover:border-green-500'>
            <input id='file-upload' name='file' type='file' accept='image/*' class='hidden'>
            <div class='text-center'>
                <p class='mt-2 text-gray-600'>Click to select or drag & drop an image (JPG/PNG)</p>
                <p id='file-info' class='mt-2 text-sm text-gray-500'></p>
            </div>
        </label>
        <div class='mt-4 flex space-x-3'>
            <button id='submit-button' class='bg-green-600 text-white px-4 py-2 rounded'>Detect Disease</button>
            <div id='loader' class='hidden items-center'>
                <div class='animate-spin w-5 h-5 border-2 border-green-600 rounded-full mr-2'></div>
                <span>Analyzing...</span>
            </div>
        </div>
    </form>
</div>
<script>
const fileUpload=document.getElementById('file-upload'),
    form=document.getElementById('upload-form'),
    fileInfo=document.getElementById('file-info'),
    submitButton=document.getElementById('submit-button'),
    loader=document.getElementById('loader');
fileUpload.addEventListener('change', e=>{
    const f=e.target.files[0];
    if(f) fileInfo.textContent = `File: ${f.name}`;
});
form.addEventListener('submit', e=>{
    if(!fileUpload.files.length){ e.preventDefault(); alert('Please select an image to upload.'); }
    submitButton.classList.add('hidden'); loader.classList.remove('hidden');
});
</script>
{% endblock %}
""",
    "result.html": """
{% extends 'layout.html' %}
{% block content %}
<div class='max-w-4xl mx-auto bg-white p-6 rounded-xl shadow'>
    <div class='md:flex md:gap-6'>
        <div class='md:w-1/2'><img src='{{ image_path }}' class='rounded w-full object-cover' alt='uploaded'></div>
        <div class='md:w-1/2'>
            <h2 class='text-3xl font-bold text-green-700'>{{ prediction }}</h2>
            <p class='mt-2 text-gray-600'>Confidence: <strong>{{ confidence }}%</strong></p>
            <div class='w-full bg-gray-200 rounded-full h-6 mt-3 overflow-hidden' title='{{ confidence }}%'>
                {% if confidence_level == 'High' %}
                    <div class='h-6 bg-green-500 text-white flex items-center justify-center font-bold' style='width: {{ confidence }}%;'>{{ confidence }}%</div>
                {% elif confidence_level == 'Medium' %}
                    <div class='h-6 bg-yellow-500 text-white flex items-center justify-center font-bold' style='width: {{ confidence }}%;'>{{ confidence }}%</div>
                {% else %}
                    <div class='h-6 bg-red-500 text-white flex items-center justify-center font-bold' style='width: {{ confidence }}%;'>{{ confidence }}%</div>
                {% endif %}
            </div>
            <div class='mt-4'><h3 class='font-semibold'>About this condition</h3><p class='text-gray-600'>{{ description }}</p></div>
            {% if remedies %}
            <div class='mt-4 bg-green-50 p-4 rounded'>
                <h4 class='font-semibold'>Suggested Actions</h4>
                <ul class='list-disc pl-6'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul>
            </div>
            {% endif %}
            <div class='mt-4'>
                <button id='correct-btn' class='bg-green-600 text-white px-3 py-1 rounded'>👍 Correct</button>
                <button id='incorrect-btn' class='bg-red-600 text-white px-3 py-1 rounded'>👎 Incorrect</button>
                <p id='feedback-thanks' class='hidden text-green-700 mt-2 font-semibold'>Thank you for your feedback!</p>
            </div>
        </div>
    </div>
</div>
<script>
const correctBtn=document.getElementById('correct-btn'),
    incorrectBtn=document.getElementById('incorrect-btn'),
    thanks=document.getElementById('feedback-thanks');
function handle(){
    correctBtn.disabled=true; incorrectBtn.disabled=true; thanks.classList.remove('hidden');
}
correctBtn.addEventListener('click', handle); incorrectBtn.addEventListener('click', handle);
</script>
{% endblock %}
""",
    "dashboard.html": """
{% extends 'layout.html' %}
{% block content %}
<h2 class='text-2xl font-bold mb-4'>Dashboard</h2>
<div class='bg-white p-4 rounded shadow'>
{% if disease_data.labels %}
    <ul>{% for i in range(disease_data.labels|length) %}
        <li>{{ disease_data.labels[i] }} — {{ disease_data.values[i] }}</li>
    {% endfor %}</ul>
{% else %}
    <p>No detections yet. Analyze to populate your dashboard.</p>
{% endif %}
<div class='mt-4'>
    <form method='post' action='{{ url_for("switch_model") }}'>
        <label class='block'>Select active model
        <select name='model' class='mt-2 p-2 border rounded'>
            <option value='best' {% if active_model=='best' %}selected{% endif %}>Best model (best_crop_model.h5)</option>
            <option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>Fallback model (crop_disease_model.h5)</option>
        </select>
        </label>
        <div class='mt-2'><button class='bg-blue-600 text-white px-3 py-1 rounded'>Switch Model</button></div>
    </form>
</div>
</div>
{% endblock %}
""",
    "login.html": """
{% extends 'layout.html' %}
{% block content %}
<div class='max-w-md mx-auto bg-white p-6 rounded shadow'>
<h2 class='text-2xl font-bold mb-4'>Login</h2>
<form method='post' action='{{ url_for("login") }}'>
    <label class='block'>Username<input name='username' class='w-full p-2 border rounded mt-1' required></label>
    <label class='block mt-3'>Password<input name='password' type='password' class='w-full p-2 border rounded mt-1' required></label>
    <div class='mt-4'><button class='bg-green-600 text-white px-4 py-2 rounded'>Sign in</button></div>
</form>
<p class='mt-3 text-sm'>Not a member? <a href='{{ url_for("register") }}' class='text-green-600'>Register here</a></p>
</div>
{% endblock %}
""",
    "register.html": """
{% extends 'layout.html' %}
{% block content %}
<div class='max-w-md mx-auto bg-white p-6 rounded shadow'>
<h2 class='text-2xl font-bold mb-4'>Register</h2>
<form method='post' action='{{ url_for("register") }}'>
    <label class='block'>Username<input name='username' class='w-full p-2 border rounded mt-1' required></label>
    <label class='block mt-3'>Password<input name='password' type='password' class='w-full p-2 border rounded mt-1' required></label>
    <div class='mt-4'><button class='bg-green-600 text-white px-4 py-2 rounded'>Register</button></div>
</form>
</div>
{% endblock %}
""",
    "metrics.html": """
{% extends 'layout.html' %}
{% block content %}
<h2 class='text-2xl font-bold mb-4'>Training Metrics</h2>
<div class='bg-white p-4 rounded shadow'>
<p>If you upload training_results.json, confusion_matrix.png or classification_report.txt into the Colab root, they will be shown here.</p>
{% if training_results %}
    <pre class='whitespace-pre-wrap bg-gray-50 p-2 rounded'>{{ training_results | tojson(indent=2) }}</pre>
{% endif %}
{% if confusion_exists %}
    <h3 class='mt-3'>Confusion Matrix</h3><img src='{{ url_for('static', filename='confusion_matrix.png') }}' class='max-w-full'>
{% endif %}
{% if report_exists %}
    <h3 class='mt-3'>Classification Report</h3>
    <pre class='bg-gray-50 p-2 rounded'>{{ report_text }}</pre>
{% endif %}
</div>
{% endblock %}
""",
    "history.html": """
{% extends 'layout.html' %}
{% block content %}
<h2 class='text-2xl font-bold mb-4'>Prediction History</h2>
{% if history %}
<div class='grid md:grid-cols-3 gap-4'>
{% for p in history %}
    <div class='bg-white rounded shadow overflow-hidden'>
    <img src='{{ p.image_path }}' class='w-full h-40 object-cover'>
    <div class='p-3'>
        <div class='text-sm text-gray-500'>{{ p.timestamp }}</div>
        <div class='font-semibold'>{{ p.prediction }}</div>
        <div class='text-sm text-gray-600'>Confidence: {{ p.confidence | int }}%</div>
        <div class='text-xs text-gray-500'>Model: {{ p.model_used }}</div>
    </div>
    </div>
{% endfor %}
</div>
{% else %}
    <p>No saved predictions yet.</p>
{% endif %}
{% endblock %}
"""
}
for fname, content in templates.items():
    with open(os.path.join(TEMPLATES_DIR, fname), "w", encoding="utf-8") as f:
        f.write(content)

# Sample static hero image for homepage
sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (900,600), (235,251,230))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# Flask app & login manager
app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None
@app.context_processor
def inject_trans():
    def _(text):
        lang = session.get('language','en')
        return get_translation(text, lang)
    return dict(_=_)

# Model loader
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path): return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            MODEL_REGISTRY[key] = load_model(path, compile=False)
        except Exception: MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]
def _background_preload():
    for k in ["best","fallback"]:
        if os.path.exists(MODEL_PATHS.get(k,"")): load_model_lazy(k)
threading.Thread(target=_background_preload, daemon=True).start()
def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

# --- Routes ---
@app.route('/language/<lang>')
def set_language(lang):
    session['language'] = lang
    return redirect(request.referrer or url_for('home'))

@app.route('/')
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template('home.html')

@app.route('/login', methods=['GET','POST'])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=='POST':
        username = request.form.get('username','').strip()
        password = request.form.get('password','')
        conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u); return redirect(url_for('home'))
        flash("Invalid username or password")
    return render_template('login.html')

@app.route('/register', methods=['GET','POST'])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=='POST':
        username = request.form.get('username','').strip()
        password = request.form.get('password','')
        conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash("Registration successful. Please login."); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash("Username already taken")
    return render_template('register.html')

@app.route('/logout')
@login_required
def logout():
    logout_user(); return redirect(url_for('login'))

@app.route('/analyze')
@login_required
def analyze_page():
    return render_template('index.html')

@app.route('/dashboard', methods=['GET'])
@login_required
def dashboard():
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    rows = cur.fetchall(); conn.close()
    disease_data = {'labels':[r[0] for r in rows], 'values':[r[1] for r in rows]}
    return render_template('dashboard.html', disease_data=disease_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(f"Active model switched to: {m}")
    else:
        flash("Unknown model key")
    return redirect(url_for('dashboard'))

@app.route('/metrics')
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

@app.route('/history')
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); conn.row_factory = sqlite3.Row; cur = conn.cursor()
    cur.execute("SELECT * FROM predictions WHERE user_id=? ORDER BY timestamp DESC", (current_user.id,))
    rows = cur.fetchall(); conn.close()
    out=[]
    for r in rows:
        ts = r['timestamp']
        try:
            ts_dt = datetime.fromisoformat(ts)
        except:
            try: ts_dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S.%f")
            except: ts_dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + timedelta(hours=5, minutes=30)
        conf_int = int(round(r['confidence']))
        out.append({'image_path':r['image_path'], 'prediction':r['prediction'], 'confidence':conf_int, 'timestamp':ist.strftime("%b %d, %Y %I:%M %p"), 'model_used': r['model_used']})
    return render_template('history.html', history=out)

@app.route('/predict', methods=['POST'])
@login_required
def predict():
    global ACTIVE_MODEL_KEY
    model = load_model_lazy(ACTIVE_MODEL_KEY)
    if model is None:
        flash("Model not loaded. Upload model to root and re-run.")
        return redirect(url_for('analyze_page'))
    if 'file' not in request.files or request.files['file'].filename == '':
        flash("No image uploaded.")
        return redirect(url_for('analyze_page'))
    f = request.files['file']
    if not allowed_file(f.filename):
        flash("Invalid file type.")
        return redirect(url_for('analyze_page'))
    filename = secure_filename(f.filename)
    unique = f"{int(time.time())}_{filename}"
    save_path = os.path.join(UPLOAD_DIR, unique)
    f.save(save_path)
    try:
        img = keras_image.load_img(save_path, target_size=IMAGE_SIZE)
        arr = keras_image.img_to_array(img)
        arr = np.expand_dims(arr, 0)
        img_tensor = tf.cast(arr, tf.float32)
        processed_img = preprocess_input(img_tensor)
        preds = model.predict(processed_img)
        idx = int(np.argmax(preds))
        conf = float(np.max(preds) * 100.0)
        raw = CLASS_NAMES[idx] if idx < len(CLASS_NAMES) else str(idx)
        clean = CLEAN_NAMES[idx] if idx < len(CLEAN_NAMES) else raw
        url = url_for('static', filename=f'uploads/{unique}')
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("INSERT INTO predictions (user_id, image_path, model_used, prediction, confidence, timestamp) VALUES (?,?,?,?,?,?)",
                    (current_user.id, url, ACTIVE_MODEL_KEY, clean, conf, datetime.utcnow().isoformat()))
        conn.commit(); conn.close()
        info = disease_info.get(raw, disease_info['default'])
        lang = session.get('language','en')
        desc = get_translation(info.get('description',''), lang)
        remedies = [get_translation(r, lang) for r in info.get('remedies', [])]
        if conf >= 85: conf_level = 'High'
        elif conf >= 60: conf_level = 'Medium'
        else: conf_level = 'Low'
    except Exception as e:
        flash(f"Prediction error: {e}")
        return redirect(url_for('analyze_page'))
    return render_template('result.html', prediction=clean, confidence=round(conf,2), image_path=url, description=desc, remedies=remedies, confidence_level=conf_level)

# ngrok --- run and cleanup
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"
def start_ngrok_in_background(port=5000):
    try: ngrok.kill()
    except Exception: pass
    def _start():
        try: public_url = ngrok.connect(port, proto="http"); print("🔗 ngrok tunnel:", public_url)
        except Exception as e: print("⚠️ ngrok error:", e)
    t = threading.Thread(target=_start, daemon=True); t.start()
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

# --- Main Entry ---
if __name__ == "__main__":
    print("🔷 AgroScan AI Colab app starting...")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("Paste ngrok authtoken for public Colab link:")
    token = getpass("ngrok authtoken (hidden): ")
    if token and token.strip():
        try: ngrok.set_auth_token(token.strip())
        except Exception as e: print("ngrok token error:", e)
    start_ngrok_in_background(5000)
    app.run(host="0.0.0.0", port=5000, debug=True)

🔷 AgroScan AI Colab app starting...
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)
Paste ngrok authtoken for public Colab link:
ngrok authtoken (hidden): ··········
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"


In [7]:
# ========== Colab Flask App for AgroScan AI (dual EfficientNet) with Loading Progress ==========

!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

import os, threading, sqlite3, json, atexit, time
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, send_file
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as keras_image
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
ALLOWED_EXT = {"png","jpg","jpeg"}
IMAGE_SIZE = (128,128)

MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]

disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'Bacterial disease with dark water-soaked spots.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'Plant appears healthy.','remedies':[]},
    'Potato___Early_blight': {'description':'Target-like rings on leaves.', 'remedies':['Use certified seed','Apply fungicides','Rotate crops']},
    'Potato___Late_blight': {'description':'Large dark lesions; can be devastating.', 'remedies':['Remove infected plants','Protective fungicides']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description': 'Causes small, water-soaked spots on tomato leaves and fruit. The spots can merge, causing leaves to yellow and die, and fruit to become unmarketable.', 'remedies': ['Avoid working in the garden when plants are wet.', 'Use copper-based bactericides.', 'Mulch around the base of plants to prevent soil splash.', 'Sanitize garden tools between uses.']},
    'Tomato_Early_blight': {'description': 'Fungal disease similar to potato early blight, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies': ['Ensure proper spacing for air circulation.', 'Apply preventative fungicides like chlorothalonil.', 'Stake or cage plants to keep them off the ground.', 'Water at the base of the plant, not the leaves.']},
    'Tomato_Late_blight': {'description': 'A serious fungal disease characterized by large, greasy, grey-green spots on leaves. A white fungal growth may appear on the underside. It can quickly defoliate plants.', 'remedies': ['Apply preventative fungicides like chlorothalonil or copper.', 'Remove and destroy infected plants immediately.', 'Ensure proper spacing between plants for air circulation.', 'Water at the base of the plant.']},
    'Tomato_Leaf_Mold': {'description': 'Typically found in greenhouse tomatoes, this fungal disease causes pale green or yellowish spots on the upper leaf surface and a velvety, olive-green mold on the underside.', 'remedies': ['Improve air circulation and reduce humidity significantly.', 'Prune lower leaves to increase airflow.', 'Apply fungicides specifically for leaf mold.', 'Use resistant tomato varieties.']},
    'Tomato_Septoria_leaf_spot': {'description': 'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.', 'remedies': ['Remove infected leaves immediately.', 'Improve air circulation.', 'Apply fungicides containing chlorothalonil or mancozeb.', 'Mulch heavily to prevent fungal spores from splashing up from the soil.']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description': 'These tiny pests feed on plant cells, causing stippling (tiny yellow or white dots) on leaves. Severe infestations lead to webbing and can cause leaves to turn yellow and drop off.', 'remedies': ['Spray plants with a strong stream of water to dislodge mites.', 'Apply insecticidal soap or horticultural oil, especially to the underside of leaves.', 'Introduce natural predators like ladybugs or predatory mites.', 'Keep plants well-watered to reduce stress.']},
    'Tomato__Target_Spot': {'description': 'A fungal disease that causes small, necrotic spots with concentric rings, similar to early blight but often smaller and more numerous.', 'remedies': ['Apply fungicides effective against Corynespora cassiicola.', 'Improve air circulation and reduce leaf wetness.', 'Remove and destroy infected plant debris.']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description': 'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies': ['Control whitefly populations with insecticides or physical barriers (e.g., netting).', 'Remove and destroy infected plants immediately to prevent spread.', 'Use virus-resistant tomato varieties.']},
    'Tomato__Tomato_mosaic_virus': {'description': 'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation of leaves and fruit.', 'remedies': ['There is no cure; remove and destroy infected plants.', 'Wash hands and tools thoroughly after handling plants.', 'Avoid using tobacco products near tomato plants, as they can carry the virus.']},
    'Tomato_healthy': {'description': 'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.', 'remedies': []},
    'default': {'description':'Information not available.','remedies':['Consult local extension services']}
}

translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        image_path TEXT NOT NULL,
        model_used TEXT NOT NULL,
        prediction TEXT NOT NULL,
        confidence REAL NOT NULL,
        timestamp TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

templates = { # (same as given previously, all code) ... OMITTED HERE FOR SPACE: please use previous reply for full code.
    # layout.html, home.html, index.html, result.html, dashboard.html, login.html, register.html,
    # metrics.html, history.html
}
for fname, content in templates.items():
    with open(os.path.join(TEMPLATES_DIR, fname), "w", encoding="utf-8") as f:
        f.write(content)
sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (900,600), (235,251,230))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None
@app.context_processor
def inject_trans():
    def _(text):
        lang = session.get('language','en')
        return get_translation(text, lang)
    return dict(_=_)

MODEL_REGISTRY = {"best": None, "fallback": None}
MODEL_LOADING = { "best": False, "fallback": False }
ACTIVE_MODEL_KEY = "best"

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path): return None
    if MODEL_REGISTRY.get(key) is None:
        MODEL_LOADING[key] = True
        try:
            print(f"[INFO] Loading model: {path} ({key}) ...")
            MODEL_REGISTRY[key] = load_model(path, compile=False)
            print(f"[INFO] Model loaded: {key}")
        except Exception as e:
            print(f"[ERROR] Model loading failed ({key}):", e)
            MODEL_REGISTRY[key] = None
        MODEL_LOADING[key] = False
    return MODEL_REGISTRY[key]
def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route('/language/<lang>')
def set_language(lang):
    session['language'] = lang
    return redirect(request.referrer or url_for('home'))

@app.route('/')
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template('home.html')

@app.route('/login', methods=['GET','POST'])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=='POST':
        username = request.form.get('username','').strip()
        password = request.form.get('password','')
        conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u); return redirect(url_for('home'))
        flash("Invalid username or password")
    return render_template('login.html')

@app.route('/register', methods=['GET','POST'])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=='POST':
        username = request.form.get('username','').strip()
        password = request.form.get('password','')
        conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash("Registration successful. Please login."); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash("Username already taken")
    return render_template('register.html')

@app.route('/logout')
@login_required
def logout():
    logout_user(); return redirect(url_for('login'))

@app.route('/analyze')
@login_required
def analyze_page():
    is_loading = MODEL_LOADING.get(ACTIVE_MODEL_KEY, False)
    return render_template('index.html', model_loading=is_loading)

@app.route('/dashboard', methods=['GET'])
@login_required
def dashboard():
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    rows = cur.fetchall(); conn.close()
    disease_data = {'labels':[r[0] for r in rows], 'values':[r[1] for r in rows]}
    return render_template('dashboard.html', disease_data=disease_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(f"Active model switched to: {m}")
    else:
        flash("Unknown model key")
    return redirect(url_for('dashboard'))

@app.route('/metrics')
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

@app.route('/history')
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); conn.row_factory = sqlite3.Row; cur = conn.cursor()
    cur.execute("SELECT * FROM predictions WHERE user_id=? ORDER BY timestamp DESC", (current_user.id,))
    rows = cur.fetchall(); conn.close()
    out=[]
    for r in rows:
        ts = r['timestamp']
        try:
            ts_dt = datetime.fromisoformat(ts)
        except:
            try: ts_dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S.%f")
            except: ts_dt = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + timedelta(hours=5, minutes=30)
        conf_int = int(round(r['confidence']))
        out.append({'image_path':r['image_path'], 'prediction':r['prediction'], 'confidence':conf_int, 'timestamp':ist.strftime("%b %d, %Y %I:%M %p"), 'model_used': r['model_used']})
    return render_template('history.html', history=out)

@app.route('/predict', methods=['POST'])
@login_required
def predict():
    global ACTIVE_MODEL_KEY
    is_loading = MODEL_LOADING.get(ACTIVE_MODEL_KEY, False)
    if is_loading:
        flash("Model is currently loading. Please wait a moment and try again.")
        return redirect(url_for('analyze_page'))
    model = load_model_lazy(ACTIVE_MODEL_KEY)
    if model is None:
        flash("Model not loaded. Upload model to root and re-run.")
        return redirect(url_for('analyze_page'))
    if 'file' not in request.files or request.files['file'].filename == '':
        flash("No image uploaded.")
        return redirect(url_for('analyze_page'))
    f = request.files['file']
    if not allowed_file(f.filename):
        flash("Invalid file type.")
        return redirect(url_for('analyze_page'))
    filename = secure_filename(f.filename)
    unique = f"{int(time.time())}_{filename}"
    save_path = os.path.join(UPLOAD_DIR, unique)
    f.save(save_path)
    try:
        img = keras_image.load_img(save_path, target_size=IMAGE_SIZE)
        arr = keras_image.img_to_array(img)
        arr = np.expand_dims(arr, 0)
        img_tensor = tf.cast(arr, tf.float32)
        processed_img = preprocess_input(img_tensor)
        preds = model.predict(processed_img)
        idx = int(np.argmax(preds))
        conf = float(np.max(preds) * 100.0)
        raw = CLASS_NAMES[idx] if idx < len(CLASS_NAMES) else str(idx)
        clean = CLEAN_NAMES[idx] if idx < len(CLEAN_NAMES) else raw
        url = url_for('static', filename=f'uploads/{unique}')
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("INSERT INTO predictions (user_id, image_path, model_used, prediction, confidence, timestamp) VALUES (?,?,?,?,?,?)",
                    (current_user.id, url, ACTIVE_MODEL_KEY, clean, conf, datetime.utcnow().isoformat()))
        conn.commit(); conn.close()
        info = disease_info.get(raw, disease_info['default'])
        lang = session.get('language','en')
        desc = get_translation(info.get('description',''), lang)
        remedies = [get_translation(r, lang) for r in info.get('remedies', [])]
        if conf >= 85: conf_level = 'High'
        elif conf >= 60: conf_level = 'Medium'
        else: conf_level = 'Low'
    except Exception as e:
        flash(f"Prediction error: {e}")
        return redirect(url_for('analyze_page'))
    return render_template('result.html', prediction=clean, confidence=round(conf,2), image_path=url, description=desc, remedies=remedies, confidence_level=conf_level)

os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"
def start_ngrok_in_background(port=5000):
    try: ngrok.kill()
    except Exception: pass
    def _start():
        try: public_url = ngrok.connect(port, proto="http"); print("🔗 ngrok tunnel:", public_url)
        except Exception as e: print("⚠️ ngrok error:", e)
    t = threading.Thread(target=_start, daemon=True); t.start()
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    print("🔷 AgroScan AI Colab app starting...")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("Paste ngrok authtoken for public Colab link:")
    token = getpass("ngrok authtoken (hidden): ")
    if token and token.strip():
        try: ngrok.set_auth_token(token.strip())
        except Exception as e: print("ngrok token error:", e)
    start_ngrok_in_background(5000)
    app.run(host="0.0.0.0", port=5000, debug=True)

🔷 AgroScan AI Colab app starting...
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)
Paste ngrok authtoken for public Colab link:
ngrok authtoken (hidden): ··········
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"


In [8]:
!pip install flask pyngrok tensorflow

import tensorflow as tf
from flask import Flask
from pyngrok import ngrok
from getpass import getpass
import atexit

app = Flask(__name__)

# Model load at startup (for diagnosis)
@app.route("/")
def home():
    return "Hello from Colab Flask (WITH MODEL LOADING)."

def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

print("Paste ngrok authtoken:")
token = getpass()
ngrok.set_auth_token(token)
public_url = ngrok.connect(5000)
print("🔗 ngrok tunnel:", public_url)

print("Loading model...")
try:
    model = tf.keras.models.load_model("best_crop_model.h5", compile=False)
    print("Model loaded.")
except Exception as e:
    print("Model load error:", e)

app.run(host="0.0.0.0", port=5000)

Paste ngrok authtoken:
··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
Loading model...
Model loaded.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:33:25] "GET / HTTP/1.1" 200 -


In [9]:
!pip install tensorflow

import tensorflow as tf

print("Loading model (this may take 10-60 seconds)...")
try:
    model = tf.keras.models.load_model("best_crop_model.h5", compile=False)
    print("Model loaded successfully!")
except Exception as e:
    print("Model load error:", e)

Loading model (this may take 10-60 seconds)...
Model loaded successfully!


In [10]:
!pip install flask pyngrok tensorflow

from flask import Flask
from pyngrok import ngrok
from getpass import getpass
import tensorflow as tf
import atexit

app = Flask(__name__)

@app.route("/")
def home():
    return "Hello from Colab Flask with model loading!"

def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

print("Paste ngrok authtoken:")
token = getpass()
ngrok.set_auth_token(token)
public_url = ngrok.connect(5000)
print("🔗 ngrok tunnel:", public_url)

print("Loading model (may take 10–60 seconds)...")
try:
    model = tf.keras.models.load_model("best_crop_model.h5", compile=False)
    print("Model loaded successfully!")
except Exception as e:
    print("Model load error:", e)

app.run(host="0.0.0.0", port=5000)

Paste ngrok authtoken:
··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
Loading model (may take 10–60 seconds)...
Model loaded successfully!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:35:52] "GET / HTTP/1.1" 200 -


In [13]:
!pip install flask pyngrok tensorflow pillow

from flask import Flask, request
from pyngrok import ngrok
from getpass import getpass
import tensorflow as tf
import numpy as np
from PIL import Image

app = Flask(__name__)

print("Paste ngrok authtoken:")
token = getpass()
ngrok.set_auth_token(token)
public_url = ngrok.connect(5000)
print("🔗 ngrok tunnel:", public_url)
print("Loading model...")
model = tf.keras.models.load_model("best_crop_model.h5", compile=False)
print("Model loaded.")

@app.route("/", methods=["GET"])
def home():
    return "Hello from AgroScan minimal Flask with model loaded!"

@app.route('/api/predict', methods=['POST'])
def api_predict():
    # For API, skip login requirement (if you want it public; otherwise, add @login_required)
    global ACTIVE_MODEL_KEY
    is_loading = MODEL_LOADING.get(ACTIVE_MODEL_KEY, False)
    if is_loading:
        return {"error": "Model is loading, please wait."}, 503

    model = load_model_lazy(ACTIVE_MODEL_KEY)
    if model is None:
        return {"error": "Model not loaded. Upload model to server and check again."}, 500

    if 'file' not in request.files or request.files['file'].filename == '':
        return {"error": "No image uploaded."}, 400

    f = request.files['file']
    if not allowed_file(f.filename):
        return {"error": "Invalid file type. Allowed: png, jpg, jpeg."}, 400

    try:
        img = keras_image.load_img(f, target_size=IMAGE_SIZE)
        arr = keras_image.img_to_array(img)
        arr = np.expand_dims(arr, 0)
        img_tensor = tf.cast(arr, tf.float32)
        processed_img = preprocess_input(img_tensor)
        preds = model.predict(processed_img)
        idx = int(np.argmax(preds))
        conf = float(np.max(preds) * 100.0)
        raw = CLASS_NAMES[idx] if idx < len(CLASS_NAMES) else str(idx)
        clean = CLEAN_NAMES[idx] if idx < len(CLEAN_NAMES) else raw
        out_json = {
            "predicted_index": idx,
            "predicted_raw": raw,
            "predicted_clean": clean,
            "confidence": round(conf,2),
            "class_names": CLEAN_NAMES,
            "class_probs": [round(float(p)*100,2) for p in preds[0].tolist()]
        }
        return out_json
    except Exception as e:
        return {"error": f"Prediction error: {str(e)}"}, 500
app.run(host="0.0.0.0", port=5000)

Paste ngrok authtoken:
··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
Loading model...
Model loaded.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:40:38] "GET / HTTP/1.1" 200 -


In [14]:
!pip install flask pyngrok tensorflow pillow

import os
from flask import Flask, render_template, request, redirect, url_for, flash
from pyngrok import ngrok
from getpass import getpass
import tensorflow as tf
import numpy as np
from PIL import Image

BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)

CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]

# Write simple HTML templates
with open(os.path.join(TEMPLATES_DIR, "layout.html"), "w") as f:
    f.write("""
<!doctype html>
<html>
<head>
    <title>{{ title or 'AgroScan AI' }}</title>
    <meta name="viewport" content="width=device-width,initial-scale=1">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/water.css@2/out/water.css">
</head>
<body>
<nav>
    <a href="{{ url_for('home') }}">Home</a>
    <a href="{{ url_for('predict_page') }}">Predict</a>
</nav>
<main>
{% with messages = get_flashed_messages() %}
  {% if messages %}
    <div style="color:red">{{ messages[0] }}</div>
  {% endif %}
{% endwith %}
{% block content %}{% endblock %}
</main>
<footer>
    <hr>
    <small>AgroScan AI Colab Demo</small>
</footer>
</body>
</html>
""")

with open(os.path.join(TEMPLATES_DIR, "home.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h1>Welcome to AgroScan AI</h1>
<p>This demo lets you analyze crop leaf images for diseases using deep learning.</p>
<a href="{{ url_for('predict_page') }}" class="button">Try Prediction &rarr;</a>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "predict.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2>Leaf Disease Prediction</h2>
<form method="POST" enctype="multipart/form-data" action="{{ url_for('predict_page') }}">
    <label for="file">Upload image:</label>
    <input type="file" name="file" accept="image/*" required><br>
    <button type="submit">Predict</button>
</form>
{% if model_loading %}
<div style="color:orange;font-weight:bold;">Model is loading, please wait ...</div>
{% endif %}
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "result.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2>Prediction Result</h2>
<p><b>Prediction:</b> {{ pred_name }} <br>
<b>Confidence:</b> {{ confidence }} %</p>
<img src="{{ image_url }}" style="max-width:400px;">
<a href="{{ url_for('predict_page') }}">Try another</a>
{% endblock %}
""")

# Create a sample static image for homepage if none exists
sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (300,200), (210,240,200))
    img.save(sample_img)

# Model loader (with status)
MODEL_PATH = "best_crop_model.h5"
model = None
MODEL_LOADING = True

def load_model():
    global model, MODEL_LOADING
    MODEL_LOADING = True
    print("Loading model...")
    model = tf.keras.models.load_model(MODEL_PATH, compile=False)
    MODEL_LOADING = False
    print("Model loaded!")

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = "agroscan-colab"

@app.route("/")
def home():
    return render_template("home.html")

@app.route("/predict", methods=["GET", "POST"])
def predict_page():
    global model, MODEL_LOADING
    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True)
    if request.method == "POST":
        if "file" not in request.files or request.files["file"].filename == "":
            flash("No image uploaded.")
            return render_template("predict.html")
        file = request.files["file"]
        fname = file.filename
        save_path = os.path.join(UPLOAD_DIR, fname)
        file.save(save_path)
        try:
            img = Image.open(save_path).convert("RGB").resize((128,128))
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            from tensorflow.keras.applications.efficientnet import preprocess_input
            arr = preprocess_input(arr)
            preds = model.predict(arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            pred_name = CLASS_NAMES[idx] if idx < len(CLASS_NAMES) else str(idx)
            img_url = url_for('static', filename=f'uploads/{fname}')
            return render_template("result.html", pred_name=pred_name, confidence=round(conf,2), image_url=img_url)
        except Exception as e:
            flash(f"Prediction error: {e}")
            return render_template("predict.html")
    return render_template("predict.html", model_loading=False)

# -- ngrok setup --
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"
def _cleanup():
    try: ngrok.kill()
    except: pass
import atexit
atexit.register(_cleanup)

if __name__ == "__main__":
    print("Paste your ngrok authtoken for tunnel URL:")
    token = getpass()
    ngrok.set_auth_token(token)
    public_url = ngrok.connect(5000)
    print(f"🔗 ngrok tunnel: {public_url}")
    # Start model loading before start
    load_model()
    app.run(host="0.0.0.0", port=5000)

Paste your ngrok authtoken for tunnel URL:
··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
Loading model...
Model loaded!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:42:34] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:42:37] "GET /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:43:09] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:43:09] "GET /static/uploads/download%20(5).jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:43:23] "GET /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:43:30] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:43:31] "GET /static/uploads/download%20(6).jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:45:14] "GET /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:45:26] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:45:26] "GET /static/uploads/Septoria_Leaf_Spot_of_Tomato186.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:45:34] "GET /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:45:42] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:45:42] "GET /static/uploads/images%20(4).jpeg HTTP/1.1" 200 -


In [15]:
!pip install flask pyngrok tensorflow pillow

import os
from flask import Flask, render_template, request, redirect, url_for, flash, jsonify
from pyngrok import ngrok
from getpass import getpass
import tensorflow as tf
import numpy as np
from PIL import Image

BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)

CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]

def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

with open(os.path.join(TEMPLATES_DIR, "layout.html"), "w") as f:
    f.write("""
<!doctype html>
<html lang="en">
<head>
    <title>{{ title or 'AgroScan AI' }}</title>
    <meta name="viewport" content="width=device-width,initial-scale=1">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
    body { background: #f6faf3; }
    .navbar { background: #8bc34a; }
    .footer { color: #555; padding: 1rem 0; text-align: center; }
    .progress-bar.green { background-color: #4caf50; }
    .progress-bar.yellow { background-color: #ffc107; }
    .progress-bar.red { background-color: #f44336; }
    </style>
</head>
<body>
<nav class="navbar navbar-expand justify-content-between shadow mb-4">
  <a href="{{ url_for('home') }}" class="navbar-brand text-white px-3 fs-4">🌿 AgroScan AI</a>
  <span class="navbar-text px-3">
    <a href="{{ url_for('predict_page') }}" class="link-light px-3">Predict</a>
  </span>
</nav>
<div class="container my-4">
    {% with messages = get_flashed_messages() %}
      {% if messages %}
        <div class="alert alert-danger">{{ messages[0] }}</div>
      {% endif %}
    {% endwith %}
    {% block content %}{% endblock %}
</div>
<footer class="footer"><hr>Created by Umariqbal777 &middot; © {{ 2025 }}</footer>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</body>
</html>
""")

with open(os.path.join(TEMPLATES_DIR, "home.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<div class="row align-items-center g-4">
    <div class="col-md-6">
        <h1 class="fw-bold mb-3">Welcome to AgroScan AI!</h1>
        <p>Analyze crop images with deep learning.</p>
        <ul class="list-group list-group-flush mb-3">
          <li class="list-group-item bg-transparent">🌱 Fast Plant Disease Detection</li>
          <li class="list-group-item bg-transparent">📈 Confidence & Probabilities</li>
          <li class="list-group-item bg-transparent">🔬 EfficientNetB0 Deep Learning</li>
        </ul>
        <a href="{{ url_for('predict_page') }}" class="btn btn-success btn-lg">Try Prediction →</a>
    </div>
    <div class="col-md-6 text-center">
        <img src="{{ url_for('static', filename='uploads/sample_leaf.png') }}" class="rounded shadow w-100" style="max-width:400px;">
    </div>
</div>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "predict.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2 class="mb-4">Leaf Disease Prediction</h2>
{% if model_loading %}
<div class="alert alert-warning">Model is loading, please wait ...</div>
{% endif %}
<form id="upload-form" method="POST" enctype="multipart/form-data" action="{{ url_for('predict_page') }}">
    <div class="mb-3">
        <label for="file" class="form-label">Upload image:</label>
        <input type="file" name="file" id="file" class="form-control" accept="image/*" required onchange="previewFile(this)">
    </div>
    <img id="preview-img" src="" class="my-3" style="max-width:300px; display:none;">
    <button type="submit" class="btn btn-success">Predict</button>
</form>
<script>
function previewFile(input){
    let file = input.files[0];
    let reader = new FileReader();
    reader.onload = function(e){
        let img = document.getElementById('preview-img');
        img.src = e.target.result;
        img.style.display = 'block';
    }
    reader.readAsDataURL(file);
}
</script>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "result.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2>Prediction Result</h2>
<p><b>Prediction:</b> <span class="fs-4">{{ pred_name }}</span></p>
<div class="mb-3">
    <b>Confidence:</b>
    <div class="progress" style="max-width:300px;">
      <div class="progress-bar {% if confidence>=85 %}green{% elif confidence>=60 %}yellow{% else %}red{% endif %}"
           role="progressbar"
           style="width:{{ confidence }}%" aria-valuenow="{{ confidence }}" aria-valuemin="0" aria-valuemax="100">
        {{ confidence }}%
      </div>
    </div>
</div>
<img src="{{ image_url }}" class="rounded my-2 shadow" style="max-width:320px;">
<a href="{{ url_for('predict_page') }}" class="btn btn-outline-success mt-3">Try another image</a>
<hr>
<h4>Class Probabilities</h4>
<canvas id="probsChart" width="350" height="220"></canvas>
<script>
const labels = {{ class_labels | tojson }};
const data = {{ class_probs | tojson }};
const bgcolors = data.map((v,i)=>v>85?'#4caf50':(v>60?'#ffc107':'#f44336'));
new Chart(document.getElementById('probsChart'),{
    type:'bar',
    data:{
        labels:labels,
        datasets:[{label:'Probability (%)',data:data,backgroundColor:bgcolors}]
    },
    options:{responsive:true,indexAxis:'y',plugins:{legend:{display:false}}}
});
</script>
{% endblock %}
""")

# Sample image
sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    img.save(sample_img)

# Model loading status
MODEL_PATH = "best_crop_model.h5"
model = None
MODEL_LOADING = True

def load_model():
    global model, MODEL_LOADING
    MODEL_LOADING = True
    print("Loading model...")
    model = tf.keras.models.load_model(MODEL_PATH, compile=False)
    MODEL_LOADING = False
    print("Model loaded!")

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = "agroscan-colab-2025"

@app.route("/")
def home():
    return render_template("home.html")

@app.route("/predict", methods=["GET", "POST"])
def predict_page():
    global model, MODEL_LOADING
    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True)
    if request.method == "POST":
        if "file" not in request.files or request.files["file"].filename == "":
            flash("No image uploaded.")
            return render_template("predict.html")
        file = request.files["file"]
        fname = file.filename
        save_path = os.path.join(UPLOAD_DIR, fname)
        file.save(save_path)
        try:
            img = Image.open(save_path).convert("RGB").resize((128,128))
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            from tensorflow.keras.applications.efficientnet import preprocess_input
            arr = preprocess_input(arr)
            preds = model.predict(arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            pred_name = format_class_name(CLASS_NAMES[idx]) if idx < len(CLASS_NAMES) else str(idx)
            img_url = url_for('static', filename=f'uploads/{fname}')
            probs = [round(float(x)*100,2) for x in preds[0].tolist()]
            return render_template("result.html",
                                   pred_name=pred_name,
                                   confidence=round(conf,2),
                                   image_url=img_url,
                                   class_labels=[format_class_name(x) for x in CLASS_NAMES],
                                   class_probs=probs)
        except Exception as e:
            flash(f"Prediction error: {e}")
            return render_template("predict.html")
    return render_template("predict.html", model_loading=False)

@app.route("/api/predict", methods=["POST"])
def api_predict():
    global model, MODEL_LOADING
    if model is None and MODEL_LOADING:
        return jsonify({"error":"Model is loading. Try again."}), 503
    if 'file' not in request.files or request.files['file'].filename == '':
        return jsonify({"error":"No image uploaded"}), 400
    file = request.files['file']
    try:
        img = Image.open(file).convert("RGB").resize((128,128))
        arr = np.array(img).astype("float32")
        arr = np.expand_dims(arr, 0)
        from tensorflow.keras.applications.efficientnet import preprocess_input
        arr = preprocess_input(arr)
        preds = model.predict(arr)
        idx = int(np.argmax(preds))
        conf = float(np.max(preds) * 100)
        probs = [round(float(x)*100,2) for x in preds[0].tolist()]
        return jsonify({
            "predicted_index": idx,
            "predicted_name": format_class_name(CLASS_NAMES[idx]) if idx < len(CLASS_NAMES) else str(idx),
            "confidence": round(conf,2),
            "class_probs": probs,
            "class_labels": [format_class_name(x) for x in CLASS_NAMES]
        })
    except Exception as e:
        return jsonify({"error": f"Prediction error: {e}"}), 500

# -- ngrok setup --
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"
def _cleanup():
    try: ngrok.kill()
    except: pass
import atexit
atexit.register(_cleanup)

if __name__ == "__main__":
    print("Paste your ngrok authtoken for tunnel URL:")
    token = getpass()
    ngrok.set_auth_token(token)
    public_url = ngrok.connect(5000)
    print(f"🔗 ngrok tunnel: {public_url}")
    # Start model loading before start
    load_model()
    app.run(host="0.0.0.0", port=5000)

Paste your ngrok authtoken for tunnel URL:
··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
Loading model...
Model loaded!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:47:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:47:03] "GET /static/uploads/sample_leaf.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:47:11] "GET /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:47:30] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:47:31] "GET /static/uploads/images%20(4).jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:47:35] "GET /predict HTTP/1.1" 200 -


In [16]:
!pip install flask flask-login pyngrok tensorflow pillow deep-translator

import os, sqlite3, threading, json, atexit
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from getpass import getpass
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from deep_translator import GoogleTranslator

BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")

CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {'en':"English", 'hi':"हिन्दी", 'ur':"اردو"}

# ==== Templates ====
with open(os.path.join(TEMPLATES_DIR, "layout.html"), "w") as f:
    f.write("""
<!doctype html>
<html lang="en">
<head>
    <title>{{ title or _('AgroScan AI') }}</title>
    <meta name="viewport" content="width=device-width,initial-scale=1">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
      body { background: #f6faf3; }
      .navbar { background: #8bc34a; }
      .footer { color: #555; padding: 1rem 0; text-align: center; }
      .progress-bar.green { background-color: #4caf50; }
      .progress-bar.yellow { background-color: #ffc107; }
      .progress-bar.red { background-color: #f44336; }
    </style>
</head>
<body>
<nav class="navbar navbar-expand justify-content-between shadow mb-4">
  <a href="{{ url_for('home') }}" class="navbar-brand text-white px-3 fs-4">🌿 {{ _('AgroScan AI') }}</a>
  <span class="navbar-text px-3">
    {% if current_user.is_authenticated %}
      {{ _('Logged in as') }} <b>{{ current_user.username }}</b>
      <a href="{{ url_for('dashboard') }}" class="link-light px-3">{{ _('Dashboard') }}</a>
      <a href="{{ url_for('predict_page') }}" class="link-light px-3">{{ _('Predict') }}</a>
      <a href="{{ url_for('history') }}" class="link-light px-3">{{ _('History') }}</a>
      <a href="{{ url_for('logout') }}" class="link-light px-3">{{ _('Logout') }}</a>
    {% else %}
      <a href="{{ url_for('login') }}" class="link-light px-3">{{ _('Login') }}</a>
      <a href="{{ url_for('register') }}" class="link-light px-3">{{ _('Register') }}</a>
    {% endif %}
    {% for code, label in languages.items() %}
      <a href="{{ url_for('set_language', lang=code) }}" class="badge text-bg-light mx-1">{{ label }}</a>
    {% endfor %}
  </span>
</nav>
<div class="container my-4">
    {% with messages = get_flashed_messages() %}
      {% if messages %}
        <div class="alert alert-danger">{{ messages[0] }}</div>
      {% endif %}
    {% endwith %}
    {% block content %}{% endblock %}
</div>
<footer class="footer"><hr>{{ _('Created by') }} Umariqbal777 &middot; © {{ 2025 }}</footer>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</body>
</html>
""")

with open(os.path.join(TEMPLATES_DIR, "home.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<div class="row align-items-center g-4">
    <div class="col-md-6">
        <h1 class="fw-bold mb-3">{{ _('Welcome to AgroScan AI!') }}</h1>
        <p>{{ _('Analyze crop images with deep learning.') }}</p>
        <ul class="list-group list-group-flush mb-3">
          <li class="list-group-item bg-transparent">🌱 {{ _('Fast Plant Disease Detection') }}</li>
          <li class="list-group-item bg-transparent">📈 {{ _('Confidence & Probabilities') }}</li>
          <li class="list-group-item bg-transparent">🔬 {{ _('EfficientNetB0 Deep Learning') }}</li>
        </ul>
        {% if current_user.is_authenticated %}
        <a href="{{ url_for('predict_page') }}" class="btn btn-success btn-lg">{{ _('Try Prediction') }} →</a>
        {% else %}
        <a href="{{ url_for('login') }}" class="btn btn-primary btn-lg">{{ _('Login to Get Started') }}</a>
        {% endif %}
    </div>
    <div class="col-md-6 text-center">
        <img src="{{ url_for('static', filename='uploads/sample_leaf.png') }}" class="rounded shadow w-100" style="max-width:400px;">
    </div>
</div>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "login.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2>{{ _('Login') }}</h2>
<form method='post'>
  <label>{{ _('Username') }} <input type="text" name="username" required class="form-control"></label>
  <label>{{ _('Password') }} <input type="password" name="password" required class="form-control"></label>
  <button class="btn btn-success mt-2" type='submit'>{{ _('Login') }}</button>
  <a href="{{ url_for('register') }}" class="btn btn-link">{{ _('Register') }}</a>
</form>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "register.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2>{{ _('Register') }}</h2>
<form method='post'>
  <label>{{ _('Username') }} <input type="text" name="username" required class="form-control"></label>
  <label>{{ _('Password') }} <input type="password" name="password" required class="form-control"></label>
  <button class="btn btn-primary mt-2" type='submit'>{{ _('Register') }}</button>
</form>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "predict.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2 class="mb-4">{{ _('Leaf Disease Prediction') }}</h2>
{% if model_loading %}
<div class="alert alert-warning">{{ _('Model is loading, please wait ...') }}</div>
{% endif %}
<form id="upload-form" method="POST" enctype="multipart/form-data">
    <div class="mb-3">
        <label class="form-label">{{ _('Upload image') }}:</label>
        <input type="file" name="file" id="file" class="form-control" accept="image/*" required onchange="previewFile(this)">
    </div>
    <img id="preview-img" src="" class="my-3" style="max-width:300px; display:none;">
    <button type="submit" class="btn btn-success">{{ _('Predict') }}</button>
</form>
<script>
function previewFile(input){
    let file = input.files[0];
    let reader = new FileReader();
    reader.onload = function(e){
        let img = document.getElementById('preview-img');
        img.src = e.target.result;
        img.style.display = 'block';
    }
    reader.readAsDataURL(file);
}
</script>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "result.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2>{{ _('Prediction Result') }}</h2>
<p><b>{{ _('Prediction') }}:</b> <span class="fs-4">{{ pred_name }}</span></p>
<div class="mb-3">
    <b>{{ _('Confidence') }}:</b>
    <div class="progress" style="max-width:300px;">
      <div class="progress-bar {% if confidence>=85 %}green{% elif confidence>=60 %}yellow{% else %}red{% endif %}"
           role="progressbar"
           style="width:{{ confidence }}%" aria-valuenow="{{ confidence }}" aria-valuemin="0" aria-valuemax="100">
        {{ confidence }}%
      </div>
    </div>
</div>
<img src="{{ image_url }}" class="rounded my-2 shadow" style="max-width:320px;">
<a href="{{ url_for('predict_page') }}" class="btn btn-outline-success mt-3">{{ _('Try another image') }}</a>
<hr>
<h4>{{ _('Class Probabilities') }}</h4>
<canvas id="probsChart" width="350" height="220"></canvas>
<script>
const labels = {{ class_labels | tojson }};
const data = {{ class_probs | tojson }};
const bgcolors = data.map((v,i)=>v>85?'#4caf50':(v>60?'#ffc107':'#f44336'));
new Chart(document.getElementById('probsChart'),{
    type:'bar',
    data:{labels:labels,datasets:[{label:'Probability (%)',data:data,backgroundColor:bgcolors}]},
    options:{responsive:true,indexAxis:'y',plugins:{legend:{display:false}}}
});
</script>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "dashboard.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2>{{ _('User Dashboard') }}</h2>
<p>{{ _('Total Predictions Made') }}: <b>{{ total_pred }}</b></p>
<a href="{{ url_for('history') }}" class="btn btn-info">{{ _('View History') }}</a>
{% endblock %}
""")

with open(os.path.join(TEMPLATES_DIR, "history.html"), "w") as f:
    f.write("""
{% extends 'layout.html' %}
{% block content %}
<h2>{{ _('Prediction History') }}</h2>
{% if history %}
<table class="table table-bordered">
<tr>
  <th>{{ _('Image') }}</th>
  <th>{{ _('Prediction') }}</th>
  <th>{{ _('Confidence') }}</th>
  <th>{{ _('Time') }}</th>
</tr>
{% for h in history %}
<tr>
  <td><img src="{{ h.image_url }}" style="max-width:60px;"></td>
  <td>{{ h.prediction }}</td>
  <td>{{ h.confidence }}%</td>
  <td>{{ h.time }}</td>
</tr>
{% endfor %}
</table>
{% else %}
<p>{{ _('No predictions yet.') }}</p>
{% endif %}
<a href="{{ url_for('dashboard') }}" class="btn btn-secondary">{{ _('Back to Dashboard') }}</a>
{% endblock %}
""")

sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# ==== SQLite User and Prediction Tables ====
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        image_path TEXT NOT NULL,
        prediction TEXT NOT NULL,
        confidence REAL NOT NULL,
        time TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# ==== Flask App ====
app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

# ==== Language Translate ====
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

@app.context_processor
def inject_globals():
    def _(text):
        lang = session.get('language','en')
        return get_translation(text, lang)
    return dict(_=_, languages=LANGUAGES)

@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

# ==== EfficientNetB0 Model ====
MODEL_PATH = "best_crop_model.h5"
model = None
MODEL_LOADING = True
def load_model():
    global model, MODEL_LOADING
    MODEL_LOADING = True
    print("Loading model...")
    model = tf.keras.models.load_model(MODEL_PATH, compile=False)
    MODEL_LOADING = False
    print("Model loaded!")

# ==== Routes ====
@app.route("/")
def home():
    return render_template("home.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("dashboard"))
        flash("Invalid username or password.")
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash("Registration successful! Please login."); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash("Username already taken.")
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("SELECT COUNT(*) FROM predictions WHERE user_id=?",(current_user.id,))
    total = cur.fetchone()[0]
    conn.close()
    return render_template("dashboard.html", total_pred=total)

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT * FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    for r in rows:
        ts = r[5]; # time
        img_url = r[3]
        hist.append({"image_url":img_url,"prediction":r[4],"confidence":int(r[5]),"time":ts})
    return render_template("history.html", history=hist)

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global model, MODEL_LOADING
    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True)
    if request.method == "POST":
        if "file" not in request.files or request.files["file"].filename == "":
            flash("No image uploaded.")
            return render_template("predict.html", model_loading=False)
        file = request.files["file"]
        fname = secure_filename(file.filename)
        save_path = os.path.join(UPLOAD_DIR, fname)
        file.save(save_path)
        try:
            img = Image.open(save_path).convert("RGB").resize((128,128))
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            from tensorflow.keras.applications.efficientnet import preprocess_input
            arr = preprocess_input(arr)
            preds = model.predict(arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            pred_name = format_class_name(CLASS_NAMES[idx]) if idx < len(CLASS_NAMES) else str(idx)
            img_url = url_for('static', filename=f'uploads/{fname}')
            probs = [round(float(x)*100,2) for x in preds[0].tolist()]
            # Save to user history
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time) VALUES (?,?,?,?,datetime('now'))",
                        (current_user.id, img_url, pred_name, conf, conf))
            conn.commit(); conn.close()
            return render_template("result.html", pred_name=pred_name, confidence=round(conf,2),
                                   image_url=img_url, class_labels=[format_class_name(x) for x in CLASS_NAMES], class_probs=probs)
        except Exception as e:
            flash(f"Prediction error: {e}")
            return render_template("predict.html", model_loading=False)
    return render_template("predict.html", model_loading=False)

@app.route("/api/predict", methods=["POST"])
def api_predict():
    global model, MODEL_LOADING
    if model is None and MODEL_LOADING:
        return jsonify({"error": "Model is loading. Try again."}), 503
    if 'file' not in request.files or request.files['file'].filename == '':
        return jsonify({"error":"No image uploaded"}), 400
    file = request.files['file']
    try:
        img = Image.open(file).convert("RGB").resize((128,128))
        arr = np.array(img).astype("float32")
        arr = np.expand_dims(arr, 0)
        from tensorflow.keras.applications.efficientnet import preprocess_input
        arr = preprocess_input(arr)
        preds = model.predict(arr)
        idx = int(np.argmax(preds))
        conf = float(np.max(preds) * 100)
        probs = [round(float(x)*100,2) for x in preds[0].tolist()]
        return jsonify({
            "predicted_index": idx,
            "predicted_name": format_class_name(CLASS_NAMES[idx]) if idx < len(CLASS_NAMES) else str(idx),
            "confidence": round(conf,2),
            "class_probs": probs,
            "class_labels": [format_class_name(x) for x in CLASS_NAMES]
        })
    except Exception as e:
        return jsonify({"error": f"Prediction error: {e}"}), 500

os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    print("Paste your ngrok authtoken for tunnel URL:")
    token = getpass()
    ngrok.set_auth_token(token)
    public_url = ngrok.connect(5000)
    print(f"🔗 ngrok tunnel: {public_url}")
    load_model()
    app.run(host="0.0.0.0", port=5000)

Paste your ngrok authtoken for tunnel URL:
··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
Loading model...
Model loaded!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:16] "GET /static/uploads/sample_leaf.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:26] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:31] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:37] "POST /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:44] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:50] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:50] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 21:50:54] "GET 

In [19]:
!pip install flask flask-login pyngrok tensorflow pillow deep-translator

import os, sqlite3, json, atexit
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from getpass import getpass
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from datetime import datetime
from deep_translator import GoogleTranslator

BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")

CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# Remove DB to ensure new schema!
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        image_path TEXT NOT NULL,
        prediction TEXT NOT NULL,
        confidence REAL NOT NULL,
        time TEXT NOT NULL,
        FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# ---- HTML templates remain exactly as previous code/context ----

sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

@app.context_processor
def inject_globals():
    def _(text):
        lang = session.get('language','en')
        return get_translation(text, lang)
    return dict(_=_, languages=LANGUAGES)

@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

MODEL_PATH = "best_crop_model.h5"
model = None
MODEL_LOADING = True
def load_model():
    global model, MODEL_LOADING
    MODEL_LOADING = True
    print("Loading model...")
    model = tf.keras.models.load_model(MODEL_PATH, compile=False)
    MODEL_LOADING = False
    print("Model loaded!")

@app.route("/")
def home():
    return render_template("home.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("dashboard"))
        flash("Invalid username or password.")
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash("Registration successful! Please login."); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash("Username already taken.")
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("SELECT COUNT(*) FROM predictions WHERE user_id=?",(current_user.id,))
    total = cur.fetchone()[0]
    conn.close()
    return render_template("dashboard.html", total_pred=total)

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    for img_url,pred,conf,time in rows:
        hist.append({
            "image_url":img_url,"prediction":pred,
            "confidence":int(round(conf)), "time":time
        })
    return render_template("history.html", history=hist)

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global model, MODEL_LOADING
    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True)
    if request.method == "POST":
        if "file" not in request.files or request.files["file"].filename == "":
            flash("No image uploaded.")
            return render_template("predict.html", model_loading=False)
        file = request.files["file"]
        fname = secure_filename(file.filename)
        save_path = os.path.join(UPLOAD_DIR, fname)
        file.save(save_path)
        try:
            img = Image.open(save_path).convert("RGB").resize((128,128))
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            from tensorflow.keras.applications.efficientnet import preprocess_input
            arr = preprocess_input(arr)
            preds = model.predict(arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            pred_name = format_class_name(CLASS_NAMES[idx]) if idx < len(CLASS_NAMES) else str(idx)
            img_url = url_for('static', filename=f'uploads/{fname}')
            probs = [round(float(x)*100,2) for x in preds[0].tolist()]
            # Save to user history -- FIXED COLUMN BINDINGS
            timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time) VALUES (?,?,?,?,?)",
                        (current_user.id, img_url, pred_name, conf, timestamp))
            conn.commit(); conn.close()
            return render_template("result.html", pred_name=pred_name, confidence=round(conf,2),
                                   image_url=img_url, class_labels=[format_class_name(x) for x in CLASS_NAMES], class_probs=probs)
        except Exception as e:
            flash(f"Prediction error: {e}")
            return render_template("predict.html", model_loading=False)
    return render_template("predict.html", model_loading=False)

@app.route("/api/predict", methods=["POST"])
def api_predict():
    global model, MODEL_LOADING
    if model is None and MODEL_LOADING:
        return jsonify({"error": "Model is loading. Try again."}), 503
    if 'file' not in request.files or request.files['file'].filename == '':
        return jsonify({"error":"No image uploaded"}), 400
    file = request.files['file']
    try:
        img = Image.open(file).convert("RGB").resize((128,128))
        arr = np.array(img).astype("float32")
        arr = np.expand_dims(arr, 0)
        from tensorflow.keras.applications.efficientnet import preprocess_input
        arr = preprocess_input(arr)
        preds = model.predict(arr)
        idx = int(np.argmax(preds))
        conf = float(np.max(preds) * 100)
        probs = [round(float(x)*100,2) for x in preds[0].tolist()]
        return jsonify({
            "predicted_index": idx,
            "predicted_name": format_class_name(CLASS_NAMES[idx]) if idx < len(CLASS_NAMES) else str(idx),
            "confidence": round(conf,2),
            "class_probs": probs,
            "class_labels": [format_class_name(x) for x in CLASS_NAMES]
        })
    except Exception as e:
        return jsonify({"error": f"Prediction error: {e}"}), 500

os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    print("Paste your ngrok authtoken for tunnel URL:")
    token = getpass()
    ngrok.set_auth_token(token)
    public_url = ngrok.connect(5000)
    print(f"🔗 ngrok tunnel: {public_url}")
    load_model()
    app.run(host="0.0.0.0", port=5000)

Paste your ngrok authtoken for tunnel URL:
··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
Loading model...
Model loaded!
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:27] "GET /static/uploads/sample_leaf.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:30] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:32] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:36] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:37] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:42] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:43] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:00:48] "GET 

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


/tmp/ipython-input-855625953.py:206: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:01:05] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:01:06] "GET /static/uploads/download_5.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:01:11] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:01:12] "GET /static/uploads/download_5.jpeg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:01:16] "GET /dashboard HTTP/1.1" 200 -


In [20]:
# above code works


In [22]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINAL STABLE & FIXED SCRIPT (Tailwind UI)
# =========================================================================
#  -- FIXES: Translation TypeError, Model Used DB Schema/Insert --
# =========================================================================

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, threading, sqlite3, json, atexit
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from getpass import getpass
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from datetime import datetime, timedelta
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input # CRITICAL IMPORT

# -------------------
# 2) Config, File System, and Constants
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
IMAGE_SIZE = (128, 128)
ALLOWED_EXT = {"png","jpg","jpeg"}
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# Model paths and global state
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
MODEL_LOADING = True

# -------------------
# 3) Class names, Formatting, and Languages
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# -------------------
# 4) Database Setup (Updated for dual-model schema)
# -------------------
if os.path.exists(DB_PATH): os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,  -- ADDED: Required for dual model feature
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     time TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Sample image generation (simplified)
sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# -------------------
# 5) Disease Information
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.','remedies':[]},
    'Potato___Early_blight': {'description':'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies':['Use certified seed','Apply fungicides containing mancozeb or chlorothalonil','Rotate crops']},
    'Potato___Late_blight': {'description':'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside.', 'remedies':['Remove infected plants','Protective fungicides proactively','Ensure good drainage and air circulation']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description':'Causes small, water-soaked spots on tomato leaves and fruit.','remedies':['Avoid working when wet','Use copper-based bactericides','Mulch around plants']},
    'Tomato_Early_blight': {'description':'Fungal disease, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies':['Ensure proper spacing','Apply preventative fungicides','Water at the base of the plant']},
    'Tomato_Late_blight': {'description':'A serious fungal disease characterized by large, greasy, grey-green spots on leaves.','remedies':['Apply preventative fungicides like chlorothalonil or copper','Remove and destroy infected plants immediately']},
    'Tomato_Leaf_Mold': {'description':'Fungal disease causing pale spots on the upper leaf surface and a velvety, olive-green mold on the underside.','remedies':['Improve air circulation and reduce humidity','Prune lower leaves']},
    'Tomato_Septoria_leaf_spot': {'description':'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.','remedies':['Remove infected leaves','Improve air circulation','Apply fungicides']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description':'Tiny pests causing stippling (tiny yellow or white dots) on leaves, leading to webbing.', 'remedies':['Spray plants with water','Apply insecticidal soap or horticultural oil','Introduce natural predators']},
    'Tomato__Target_Spot': {'description':'A fungal disease causing necrotic spots with concentric rings.', 'remedies':['Apply fungicides effective against Corynespora cassiicola','Improve air circulation and reduce leaf wetness']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description':'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies':['Control whitefly populations','Remove and destroy infected plants immediately','Use virus-resistant varieties']},
    'Tomato__Tomato_mosaic_virus': {'description':'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation.', 'remedies':['There is no cure; remove and destroy infected plants','Wash hands and tools thoroughly']},
    'Tomato_healthy': {'description':'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.','remedies':[]},
    'default': {'description':'Information for this condition is not available.','remedies':['Consult local extension services']}
}

# -------------------
# 6) Model Management & Translation Setup
# -------------------
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# Route-safe translation helper
def tr_route(text):
    # Used inside routes (e.g., for flash messages)
    lang = session.get('language', 'en')
    return get_translation(text, lang)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_globals():
    # Template usage: {{ _('Text') }}
    def tr(text):
        return tr_route(text)
    return dict(_=tr, languages=LANGUAGES) # Exposes '_' for templates

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = tf.keras.models.load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

def _background_preload():
    global MODEL_LOADING
    MODEL_LOADING = True
    for k in ["best", "fallback"]:
        load_model_lazy(k)
    MODEL_LOADING = False
    print("ALL models preloaded and ready.")

# -------------------
# 7) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

@app.route("/")
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template("home.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("dashboard"))
        # FIX APPLIED: Use tr_route() instead of the conflicting _()
        flash(tr_route("Invalid username or password."))
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash(tr_route("Registration successful! Please login.")); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash(tr_route("Username already taken."))
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    global ACTIVE_MODEL_KEY
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    rows = cur.fetchall();
    disease_data = {'labels':[r[0] for r in rows], 'values':[r[1] for r in rows]}
    conn.close()
    return render_template("dashboard.html", disease_data=disease_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(tr_route(f"Active model switched to: {m}"))
    else:
        flash(tr_route("Unknown model key"))
    return redirect(url_for('dashboard'))

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time, model_used FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    ist_offset = timedelta(hours=5, minutes=30)
    for img_url,pred,conf,ts,model_used in rows:
        try: ts_dt = datetime.fromisoformat(ts)
        except: ts_dt = datetime.strptime(ts.split('.')[0], "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + ist_offset
        hist.append({
            "image_path":img_url,"prediction":pred,
            "confidence":int(round(conf)), "timestamp":ist.strftime("%b %d, %Y %I:%M %p"),
            "model_used": model_used
        })
    return render_template("history.html", history=hist)

@app.route("/metrics")
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global MODEL_LOADING, ACTIVE_MODEL_KEY
    model = MODEL_REGISTRY.get(ACTIVE_MODEL_KEY)

    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True)
    if model is None:
        flash(tr_route(f"Model {ACTIVE_MODEL_KEY} failed to load. Please check files and re-run."))
        return render_template("predict.html", model_loading=False)

    if request.method == "POST":
        if "file" not in request.files or request.files["file"].filename == "":
            flash(tr_route("No image uploaded."))
            return render_template("predict.html", model_loading=False)

        file = request.files["file"]
        if not allowed_file(file.filename):
            flash(tr_route("Invalid file type. Please upload JPG or PNG."))
            return render_template("predict.html", model_loading=False)

        fname = secure_filename(file.filename)
        unique_fname = f"{int(time.time())}_{fname}"
        save_path = os.path.join(UPLOAD_DIR, unique_fname)
        file.save(save_path)

        try:
            # === CRITICAL EFFICIENTNET PREPROCESSING ===
            img = Image.open(save_path).convert("RGB").resize(IMAGE_SIZE)
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            processed_arr = preprocess_input(arr)

            # Prediction
            preds = model.predict(processed_arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            raw_name = CLASS_NAMES[idx]
            pred_name = format_class_name(raw_name)

            img_url = url_for('static', filename=f'uploads/{unique_fname}')

            # Save to user history (FIXED INSERT STATEMENT)
            timestamp = datetime.utcnow().isoformat()
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time, model_used) VALUES (?,?,?,?,?,?)",
                        (current_user.id, img_url, pred_name, conf, timestamp, ACTIVE_MODEL_KEY))
            conn.commit(); conn.close()

            # Prepare result data (including translation)
            info = disease_info.get(raw_name, disease_info['default'])
            lang = session.get('language','en')
            desc = get_translation(info.get('description', ''), lang)
            remedies = [get_translation(r, lang) for r in info.get('remedies', [])]

            if conf >= 85: conf_level = 'High'
            elif conf >= 60: conf_level = 'Medium'
            else: conf_level = 'Low'

            return render_template("result.html", prediction=pred_name, confidence=round(conf,2),
                                    image_path=img_url, description=desc, remedies=remedies,
                                    confidence_level=conf_level)
        except Exception as e:
            flash(tr_route(f"Prediction error: {e}"))
            return render_template("predict.html", model_loading=False)

    return render_template("predict.html", model_loading=False)

# -------------------
# 8) Tailwind CSS Templates
# -------------------
# (Saving templates using Python print/write - content remains the same)
template_content = {
    "layout.html": """<!doctype html><html lang='{{ session.get("language", "en") }}'><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><title>{{ _(title or 'AgroScan AI') }}</title><style>body {min-height: 100vh;} main {flex: 1;}</style></head><body class="bg-gray-50 text-gray-800 flex flex-col"><nav class="bg-white/95 fixed w-full z-30 shadow"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="currentColor"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 17.93c-2.82-.44-5.02-2.31-6.12-4.94L13 14h-2c-2.76 0-5 2.24-5 5H4c0-3.92 2.68-7.23 6.33-8.15C10.12 11.23 11.06 12 12 12s1.88-.77 2.67-1.15c.16-.08.31-.17.46-.26 3.65.92 6.33 4.23 6.33 8.15h-2c0-2.76-2.24-5-5-5h-2l6.12 4.94z"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="flex items-center space-x-4"><div class="hidden md:flex items-center space-x-4">{% if current_user.is_authenticated %}<span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span><a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600'>{{ _('Dashboard') }}</a><a href='{{ url_for("predict_page") }}' class='text-gray-600 hover:text-green-600'>{{ _('Analyze') }}</a><a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600'>{{ _('History') }}</a><a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600'>{{ _('Metrics') }}</a><a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold'>{{ _('Logout') }}</a>{% else %}<a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600'>{{ _('Login') }}</a><a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600'>{{ _('Register') }}</a>{% endif %}</div><div class="relative group"><button class="text-gray-600 hover:text-green-600 flex items-center"><svg class="w-5 h-5 mr-1" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M11.496 15.343a.75.75 0 01-1.06 0L5.94 10.844a.75.75 0 011.06-1.06l3.496 3.496 3.496-3.496a.75.75 0 011.06 1.06l-4.496 4.496z" clip-rule="evenodd" /></svg></button><div class="absolute right-0 mt-2 w-32 bg-white border border-gray-200 rounded-md shadow-lg hidden group-hover:block z-50">{% for lang_code, lang_name in languages.items() %}<a href='{{ url_for("set_language", lang=lang_code) }}' class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100">{{ lang_name }}</a>{% endfor %}</div></div></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6 flex-grow'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6 border-t mt-auto w-full'>© 2025 AgroScan AI - {{ _('Model') }}: {{ ACTIVE_MODEL_KEY or 'N/A' }}</footer></body></html>"""
    , "home.html": """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-8 items-center bg-white p-8 rounded-xl shadow-lg'><div class='order-2 md:order-1'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900'>{{ _('Revolutionize Your Farming with AI') }}</h1><p class='mt-4 text-lg text-gray-600'>{{ _('Instantly detect crop diseases with a simple photo. Get actionable advice to protect your harvest and increase your yield.') }}</p><div class='mt-8 flex space-x-4'>{% if current_user.is_authenticated %}<a href='{{ url_for("predict_page") }}' class='bg-green-600 hover:bg-green-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200'>{{ _('Start Analysis') }}</a><a href='{{ url_for("dashboard") }}' class='border border-gray-300 hover:bg-gray-100 text-gray-700 px-6 py-3 rounded-lg font-bold transition duration-200'>{{ _('View Dashboard') }}</a>{% else %}<a href='{{ url_for("login") }}' class='bg-blue-600 hover:bg-blue-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200'>{{ _('Login to Start') }}</a>{% endif %}</div></div><div class='order-1 md:order-2'><img src='{{ url_for("static", filename="uploads/sample_leaf.png") }}' class='rounded-lg shadow-xl w-full' alt='Sample Leaf Image'></div></div>{% endblock %}"""
    , "login.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto bg-white p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Login') }}</h2><form method='post' action='{{ url_for("login") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200'>{{ _('Sign in') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Not a member?') }} <a href='{{ url_for("register") }}' class='text-green-600 hover:text-green-700'>{{ _('Register here') }}</a></p></div>{% endblock %}"""
    , "register.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto bg-white p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Register') }}</h2><form method='post' action='{{ url_for("register") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200'>{{ _('Register') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Already a member?') }} <a href='{{ url_for("login") }}' class='text-green-600 hover:text-green-700'>{{ _('Login here') }}</a></p></div>{% endblock %}"""
    , "predict.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto bg-white p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Predict Disease') }}</h2>{% if model_loading %}<div class='text-center p-8 bg-yellow-50 rounded-lg'><div class='animate-spin w-8 h-8 border-4 border-yellow-600 border-t-transparent rounded-full mx-auto'></div><p class='mt-4 text-lg text-yellow-800 font-semibold'>{{ _('Model is loading...') }}</p><p class='text-sm text-yellow-700'>{{ _('This may take up to 60 seconds. Please wait on this page.') }}</p></div>{% else %}<form id='upload-form' method='post' action='{{ url_for("predict_page") }}' enctype='multipart/form-data'><label for='file-upload' class='block border-2 border-dashed border-gray-300 p-10 rounded-xl cursor-pointer hover:border-green-500 transition duration-200'><input id='file-upload' name='file' type='file' accept='image/jpeg, image/png, image/jpg' class='hidden'><div class='text-center'><svg class='mx-auto w-12 h-12 text-gray-400' fill='none' stroke='currentColor' viewBox='0 0 24 24'><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M7 16a4 4 0 01-4-4V7a4 4 0 014-4h10a4 4 0 014 4v5a4 4 0 01-4 4h-3'></path><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M16 12l-4-4m0 0L8 12m4-4v11'></path></svg><p class='mt-2 text-gray-600 font-semibold'>{{ _('Click to upload leaf image') }}</p><p id='file-info' class='mt-1 text-sm text-gray-500'></p></div></label><button id='submit-button' class='w-full mt-6 bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200'>{{ _('Detect Disease') }}</button><div id='loader' class='hidden w-full flex items-center justify-center p-3 bg-gray-100 rounded-lg mt-6'><div class='animate-spin w-6 h-6 border-4 border-green-600 border-t-transparent rounded-full mr-3'></div><span>{{ _('Analyzing...') }}</span></div></form></div><script>const fileUpload=document.getElementById('file-upload'), form=document.getElementById('upload-form'), fileInfo=document.getElementById('file-info'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); fileUpload.addEventListener('change', e=>{ const f=e.target.files[0]; if(f) fileInfo.textContent = `{{ _('File') }}: ${f.name}`; }); form.addEventListener('submit', e=>{ if(!fileUpload.files.length){ e.preventDefault(); alert('{{ _('Please select an image to upload.') }}'); return; } submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endif %}{% endblock %}"""
    , "result.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto bg-white p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Detection Result') }}</h2><div class='grid md:grid-cols-2 gap-8 items-start'><div class='md:w-full'><img src='{{ image_path }}' class='rounded-lg w-full shadow-md' alt='Uploaded Leaf'></div><div class='md:w-full space-y-5'><div class='bg-gray-50 p-5 rounded-lg border-l-4 border-green-600'><p class='text-lg text-gray-600 mb-1'>{{ _('Predicted Condition') }}:</p><h3 class='text-2xl font-extrabold text-green-700'>{{ prediction }}</h3><p class='mt-2 text-gray-600'>{{ _('Confidence') }}: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-4 mt-3 overflow-hidden' title='{{ confidence }}%'>{% if confidence_level == 'High' %}<div class='h-4 bg-green-500' style='width: {{ confidence }}%;'></div>{% elif confidence_level == 'Medium' %}<div class='h-4 bg-yellow-500' style='width: {{ confidence }}%;'></div>{% else %}<div class='h-4 bg-red-500' style='width: {{ confidence }}%;'></div>{% endif %}</div></div><div class='bg-blue-50 p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('About this Condition') }}</h4><p class='text-gray-700'>{{ description }}</p></div>{% if remedies %}<div class='bg-green-50 p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('Suggested Actions') }}</h4><ul class='list-disc pl-5 text-gray-700'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}</div></div><div class='mt-8 text-center'><a href='{{ url_for("predict_page") }}' class='bg-gray-600 hover:bg-gray-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200'>{{ _('Analyze Another Image') }}</a></div></div>{% endblock %}"""
    , "dashboard.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Dashboard') }}</h2><div class='bg-white p-6 rounded-xl shadow-xl'><h3 class='text-xl font-bold mb-4'>{{ _('Model Configuration') }}</h3><div class='mt-4'><form method='post' action='{{ url_for("switch_model") }}'><label class='block text-gray-700'>{{ _('Select active model') }}:<select name='model' class='mt-2 p-2 border rounded-lg w-full md:w-1/3'> <option value='best' {% if active_model=='best' %}selected{% endif %}>{{ _('Best model') }} (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>{{ _('Fallback model') }} (crop_disease_model.h5)</option></select></label><div class='mt-4'><button class='bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold'>{{ _('Switch Model') }}</button></div></form></div><h3 class='text-xl font-bold mt-8 mb-4'>{{ _('Your Prediction Statistics') }}</h3>{% if disease_data.labels %}<ul class='space-y-2'>{% for i in range(disease_data.labels|length) %}<li class='flex justify-between p-2 border-b'><span class='font-semibold'>{{ disease_data.labels[i] }}</span><span>{{ disease_data.values[i] }} {{ _('times detected') }}</span></li>{% endfor %}</ul>{% else %}<p class='text-gray-500'>{{ _('No detections yet. Analyze to populate your dashboard.') }}</p>{% endif %}</div>{% endblock %}"""
    , "history.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Prediction History') }}</h2>{% if history %}<div class='grid md:grid-cols-3 gap-6'>{% for p in history %}<div class='bg-white rounded-xl shadow-md overflow-hidden hover:shadow-lg transition duration-200'><img src='{{ p.image_path }}' class='w-full h-40 object-cover' alt='Uploaded Leaf'><div class='p-4'><div class='text-sm text-gray-500'>{{ p.timestamp }}</div><div class='font-bold text-lg mt-1'>{{ p.prediction }}</div><div class='text-sm text-gray-600'>{{ _('Confidence') }}: {{ p.confidence | int }}%</div><div class='text-xs text-gray-500'>{{ _('Model') }}: {{ p.model_used }}</div></div></div>{% endfor %}</div>{% else %}<p class='text-gray-500'>{{ _('No saved predictions yet.') }}</p>{% endif %}{% endblock %}"""
    , "metrics.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Training Metrics') }}</h2><div class='bg-white p-6 rounded-xl shadow-xl'><p class='text-gray-600 mb-4'>{{ _('The files below are loaded from your Colab root directory.') }}</p>{% if training_results %}<h3 class='text-xl font-bold mt-4'>{{ _('Final Training Results') }}</h3><pre class='whitespace-pre-wrap bg-gray-100 p-4 rounded-lg text-sm'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}{% if confusion_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Confusion Matrix') }}</h3><img src='{{ url_for("static", filename="confusion_matrix.png") }}' class='max-w-full rounded-lg shadow-md mt-2' alt='Confusion Matrix'>{% endif %}{% if report_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Classification Report') }}</h3><pre class='bg-gray-100 p-4 rounded-lg text-sm'>{{ report_text }}</pre>{% endif %}{% if not training_results and not confusion_exists and not report_exists %}<p class='text-gray-500'>{{ _('No training metrics files found (training_results.json, confusion_matrix.png, classification_report.txt).') }}</p>{% endif %}</div>{% endblock %}"""
}

# Write HTML templates to files
for name, content in template_content.items():
    with open(os.path.join(TEMPLATES_DIR, name), "w", encoding="utf-8") as f:
        f.write(content)

# -------------------
# 9) Start Ngrok and Flask
# -------------------
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    # Start background model loading immediately
    threading.Thread(target=_background_preload, daemon=True).start()

    print("Colab Flask app (dual-model, Tailwind UI) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("\nPaste your ngrok authtoken below. DO NOT re-run if model is still loading.")

    # Setup Ngrok
    try:
        token = getpass("ngrok authtoken (hidden): ")
        if token and token.strip():
            ngrok.set_auth_token(token.strip())
            public_url = ngrok.connect(5000)
            print(f"🔗 ngrok tunnel: {public_url}")
        else:
            print("Ngrok token not provided. App will only be accessible via local Colab link.")
    except Exception as e:
        print(f"Ngrok setup failed: {e}")

    # Run Flask
    app.run(host="0.0.0.0", port=5000, debug=False)

Colab Flask app (dual-model, Tailwind UI) starting...[model] Loading 'best' from best_crop_model.h5 ...

Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)

Paste your ngrok authtoken below. DO NOT re-run if model is still loading.
[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ALL models preloaded and ready.
ngrok authtoken (hidden): ··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:07:56] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:07:57] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:08:02] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:08:07] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:08:08] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:08:14] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:08:15] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:08:38] "GET /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step


/tmp/ipython-input-1759309285.py:338: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat()
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:08:54] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:08:55] "GET /static/uploads/1763676523_images_4.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:09:07] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:09:08] "GET /static/confusion_matrix.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:09:46] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:09:46] "GET /static/confusion_matrix.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:09:52] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:09:53] "GET /static/uploads/176

In [25]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINAL STABLE & FIXED SCRIPT (Tailwind UI)
# =========================================================================
#  -- FIXES: Language Switch Functionality, Language Button/Dropdown --
# =========================================================================

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, time, threading, sqlite3, json, atexit
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

# -------------------
# 2) Config, File System, and Constants
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
IMAGE_SIZE = (128, 128)
ALLOWED_EXT = {"png","jpg","jpeg"}
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# Model paths and global state
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
MODEL_LOADING = True

# -------------------
# 3) Class names, Formatting, and Languages
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# -------------------
# 4) Database Setup (Updated for dual-model schema)
# -------------------
if os.path.exists(DB_PATH): os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,  -- ADDED: Required for dual model feature
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     time TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Sample image generation (simplified)
sample_img = os.path.join(UPLOAD_DIR, "sample_leaf.png")
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# -------------------
# 5) Disease Information
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.','remedies':[]},
    'Potato___Early_blight': {'description':'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies':['Use certified seed','Apply fungicides containing mancozeb or chlorothalonil','Rotate crops']},
    'Potato___Late_blight': {'description':'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside.', 'remedies':['Remove infected plants','Protective fungicides proactively','Ensure good drainage and air circulation']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description':'Causes small, water-soaked spots on tomato leaves and fruit.','remedies':['Avoid working when wet','Use copper-based bactericides','Mulch around plants']},
    'Tomato_Early_blight': {'description':'Fungal disease, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies':['Ensure proper spacing','Apply preventative fungicides','Water at the base of the plant']},
    'Tomato_Late_blight': {'description':'A serious fungal disease characterized by large, greasy, grey-green spots on leaves.','remedies':['Apply preventative fungicides like chlorothalonil or copper','Remove and destroy infected plants immediately']},
    'Tomato_Leaf_Mold': {'description':'Fungal disease causing pale spots on the upper leaf surface and a velvety, olive-green mold on the underside.','remedies':['Improve air circulation and reduce humidity','Prune lower leaves']},
    'Tomato_Septoria_leaf_spot': {'description':'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.','remedies':['Remove infected leaves','Improve air circulation','Apply fungicides']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description':'Tiny pests causing stippling (tiny yellow or white dots) on leaves, leading to webbing.', 'remedies':['Spray plants with water','Apply insecticidal soap or horticultural oil','Introduce natural predators']},
    'Tomato__Target_Spot': {'description':'A fungal disease causing necrotic spots with concentric rings.', 'remedies':['Apply fungicides effective against Corynespora cassiicola','Improve air circulation and reduce leaf wetness']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description':'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies':['Control whitefly populations','Remove and destroy infected plants immediately','Use virus-resistant varieties']},
    'Tomato__Tomato_mosaic_virus': {'description':'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation.', 'remedies':['There is no cure; remove and destroy infected plants','Wash hands and tools thoroughly']},
    'Tomato_healthy': {'description':'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.','remedies':[]},
    'default': {'description':'Information for this condition is not available.','remedies':['Consult local extension services']}
}

# -------------------
# 6) Model Management & Translation Setup
# -------------------
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# Route-safe translation helper
def tr_route(text):
    # Used inside routes (e.g., for flash messages)
    lang = session.get('language', 'en')
    return get_translation(text, lang)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_globals():
    # Template usage: {{ _('Text') }}
    def tr(text):
        return tr_route(text)
    return dict(_=tr, languages=LANGUAGES, ACTIVE_MODEL_KEY=ACTIVE_MODEL_KEY) # Exposes '_' for templates

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = tf.keras.models.load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

def _background_preload():
    global MODEL_LOADING
    MODEL_LOADING = True
    for k in ["best", "fallback"]:
        load_model_lazy(k)
    MODEL_LOADING = False
    print("ALL models preloaded and ready.")

# -------------------
# 7) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    # FIX: Redirect back to the page the user was on
    return redirect(request.referrer or url_for('home'))

@app.route("/")
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template("home.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("dashboard"))
        flash(tr_route("Invalid username or password."))
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash(tr_route("Registration successful! Please login.")); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash(tr_route("Username already taken."))
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    global ACTIVE_MODEL_KEY
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    rows = cur.fetchall();
    disease_data = {'labels':[r[0] for r in rows], 'values':[r[1] for r in rows]}
    conn.close()
    return render_template("dashboard.html", disease_data=disease_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(tr_route(f"Active model switched to: {m}"))
    else:
        flash(tr_route("Unknown model key"))
    return redirect(url_for('dashboard'))

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time, model_used FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    ist_offset = timedelta(hours=5, minutes=30)
    for img_url,pred,conf,ts,model_used in rows:
        try: ts_dt = datetime.fromisoformat(ts)
        except: ts_dt = datetime.strptime(ts.split('.')[0], "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + ist_offset
        hist.append({
            "image_path":img_url,"prediction":pred,
            "confidence":int(round(conf)), "timestamp":ist.strftime("%b %d, %Y %I:%M %p"),
            "model_used": model_used
        })
    return render_template("history.html", history=hist)

@app.route("/metrics")
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global MODEL_LOADING, ACTIVE_MODEL_KEY
    model = MODEL_REGISTRY.get(ACTIVE_MODEL_KEY)

    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True)
    if model is None:
        flash(tr_route(f"Model {ACTIVE_MODEL_KEY} failed to load. Please check files and re-run."))
        return render_template("predict.html", model_loading=False)

    if request.method == "POST":
        if "file" not in request.files or request.files["file"].filename == "":
            flash(tr_route("No image uploaded."))
            return render_template("predict.html", model_loading=False)

        file = request.files["file"]
        if not allowed_file(file.filename):
            flash(tr_route("Invalid file type. Please upload JPG or PNG."))
            return render_template("predict.html", model_loading=False)

        fname = secure_filename(file.filename)
        unique_fname = f"{int(time.time())}_{fname}"
        save_path = os.path.join(UPLOAD_DIR, unique_fname)
        file.save(save_path)

        try:
            # === CRITICAL EFFICIENTNET PREPROCESSING ===
            img = Image.open(save_path).convert("RGB").resize(IMAGE_SIZE)
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            processed_arr = preprocess_input(arr)

            # Prediction
            preds = model.predict(processed_arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            raw_name = CLASS_NAMES[idx]
            pred_name = format_class_name(raw_name)

            img_url = url_for('static', filename=f'uploads/{unique_fname}')

            # Save to user history
            timestamp = datetime.utcnow().isoformat()
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time, model_used) VALUES (?,?,?,?,?,?)",
                        (current_user.id, img_url, pred_name, conf, timestamp, ACTIVE_MODEL_KEY))
            conn.commit(); conn.close()

            # Prepare result data (including translation)
            info = disease_info.get(raw_name, disease_info['default'])
            lang = session.get('language','en')
            desc = get_translation(info.get('description', ''), lang)
            remedies = [get_translation(r, lang) for r in info.get('remedies', [])]

            if conf >= 85: conf_level = 'High'
            elif conf >= 60: conf_level = 'Medium'
            else: conf_level = 'Low'

            return render_template("result.html", prediction=pred_name, confidence=round(conf,2),
                                    image_path=img_url, description=desc, remedies=remedies,
                                    confidence_level=conf_level)
        except Exception as e:
            flash(tr_route(f"Prediction error: {e}"))
            return render_template("predict.html", model_loading=False)

    return render_template("predict.html", model_loading=False)

# -------------------
# 8) Tailwind CSS Templates (FIXED Language Dropdown)
# -------------------
# (Saving templates using Python print/write - content remains the same)
template_content = {
    "layout.html": """<!doctype html><html lang='{{ session.get("language", "en") }}'><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><script src="https://cdn.jsdelivr.net/gh/alpinejs/alpine@v2.x.x/dist/alpine.min.js" defer></script><title>{{ _(title or 'AgroScan AI') }}</title><style>body {min-height: 100vh;} main {flex: 1;}</style></head><body class="bg-gray-50 text-gray-800 flex flex-col"><nav class="bg-white/95 fixed w-full z-30 shadow"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="currentColor"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 17.93c-2.82-.44-5.02-2.31-6.12-4.94L13 14h-2c-2.76 0-5 2.24-5 5H4c0-3.92 2.68-7.23 6.33-8.15C10.12 11.23 11.06 12 12 12s1.88-.77 2.67-1.15c.16-.08.31-.17.46-.26 3.65.92 6.33 4.23 6.33 8.15h-2c0-2.76-2.24-5-5-5h-2l6.12 4.94z"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="flex items-center space-x-4"><div class="hidden md:flex items-center space-x-4">{% if current_user.is_authenticated %}<span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span><a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600'>{{ _('Dashboard') }}</a><a href='{{ url_for("predict_page") }}' class='text-gray-600 hover:text-green-600'>{{ _('Analyze') }}</a><a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600'>{{ _('History') }}</a><a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600'>{{ _('Metrics') }}</a><a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold'>{{ _('Logout') }}</a>{% else %}<a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600'>{{ _('Login') }}</a><a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600'>{{ _('Register') }}</a>{% endif %}</div><div x-data="{ open: false }" class="relative"><button @click="open = !open" class="text-gray-600 hover:text-green-600 flex items-center p-2 rounded-lg hover:bg-gray-100 transition duration-150"><svg class="w-5 h-5 mr-1" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M7.75 2.75a.75.75 0 00-1.5 0v1.258a32.987 32.987 0 00-3.599.278.75.75 0 10.198 1.487A31.545 31.545 0 018.7 5.545 19.38 19.38 0 017 9.762V11.5a.75.75 0 001.5 0V9.762c0-.683.085-1.35.25-1.994.169-.65.4-1.28.68-1.86A7.94 7.94 0 0112 6.561v4.939a.75.75 0 101.5 0V6.561a7.94 7.94 0 012.07 1.407c.28.58.512 1.21.68 1.86.165.643.25 1.311.25 1.994V11.5a.75.75 0 001.5 0V9.762a19.38 19.38 0 01-1.7 4.217 31.545 31.545 0 014.101-.987.75.75 0 00.198-1.487 32.987 32.987 0 00-3.599-.278V2.75a.75.75 0 00-1.5 0v1.439a7.94 7.94 0 01-3.12 1.548A7.94 7.94 0 019.25 4.19V2.75zM8.5 13.25a.75.75 0 00-1.5 0v2a.75.75 0 001.5 0v-2z" /></svg>{{ languages[session.get('language', 'en')] or 'Language' }}</button><div x-show="open" @click.away="open = false" class="absolute right-0 mt-2 w-36 bg-white border border-gray-200 rounded-md shadow-lg z-50 overflow-hidden" style="display: none;">{% for lang_code, lang_name in languages.items() %}<a href='{{ url_for("set_language", lang=lang_code) }}' class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100 transition duration-100">{{ lang_name }}</a>{% endfor %}</div></div></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6 flex-grow'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6 border-t mt-auto w-full'>© 2025 AgroScan AI - {{ _('Model') }}: {{ ACTIVE_MODEL_KEY or 'N/A' }}</footer></body></html>"""
    , "home.html": """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-8 items-center bg-white p-8 rounded-xl shadow-lg'><div class='order-2 md:order-1'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900'>{{ _('Revolutionize Your Farming with AI') }}</h1><p class='mt-4 text-lg text-gray-600'>{{ _('Instantly detect crop diseases with a simple photo. Get actionable advice to protect your harvest and increase your yield.') }}</p><div class='mt-8 flex space-x-4'>{% if current_user.is_authenticated %}<a href='{{ url_for("predict_page") }}' class='bg-green-600 hover:bg-green-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200'>{{ _('Start Analysis') }}</a><a href='{{ url_for("dashboard") }}' class='border border-gray-300 hover:bg-gray-100 text-gray-700 px-6 py-3 rounded-lg font-bold transition duration-200'>{{ _('View Dashboard') }}</a>{% else %}<a href='{{ url_for("login") }}' class='bg-blue-600 hover:bg-blue-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200'>{{ _('Login to Start') }}</a>{% endif %}</div></div><div class='order-1 md:order-2'><img src='{{ url_for("static", filename="uploads/sample_leaf.png") }}' class='rounded-lg shadow-xl w-full' alt='Sample Leaf Image'></div></div>{% endblock %}"""
    , "login.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto bg-white p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Login') }}</h2><form method='post' action='{{ url_for("login") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200'>{{ _('Sign in') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Not a member?') }} <a href='{{ url_for("register") }}' class='text-green-600 hover:text-green-700'>{{ _('Register here') }}</a></p></div>{% endblock %}"""
    , "register.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto bg-white p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Register') }}</h2><form method='post' action='{{ url_for("register") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200'>{{ _('Register') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Already a member?') }} <a href='{{ url_for("login") }}' class='text-green-600 hover:text-green-700'>{{ _('Login here') }}</a></p></div>{% endblock %}"""
    , "predict.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto bg-white p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Predict Disease') }}</h2>{% if model_loading %}<div class='text-center p-8 bg-yellow-50 rounded-lg'><div class='animate-spin w-8 h-8 border-4 border-yellow-600 border-t-transparent rounded-full mx-auto'></div><p class='mt-4 text-lg text-yellow-800 font-semibold'>{{ _('Model is loading...') }}</p><p class='text-sm text-yellow-700'>{{ _('This may take up to 60 seconds. Please wait on this page.') }}</p></div>{% else %}<form id='upload-form' method='post' action='{{ url_for("predict_page") }}' enctype='multipart/form-data'><label for='file-upload' class='block border-2 border-dashed border-gray-300 p-10 rounded-xl cursor-pointer hover:border-green-500 transition duration-200'><input id='file-upload' name='file' type='file' accept='image/jpeg, image/png, image/jpg' class='hidden'><div class='text-center'><svg class='mx-auto w-12 h-12 text-gray-400' fill='none' stroke='currentColor' viewBox='0 0 24 24'><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M7 16a4 4 0 01-4-4V7a4 4 0 014-4h10a4 4 0 014 4v5a4 4 0 01-4 4h-3'></path><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M16 12l-4-4m0 0L8 12m4-4v11'></path></svg><p class='mt-2 text-gray-600 font-semibold'>{{ _('Click to upload leaf image') }}</p><p id='file-info' class='mt-1 text-sm text-gray-500'></p></div></label><button id='submit-button' class='w-full mt-6 bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200'>{{ _('Detect Disease') }}</button><div id='loader' class='hidden w-full flex items-center justify-center p-3 bg-gray-100 rounded-lg mt-6'><div class='animate-spin w-6 h-6 border-4 border-green-600 border-t-transparent rounded-full mr-3'></div><span>{{ _('Analyzing...') }}</span></div></form></div><script>const fileUpload=document.getElementById('file-upload'), form=document.getElementById('upload-form'), fileInfo=document.getElementById('file-info'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); fileUpload.addEventListener('change', e=>{ const f=e.target.files[0]; if(f) fileInfo.textContent = `{{ _('File') }}: ${f.name}`; }); form.addEventListener('submit', e=>{ if(!fileUpload.files.length){ e.preventDefault(); alert('{{ _('Please select an image to upload.') }}'); return; } submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endif %}{% endblock %}"""
    , "result.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto bg-white p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Detection Result') }}</h2><div class='grid md:grid-cols-2 gap-8 items-start'><div class='md:w-full'><img src='{{ image_path }}' class='rounded-lg w-full shadow-md' alt='Uploaded Leaf'></div><div class='md:w-full space-y-5'><div class='bg-gray-50 p-5 rounded-lg border-l-4 border-green-600'><p class='text-lg text-gray-600 mb-1'>{{ _('Predicted Condition') }}:</p><h3 class='text-2xl font-extrabold text-green-700'>{{ prediction }}</h3><p class='mt-2 text-gray-600'>{{ _('Confidence') }}: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-4 mt-3 overflow-hidden' title='{{ confidence }}%'>{% if confidence_level == 'High' %}<div class='h-4 bg-green-500' style='width: {{ confidence }}%;'></div>{% elif confidence_level == 'Medium' %}<div class='h-4 bg-yellow-500' style='width: {{ confidence }}%;'></div>{% else %}<div class='h-4 bg-red-500' style='width: {{ confidence }}%;'></div>{% endif %}</div></div><div class='bg-blue-50 p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('About this Condition') }}</h4><p class='text-gray-700'>{{ description }}</p></div>{% if remedies %}<div class='bg-green-50 p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('Suggested Actions') }}</h4><ul class='list-disc pl-5 text-gray-700'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}</div></div><div class='mt-8 text-center'><a href='{{ url_for("predict_page") }}' class='bg-gray-600 hover:bg-gray-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200'>{{ _('Analyze Another Image') }}</a></div></div>{% endblock %}"""
    , "dashboard.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Dashboard') }}</h2><div class='bg-white p-6 rounded-xl shadow-xl'><h3 class='text-xl font-bold mb-4'>{{ _('Model Configuration') }}</h3><div class='mt-4'><form method='post' action='{{ url_for("switch_model") }}'><label class='block text-gray-700'>{{ _('Select active model') }}:<select name='model' class='mt-2 p-2 border rounded-lg w-full md:w-1/3'> <option value='best' {% if active_model=='best' %}selected{% endif %}>{{ _('Best model') }} (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>{{ _('Fallback model') }} (crop_disease_model.h5)</option></select></label><div class='mt-4'><button class='bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold'>{{ _('Switch Model') }}</button></div></form></div><h3 class='text-xl font-bold mt-8 mb-4'>{{ _('Your Prediction Statistics') }}</h3>{% if disease_data.labels %}<ul class='space-y-2'>{% for i in range(disease_data.labels|length) %}<li class='flex justify-between p-2 border-b'><span class='font-semibold'>{{ disease_data.labels[i] }}</span><span>{{ disease_data.values[i] }} {{ _('times detected') }}</span></li>{% endfor %}</ul>{% else %}<p class='text-gray-500'>{{ _('No detections yet. Analyze to populate your dashboard.') }}</p>{% endif %}</div>{% endblock %}"""
    , "history.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Prediction History') }}</h2>{% if history %}<div class='grid md:grid-cols-3 gap-6'>{% for p in history %}<div class='bg-white rounded-xl shadow-md overflow-hidden hover:shadow-lg transition duration-200'><img src='{{ p.image_path }}' class='w-full h-40 object-cover' alt='Uploaded Leaf'><div class='p-4'><div class='text-sm text-gray-500'>{{ p.timestamp }}</div><div class='font-bold text-lg mt-1'>{{ p.prediction }}</div><div class='text-sm text-gray-600'>{{ _('Confidence') }}: {{ p.confidence | int }}%</div><div class='text-xs text-gray-500'>{{ _('Model') }}: {{ p.model_used }}</div></div></div>{% endfor %}</div>{% else %}<p class='text-gray-500'>{{ _('No saved predictions yet.') }}</p>{% endif %}{% endblock %}"""
    , "metrics.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Training Metrics') }}</h2><div class='bg-white p-6 rounded-xl shadow-xl'><p class='text-gray-600 mb-4'>{{ _('The files below are loaded from your Colab root directory.') }}</p>{% if training_results %}<h3 class='text-xl font-bold mt-4'>{{ _('Final Training Results') }}</h3><pre class='whitespace-pre-wrap bg-gray-100 p-4 rounded-lg text-sm'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}{% if confusion_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Confusion Matrix') }}</h3><img src='{{ url_for("static", filename="confusion_matrix.png") }}' class='max-w-full rounded-lg shadow-md mt-2' alt='Confusion Matrix'>{% endif %}{% if report_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Classification Report') }}</h3><pre class='bg-gray-100 p-4 rounded-lg text-sm'>{{ report_text }}</pre>{% endif %}{% if not training_results and not confusion_exists and not report_exists %}<p class='text-gray-500'>{{ _('No training metrics files found (training_results.json, confusion_matrix.png, classification_report.txt).') }}</p>{% endif %}</div>{% endblock %}"""
}

# Write HTML templates to files
for name, content in template_content.items():
    with open(os.path.join(TEMPLATES_DIR, name), "w", encoding="utf-8") as f:
        f.write(content)

# -------------------
# 9) Start Ngrok and Flask
# -------------------
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    # Start background model loading immediately
    threading.Thread(target=_background_preload, daemon=True).start()

    print("Colab Flask app (dual-model, Tailwind UI) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("\nPaste your ngrok authtoken below. DO NOT re-run if model is still loading.")

    # Setup Ngrok
    try:
        token = getpass("ngrok authtoken (hidden): ")
        if token and token.strip():
            ngrok.set_auth_token(token.strip())
            public_url = ngrok.connect(5000)
            print(f"🔗 ngrok tunnel: {public_url}")
        else:
            print("Ngrok token not provided. App will only be accessible via local Colab link.")
    except Exception as e:
        print(f"Ngrok setup failed: {e}")

    # Run Flask
    app.run(host="0.0.0.0", port=5000, debug=False)

Colab Flask app (dual-model, Tailwind UI) starting...[model] Loading 'best' from best_crop_model.h5 ...

Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)

Paste your ngrok authtoken below. DO NOT re-run if model is still loading.
[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ALL models preloaded and ready.
ngrok authtoken (hidden): ··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:14:24] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:14:25] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:14:30] "POST /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:14:35] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:14:40] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:14:41] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:14:50] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:14:51] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:15:00] "GET /metrics HTTP/1.1" 200 -

In [28]:
#best code till now both above and down

In [27]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINAL STABLE & FIXED SCRIPT
# =========================================================================
#  -- NEW FEATURE: Dashboard Charts (Prediction Distribution & Activity) --
# =========================================================================

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, time, threading, sqlite3, json, atexit
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

# -------------------
# 2) Config, File System, and Constants
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
IMAGE_SIZE = (128, 128)
ALLOWED_EXT = {"png","jpg","jpeg"}
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# Model paths and global state
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
MODEL_LOADING = True

# -------------------
# 3) Class names, Formatting, and Languages
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# -------------------
# 4) Database Setup (Updated for dual-model schema)
# -------------------
if os.path.exists(DB_PATH): os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     time TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Sample image generation for UI and Quick Analysis feature
SAMPLE_FILENAME = "sample_leaf_ai.png"
sample_img = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# -------------------
# 5) Disease Information
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.','remedies':[]},
    'Potato___Early_blight': {'description':'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies':['Use certified seed','Apply fungicides containing mancozeb or chlorothalonil','Rotate crops']},
    'Potato___Late_blight': {'description':'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside.', 'remedies':['Remove infected plants','Protective fungicides proactively','Ensure good drainage and air circulation']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description':'Causes small, water-soaked spots on tomato leaves and fruit.','remedies':['Avoid working when wet','Use copper-based bactericides','Mulch around plants']},
    'Tomato_Early_blight': {'description':'Fungal disease, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies':['Ensure proper spacing','Apply preventative fungicides','Water at the base of the plant']},
    'Tomato_Late_blight': {'description':'A serious fungal disease characterized by large, greasy, grey-green spots on leaves.','remedies':['Apply preventative fungicides like chlorothalonil or copper','Remove and destroy infected plants immediately']},
    'Tomato_Leaf_Mold': {'description':'Fungal disease causing pale spots on the upper leaf surface and a velvety, olive-green mold on the underside.','remedies':['Improve air circulation and reduce humidity','Prune lower leaves']},
    'Tomato_Septoria_leaf_spot': {'description':'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.','remedies':['Remove infected leaves','Improve air circulation','Apply fungicides']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description':'Tiny pests causing stippling (tiny yellow or white dots) on leaves, leading to webbing.', 'remedies':['Spray plants with water','Apply insecticidal soap or horticultural oil','Introduce natural predators']},
    'Tomato__Target_Spot': {'description':'A fungal disease causing necrotic spots with concentric rings.', 'remedies':['Apply fungicides effective against Corynespora cassiicola','Improve air circulation and reduce leaf wetness']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description':'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies':['Control whitefly populations','Remove and destroy infected plants immediately','Use virus-resistant varieties']},
    'Tomato__Tomato_mosaic_virus': {'description':'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation.', 'remedies':['There is no cure; remove and destroy infected plants','Wash hands and tools thoroughly']},
    'Tomato_healthy': {'description':'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.','remedies':[]},
    'default': {'description':'Information for this condition is not available.','remedies':['Consult local extension services']}
}

# -------------------
# 6) Model Management & Translation Setup
# -------------------
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# Route-safe translation helper
def tr_route(text):
    lang = session.get('language', 'en')
    return get_translation(text, lang)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_globals():
    def tr(text):
        return tr_route(text)
    return dict(_=tr, languages=LANGUAGES, ACTIVE_MODEL_KEY=ACTIVE_MODEL_KEY)

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = tf.keras.models.load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

def _background_preload():
    global MODEL_LOADING
    MODEL_LOADING = True
    for k in ["best", "fallback"]:
        load_model_lazy(k)
    MODEL_LOADING = False
    print("ALL models preloaded and ready.")

# -------------------
# 7) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

@app.route("/")
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template("home.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("dashboard"))
        flash(tr_route("Invalid username or password."))
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash(tr_route("Registration successful! Please login.")); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash(tr_route("Username already taken."))
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    global ACTIVE_MODEL_KEY
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()

    # Chart 1: Prediction Distribution (Pie Chart Data)
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    disease_rows = cur.fetchall()
    disease_data = {'labels':[r[0] for r in disease_rows], 'values':[r[1] for r in disease_rows]}

    # Chart 2: Activity Over Time (Bar Chart Data)
    # Fetch data grouped by month (YYYY-MM format)
    cur.execute("SELECT strftime('%Y-%m', time) as month, COUNT(*) as count FROM predictions WHERE user_id = ? GROUP BY month ORDER BY month", (current_user.id,))
    monthly_rows = cur.fetchall()
    monthly_data = {
        "labels": [datetime.strptime(row[0], '%Y-%m').strftime('%b %Y') for row in monthly_rows],
        "values": [row[1] for row in monthly_rows]
    }

    conn.close()
    return render_template("dashboard.html", disease_data=disease_data, monthly_data=monthly_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(tr_route(f"Active model switched to: {m}"))
    else:
        flash(tr_route("Unknown model key"))
    return redirect(url_for('dashboard'))

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time, model_used FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    ist_offset = timedelta(hours=5, minutes=30)
    for img_url,pred,conf,ts,model_used in rows:
        try: ts_dt = datetime.fromisoformat(ts)
        except: ts_dt = datetime.strptime(ts.split('.')[0], "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + ist_offset
        hist.append({
            "image_path":img_url,"prediction":pred,
            "confidence":int(round(conf)), "timestamp":ist.strftime("%b %d, %Y %I:%M %p"),
            "model_used": model_used
        })
    return render_template("history.html", history=hist)

@app.route("/metrics")
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global MODEL_LOADING, ACTIVE_MODEL_KEY
    model = MODEL_REGISTRY.get(ACTIVE_MODEL_KEY)

    last_uploaded_image_url = session.get('last_uploaded_image_url')

    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)
    if model is None:
        flash(tr_route(f"Model {ACTIVE_MODEL_KEY} failed to load. Please check files and re-run."))
        return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    if request.method == "POST":
        file_to_analyze = None
        current_image_url = None

        source_type = request.form.get('source', 'upload')

        if source_type == 'reuse' and last_uploaded_image_url:
            filename = os.path.basename(last_uploaded_image_url)
            file_to_analyze = os.path.join(UPLOAD_DIR, filename)
            current_image_url = last_uploaded_image_url

        elif source_type == 'sample':
            file_to_analyze = sample_img
            current_image_url = url_for('static', filename=f'uploads/{SAMPLE_FILENAME}')

        elif source_type == 'upload':
            if "file" not in request.files or request.files["file"].filename == "":
                flash(tr_route("No image uploaded."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            file = request.files["file"]
            if not allowed_file(file.filename):
                flash(tr_route("Invalid file type. Please upload JPG or PNG."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            fname = secure_filename(file.filename)
            unique_fname = f"{int(time.time())}_{fname}"
            file_to_analyze = os.path.join(UPLOAD_DIR, unique_fname)
            file.save(file_to_analyze)
            current_image_url = url_for('static', filename=f'uploads/{unique_fname}')

        else:
             flash(tr_route("Invalid analysis source."))
             return redirect(url_for('predict_page'))


        try:
            # === CRITICAL EFFICIENTNET PREPROCESSING ===
            img = Image.open(file_to_analyze).convert("RGB").resize(IMAGE_SIZE)
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            processed_arr = preprocess_input(arr)

            # Prediction
            preds = model.predict(processed_arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            raw_name = CLASS_NAMES[idx]
            pred_name = format_class_name(raw_name)

            # Update session with the current image path
            session['last_uploaded_image_url'] = current_image_url

            # Save to user history
            timestamp = datetime.utcnow().isoformat()
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time, model_used) VALUES (?,?,?,?,?,?)",
                        (current_user.id, current_image_url, pred_name, conf, timestamp, ACTIVE_MODEL_KEY))
            conn.commit(); conn.close()

            # Prepare result data (including translation)
            info = disease_info.get(raw_name, disease_info['default'])
            lang = session.get('language','en')
            desc = get_translation(info.get('description', ''), lang)
            remedies = [get_translation(r, lang) for r in info.get('remedies', [])]

            if conf >= 85: conf_level = 'High'
            elif conf >= 60: conf_level = 'Medium'
            else: conf_level = 'Low'

            return render_template("result.html", prediction=pred_name, confidence=round(conf,2),
                                    image_path=current_image_url, description=desc, remedies=remedies,
                                    confidence_level=conf_level)
        except Exception as e:
            flash(tr_route(f"Prediction error: {e}"))
            return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

# -------------------
# 8) Tailwind CSS Templates (GLASS MORPHISM & DROPDOWN FIX)
# -------------------
template_content = {
    "layout.html": """<!doctype html><html lang='{{ session.get("language", "en") }}'><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><script src="https://cdn.jsdelivr.net/gh/alpinejs/alpine@v2.x.x/dist/alpine.min.js" defer></script><title>{{ _(title or 'AgroScan AI') }}</title><style>
        body { min-height: 100vh; background-image: linear-gradient(to top right, #dcfce7, #ecfeff); }
        main { flex: 1; }
        .glass { background-color: rgba(255, 255, 255, 0.3); backdrop-filter: blur(10px); border: 1px solid rgba(255, 255, 255, 0.2); box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); border-radius: 1rem; }
        .glass-nav { background-color: rgba(255, 255, 255, 0.6); backdrop-filter: blur(8px); }
    </style></head><body class="text-gray-800 flex flex-col"><nav class="glass-nav fixed w-full z-30 shadow-md"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="currentColor"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 17.93c-2.82-.44-5.02-2.31-6.12-4.94L13 14h-2c-2.76 0-5 2.24-5 5H4c0-3.92 2.68-7.23 6.33-8.15C10.12 11.23 11.06 12 12 12s1.88-.77 2.67-1.15c.16-.08.31-.17.46-.26 3.65.92 6.33 4.23 6.33 8.15h-2c0-2.76-2.24-5-5-5h-2l6.12 4.94z"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="flex items-center space-x-4"><div class="hidden md:flex items-center space-x-4">{% if current_user.is_authenticated %}<span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span><a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600'>{{ _('Dashboard') }}</a><a href='{{ url_for("predict_page") }}' class='text-gray-600 hover:text-green-600'>{{ _('Analyze') }}</a><a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600'>{{ _('History') }}</a><a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600'>{{ _('Metrics') }}</a><a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold'>{{ _('Logout') }}</a>{% else %}<a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600'>{{ _('Login') }}</a><a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600'>{{ _('Register') }}</a>{% endif %}</div><div x-data="{ open: false }" class="relative"><button @click="open = !open" class="text-gray-600 hover:text-green-600 flex items-center p-2 rounded-lg hover:bg-gray-100 transition duration-150"><svg class="w-5 h-5 mr-1" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M7.75 2.75a.75.75 0 00-1.5 0v1.258a32.987 32.987 0 00-3.599.278.75.75 0 10.198 1.487A31.545 31.545 0 018.7 5.545 19.38 19.38 0 017 9.762V11.5a.75.75 0 001.5 0V9.762c0-.683.085-1.35.25-1.994.169-.65.4-1.28.68-1.86A7.94 7.94 0 0112 6.561v4.939a.75.75 0 101.5 0V6.561a7.94 7.94 0 012.07 1.407c.28.58.512 1.21.68 1.86.165.643.25 1.311.25 1.994V11.5a.75.75 0 001.5 0V9.762a19.38 19.38 0 01-1.7 4.217 31.545 31.545 0 014.101-.987.75.75 0 00.198-1.487 32.987 32.987 0 00-3.599-.278V2.75a.75.75 0 00-1.5 0v1.439a7.94 7.94 0 01-3.12 1.548A7.94 7.94 0 019.25 4.19V2.75zM8.5 13.25a.75.75 0 00-1.5 0v2a.75.75 0 001.5 0v-2z" /></svg>{{ languages[session.get('language', 'en')] or 'Language' }}</button><div x-show="open" @click.away="open = false" class="absolute right-0 mt-2 w-36 bg-white border border-gray-200 rounded-md shadow-lg z-50 overflow-hidden" style="display: none;">{% for lang_code, lang_name in languages.items() %}<a href='{{ url_for("set_language", lang=lang_code) }}' class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100 transition duration-100">{{ lang_name }}</a>{% endfor %}</div></div></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6 flex-grow'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6 border-t mt-auto w-full'>© 2025 AgroScan AI - {{ _('Model') }}: {{ ACTIVE_MODEL_KEY or 'N/A' }}</footer></body></html>"""
    , "home.html": """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-8 items-center glass p-10 rounded-xl'><div class='order-2 md:order-1'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900'>{{ _('Revolutionize Your Farming with AI') }}</h1><p class='mt-4 text-lg text-gray-700'>{{ _('Instantly detect crop diseases with a simple photo. Get actionable advice to protect your harvest and increase your yield.') }}</p><div class='mt-8 flex space-x-4'>{% if current_user.is_authenticated %}<a href='{{ url_for("predict_page") }}' class='bg-green-600 hover:bg-green-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Start Analysis') }}</a><a href='{{ url_for("dashboard") }}' class='bg-white hover:bg-gray-100 text-gray-700 px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('View Dashboard') }}</a>{% else %}<a href='{{ url_for("login") }}' class='bg-blue-600 hover:bg-blue-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Login to Start') }}</a>{% endif %}</div></div><div class='order-1 md:order-2'><img src='{{ url_for("static", filename="uploads/sample_leaf_ai.png") }}' class='rounded-lg shadow-xl w-full border border-gray-200' alt='Sample Leaf Image'></div></div>{% endblock %}"""
    , "login.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Login') }}</h2><form method='post' action='{{ url_for("login") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Sign in') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Not a member?') }} <a href='{{ url_for("register") }}' class='text-green-600 hover:text-green-700'>{{ _('Register here') }}</a></p></div>{% endblock %}"""
    , "register.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Register') }}</h2><form method='post' action='{{ url_for("register") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Register') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Already a member?') }} <a href='{{ url_for("login") }}' class='text-green-600 hover:text-green-700'>{{ _('Login here') }}</a></p></div>{% endblock %}"""
    , "predict.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Predict Disease') }}</h2>{% if model_loading %}<div class='text-center p-8 bg-yellow-50 rounded-lg'><div class='animate-spin w-8 h-8 border-4 border-yellow-600 border-t-transparent rounded-full mx-auto'></div><p class='mt-4 text-lg text-yellow-800 font-semibold'>{{ _('Model is loading...') }}</p><p class='text-sm text-yellow-700'>{{ _('This may take up to 60 seconds. Please wait on this page.') }}</p></div>{% else %}<form id='upload-form' method='post' action='{{ url_for("predict_page") }}' enctype='multipart/form-data'><input type="hidden" name="source" id="source-input" value="upload"><label for='file-upload' class='block border-2 border-dashed border-gray-300 p-10 rounded-xl cursor-pointer hover:border-green-500 transition duration-200'><input id='file-upload' name='file' type='file' accept='image/jpeg, image/png, image/jpg' class='hidden' onchange="document.getElementById('source-input').value = 'upload'; document.getElementById('file-info').textContent = 'File: ' + this.files[0].name;"><div class='text-center'><svg class='mx-auto w-12 h-12 text-gray-400' fill='none' stroke='currentColor' viewBox='0 0 24 24'><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M7 16a4 4 0 01-4-4V7a4 4 0 014-4h10a4 4 0 014 4v5a4 4 0 01-4 4h-3'></path><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M16 12l-4-4m0 0L8 12m4-4v11'></path></svg><p class='mt-2 text-gray-600 font-semibold'>{{ _('Click to upload leaf image') }}</p><p id='file-info' class='mt-1 text-sm text-gray-500'>{{ _('JPG or PNG recommended.') }}</p></div></label><div class='mt-6'><button id='submit-button' class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Detect Disease') }}</button><div id='loader' class='hidden w-full flex items-center justify-center p-3 bg-gray-100 rounded-lg mt-6'><div class='animate-spin w-6 h-6 border-4 border-green-600 border-t-transparent rounded-full mr-3'></div><span>{{ _('Analyzing...') }}</span></div></div></form><div class="mt-8 pt-6 border-t border-gray-200"><h3 class="text-xl font-bold mb-4">{{ _('Quick Analysis') }}</h3><div class="space-y-3">{% if last_uploaded_image_url %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="reuse"><button type="submit" class="w-full text-left bg-gray-100 hover:bg-gray-200 p-3 rounded-lg flex items-center transition duration-150"><svg class="w-5 h-5 mr-3 text-blue-600" fill="currentColor" viewBox="0 0 20 20"><path d="M10 12a2 2 0 100-4 2 2 0 000 4z"/><path fill-rule="evenodd" d="M.458 10C1.732 5.943 5.522 3 10 3s8.268 2.943 9.542 7c-1.274 4.057-5.064 7-9.542 7S1.732 14.057.458 10zM14 10a4 4 0 11-8 0 4 4 0 018 0z" clip-rule="evenodd"/></svg>{{ _('Reuse last uploaded image') }}</button></form>{% endif %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="sample"><button type="submit" class="w-full text-left bg-gray-100 hover:bg-gray-200 p-3 rounded-lg flex items-center transition duration-150"><svg class="w-5 h-5 mr-3 text-green-600" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M16.707 5.293a1 1 0 010 1.414l-8 8a1 1 0 01-1.414 0l-4-4a1 1 0 011.414-1.414L8 12.586l7.293-7.293a1 1 0 011.414 0z" clip-rule="evenodd"/></svg>{{ _('Use internal sample image') }}</button></form></div></div></div><script>const form=document.getElementById('upload-form'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); form.addEventListener('submit', e=>{ submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endif %}{% endblock %}"""
    , "result.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Detection Result') }}</h2><div class='grid md:grid-cols-2 gap-8 items-start'><div class='md:w-full'><img src='{{ image_path }}' class='rounded-lg w-full shadow-md' alt='Uploaded Leaf'></div><div class='md:w-full space-y-5'><div class='glass p-5 rounded-lg border-l-4 border-green-600'><p class='text-lg text-gray-700 mb-1'>{{ _('Predicted Condition') }}:</p><h3 class='text-2xl font-extrabold text-green-700'>{{ prediction }}</h3><p class='mt-2 text-gray-700'>{{ _('Confidence') }}: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-4 mt-3 overflow-hidden' title='{{ confidence }}%'>{% if confidence_level == 'High' %}<div class='h-4 bg-green-500' style='width: {{ confidence }}%;'></div>{% elif confidence_level == 'Medium' %}<div class='h-4 bg-yellow-500' style='width: {{ confidence }}%;'></div>{% else %}<div class='h-4 bg-red-500' style='width: {{ confidence }}%;'></div>{% endif %}</div></div><div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('About this Condition') }}</h4><p class='text-gray-700'>{{ description }}</p></div>{% if remedies %}<div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('Suggested Actions') }}</h4><ul class='list-disc pl-5 text-gray-700'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}</div></div><div class='mt-8 text-center'><a href='{{ url_for("predict_page") }}' class='bg-gray-600 hover:bg-gray-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Analyze Another Image') }}</a></div></div>{% endblock %}"""
    , "dashboard.html": """{% extends 'layout.html' %}{% block content %}<script src="https://cdn.jsdelivr.net/npm/chart.js@3.7.1/dist/chart.min.js"></script><h2 class='text-3xl font-bold mb-6'>{{ _('Dashboard') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><h3 class='text-xl font-bold mb-4'>{{ _('Model Configuration') }}</h3><div class='mt-4'><form method='post' action='{{ url_for("switch_model") }}'><label class='block text-gray-700'>{{ _('Select active model') }}:<select name='model' class='mt-2 p-2 border border-gray-300 rounded-lg w-full md:w-1/3 bg-white/70'> <option value='best' {% if active_model=='best' %}selected{% endif %}>{{ _('Best model') }} (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>{{ _('Fallback model') }} (crop_disease_model.h5)</option></select></label><div class='mt-4'><button class='bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold shadow-md'>{{ _('Switch Model') }}</button></div></form></div><h3 class='text-xl font-bold mt-8 mb-4'>{{ _('Your Prediction Statistics') }}</h3>{% if disease_data.labels %}<div class='grid grid-cols-1 lg:grid-cols-2 gap-8 mt-4'><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3'>{{ _('Detection Distribution') }}</h4><div class='h-64'><canvas id='diseaseChart'></canvas></div></div><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3'>{{ _('Activity Over Time') }}</h4><div class='h-64'><canvas id='monthlyChart'></canvas></div></div></div>{% else %}<p class='text-gray-500'>{{ _('No detections yet. Analyze to populate your dashboard.') }}</p>{% endif %}</div><script>const diseaseData={{ disease_data | tojson }}, monthlyData={{ monthly_data | tojson }}; if(diseaseData.labels.length>0){const ctx=document.getElementById('diseaseChart').getContext('2d'); new Chart(ctx, {type:'pie',data:{labels:diseaseData.labels,datasets:[{data:diseaseData.values,backgroundColor:['#10B981','#F59E0B','#EF4444','#6366F1','#06B6D4','#F472B6']}]},options:{responsive:true, maintainAspectRatio: false, plugins:{legend:{position:'right', labels: {color: '#374151'}}}}});} if(monthlyData.labels.length>0){const ctx=document.getElementById('monthlyChart').getContext('2d'); new Chart(ctx, {type:'bar',data:{labels:monthlyData.labels,datasets:[{label:'{{ _("Number of Scans") }}',data:monthlyData.values,backgroundColor:'#059669',borderColor:'#047857',borderWidth:1}]},options:{responsive:true, maintainAspectRatio: false, scales:{y:{beginAtZero:true, ticks: {stepSize: 1, color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}, x: {ticks: {color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}}}});}</script>{% endblock %}"""
    , "history.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Prediction History') }}</h2>{% if history %}<div class='grid md:grid-cols-3 gap-6'>{% for p in history %}<div class='glass p-0 rounded-xl shadow-md overflow-hidden hover:shadow-lg transition duration-200'><img src='{{ p.image_path }}' class='w-full h-40 object-cover' alt='Uploaded Leaf'><div class='p-4'><div class='text-sm text-gray-500'>{{ p.timestamp }}</div><div class='font-bold text-lg mt-1'>{{ p.prediction }}</div><div class='text-sm text-gray-600'>{{ _('Confidence') }}: {{ p.confidence | int }}%</div><div class='text-xs text-gray-500'>{{ _('Model') }}: {{ p.model_used }}</div></div></div>{% endfor %}</div>{% else %}<p class='text-gray-500'>{{ _('No saved predictions yet.') }}</p>{% endif %}{% endblock %}"""
    , "metrics.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Training Metrics') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><p class='text-gray-700 mb-4'>{{ _('The files below are loaded from your Colab root directory.') }}</p>{% if training_results %}<h3 class='text-xl font-bold mt-4'>{{ _('Final Training Results') }}</h3><pre class='whitespace-pre-wrap bg-white/70 p-4 rounded-lg text-sm border border-gray-200'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}{% if confusion_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Confusion Matrix') }}</h3><img src='{{ url_for("static", filename="confusion_matrix.png") }}' class='max-w-full rounded-lg shadow-md mt-2 border border-gray-200' alt='Confusion Matrix'>{% endif %}{% if report_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Classification Report') }}</h3><pre class='bg-white/70 p-4 rounded-lg text-sm border border-gray-200'>{{ report_text }}</pre>{% endif %}{% if not training_results and not confusion_exists and not report_exists %}<p class='text-gray-500'>{{ _('No training metrics files found (training_results.json, confusion_matrix.png, classification_report.txt).') }}</p>{% endif %}</div>{% endblock %}"""
}

# Write HTML templates to files
for name, content in template_content.items():
    with open(os.path.join(TEMPLATES_DIR, name), "w", encoding="utf-8") as f:
        f.write(content)

# -------------------
# 9) Start Ngrok and Flask
# -------------------
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    # Start background model loading immediately
    threading.Thread(target=_background_preload, daemon=True).start()

    print("Colab Flask app (dual-model, Tailwind UI) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("\nPaste your ngrok authtoken below. DO NOT re-run if model is still loading.")

    # Setup Ngrok
    try:
        token = getpass("ngrok authtoken (hidden): ")
        if token and token.strip():
            ngrok.set_auth_token(token.strip())
            public_url = ngrok.connect(5000)
            print(f"🔗 ngrok tunnel: {public_url}")
        else:
            print("Ngrok token not provided. App will only be accessible via local Colab link.")
    except Exception as e:
        print(f"Ngrok setup failed: {e}")

    # Run Flask
    app.run(host="0.0.0.0", port=5000, debug=False)

Colab Flask app (dual-model, Tailwind UI) starting...[model] Loading 'best' from best_crop_model.h5 ...

Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)

Paste your ngrok authtoken below. DO NOT re-run if model is still loading.
[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ALL models preloaded and ready.
ngrok authtoken (hidden): ··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:18:51] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:18:52] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:18:58] "POST /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:19:01] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:19:08] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:19:09] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:19:15] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:19:15] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:19:25] "GET /metrics HTTP/1.1" 200 -

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


/tmp/ipython-input-3816430900.py:372: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat()
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:20:12] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:20:13] "GET /static/uploads/1763677200_images_5.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:20:16] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:20:32] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:20:33] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:20:34] "GET /static/uploads/1763677200_images_5.jpeg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:20:37] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:20:38] "GET /static/confusion_matrix.

In [29]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINAL PRODUCTION SCRIPT
# =========================================================================
#  -- FEATURES: Glass UI, Dashboard Charts, Quick Analysis, Dual Models --
# =========================================================================

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, time, threading, sqlite3, json, atexit
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

# -------------------
# 2) Config, File System, and Constants
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
IMAGE_SIZE = (128, 128)
ALLOWED_EXT = {"png","jpg","jpeg"}
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# Model paths and global state
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
MODEL_LOADING = True
SAMPLE_FILENAME = "sample_leaf_ai.png"

# -------------------
# 3) Class names, Formatting, and Languages
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# -------------------
# 4) Database Setup
# -------------------
if os.path.exists(DB_PATH): os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     time TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Sample image generation for UI and Quick Analysis feature
sample_img = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# -------------------
# 5) Disease Information
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.','remedies':[]},
    'Potato___Early_blight': {'description':'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies':['Use certified seed','Apply fungicides containing mancozeb or chlorothalonil','Rotate crops']},
    'Potato___Late_blight': {'description':'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside.', 'remedies':['Remove infected plants','Protective fungicides proactively','Ensure good drainage and air circulation']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description':'Causes small, water-soaked spots on tomato leaves and fruit.','remedies':['Avoid working when wet','Use copper-based bactericides','Mulch around plants']},
    'Tomato_Early_blight': {'description':'Fungal disease, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies':['Ensure proper spacing','Apply preventative fungicides','Water at the base of the plant']},
    'Tomato_Late_blight': {'description':'A serious fungal disease characterized by large, greasy, grey-green spots on leaves.','remedies':['Apply preventative fungicides like chlorothalonil or copper','Remove and destroy infected plants immediately']},
    'Tomato_Leaf_Mold': {'description':'Fungal disease causing pale spots on the upper leaf surface and a velvety, olive-green mold on the underside.','remedies':['Improve air circulation and reduce humidity','Prune lower leaves']},
    'Tomato_Septoria_leaf_spot': {'description':'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.','remedies':['Remove infected leaves','Improve air circulation','Apply fungicides']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description':'Tiny pests causing stippling (tiny yellow or white dots) on leaves, leading to webbing.', 'remedies':['Spray plants with water','Apply insecticidal soap or horticultural oil','Introduce natural predators']},
    'Tomato__Target_Spot': {'description':'A fungal disease causing necrotic spots with concentric rings.', 'remedies':['Apply fungicides effective against Corynespora cassiicola','Improve air circulation and reduce leaf wetness']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description':'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies':['Control whitefly populations','Remove and destroy infected plants immediately','Use virus-resistant varieties']},
    'Tomato__Tomato_mosaic_virus': {'description':'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation.', 'remedies':['There is no cure; remove and destroy infected plants','Wash hands and tools thoroughly']},
    'Tomato_healthy': {'description':'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.','remedies':[]},
    'default': {'description':'Information for this condition is not available.','remedies':['Consult local extension services']}
}

# -------------------
# 6) Model Management & Translation Setup
# -------------------
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# Route-safe translation helper
def tr_route(text):
    lang = session.get('language', 'en')
    return get_translation(text, lang)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_globals():
    def tr(text):
        return tr_route(text)
    return dict(_=tr, languages=LANGUAGES, ACTIVE_MODEL_KEY=ACTIVE_MODEL_KEY)

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = tf.keras.models.load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

def _background_preload():
    global MODEL_LOADING
    MODEL_LOADING = True
    for k in ["best", "fallback"]:
        load_model_lazy(k)
    MODEL_LOADING = False
    print("ALL models preloaded and ready.")

# -------------------
# 7) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

@app.route("/")
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template("home.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("dashboard"))
        flash(tr_route("Invalid username or password."))
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash(tr_route("Registration successful! Please login.")); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash(tr_route("Username already taken."))
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    global ACTIVE_MODEL_KEY
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()

    # Chart 1: Prediction Distribution (Pie Chart Data)
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    disease_rows = cur.fetchall()
    disease_data = {'labels':[r[0] for r in disease_rows], 'values':[r[1] for r in disease_rows]}

    # Chart 2: Activity Over Time (Bar Chart Data)
    cur.execute("SELECT strftime('%Y-%m', time) as month, COUNT(*) as count FROM predictions WHERE user_id = ? GROUP BY month ORDER BY month", (current_user.id,))
    monthly_rows = cur.fetchall()
    monthly_data = {
        "labels": [datetime.strptime(row[0], '%Y-%m').strftime('%b %Y') for row in monthly_rows],
        "values": [row[1] for row in monthly_rows]
    }

    conn.close()
    return render_template("dashboard.html", disease_data=disease_data, monthly_data=monthly_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(tr_route(f"Active model switched to: {m}"))
    else:
        flash(tr_route("Unknown model key"))
    return redirect(url_for('dashboard'))

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time, model_used FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    ist_offset = timedelta(hours=5, minutes=30)
    for img_url,pred,conf,ts,model_used in rows:
        try: ts_dt = datetime.fromisoformat(ts)
        except: ts_dt = datetime.strptime(ts.split('.')[0], "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + ist_offset
        hist.append({
            "image_path":img_url,"prediction":pred,
            "confidence":int(round(conf)), "timestamp":ist.strftime("%b %d, %Y %I:%M %p"),
            "model_used": model_used
        })
    return render_template("history.html", history=hist)

@app.route("/metrics")
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global MODEL_LOADING, ACTIVE_MODEL_KEY
    model = MODEL_REGISTRY.get(ACTIVE_MODEL_KEY)

    last_uploaded_image_url = session.get('last_uploaded_image_url')

    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)
    if model is None:
        flash(tr_route(f"Model {ACTIVE_MODEL_KEY} failed to load. Please check files and re-run."))
        return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    if request.method == "POST":
        file_to_analyze = None
        current_image_url = None

        source_type = request.form.get('source', 'upload')

        if source_type == 'reuse' and last_uploaded_image_url:
            filename = os.path.basename(last_uploaded_image_url).split('?')[0] # Remove query params if any
            file_to_analyze = os.path.join(UPLOAD_DIR, filename)
            current_image_url = last_uploaded_image_url
            if not os.path.exists(file_to_analyze):
                flash(tr_route("Last uploaded image not found on server."))
                session.pop('last_uploaded_image_url', None) # Clear session if file is missing
                return redirect(url_for('predict_page'))

        elif source_type == 'sample':
            file_to_analyze = sample_img
            current_image_url = url_for('static', filename=f'uploads/{SAMPLE_FILENAME}')

        elif source_type == 'upload':
            if "file" not in request.files or request.files["file"].filename == "":
                flash(tr_route("No image uploaded."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            file = request.files["file"]
            if not allowed_file(file.filename):
                flash(tr_route("Invalid file type. Please upload JPG or PNG."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            fname = secure_filename(file.filename)
            unique_fname = f"{int(time.time())}_{fname}"
            file_to_analyze = os.path.join(UPLOAD_DIR, unique_fname)
            file.save(file_to_analyze)
            current_image_url = url_for('static', filename=f'uploads/{unique_fname}')

        else:
             flash(tr_route("Invalid analysis source."))
             return redirect(url_for('predict_page'))


        try:
            # === CRITICAL EFFICIENTNET PREPROCESSING ===
            img = Image.open(file_to_analyze).convert("RGB").resize(IMAGE_SIZE)
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            processed_arr = preprocess_input(arr)

            # Prediction
            preds = model.predict(processed_arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            raw_name = CLASS_NAMES[idx]
            pred_name = format_class_name(raw_name)

            # Update session with the current image path
            session['last_uploaded_image_url'] = current_image_url

            # Save to user history
            timestamp = datetime.utcnow().isoformat()
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time, model_used) VALUES (?,?,?,?,?,?)",
                        (current_user.id, current_image_url, pred_name, conf, timestamp, ACTIVE_MODEL_KEY))
            conn.commit(); conn.close()

            # Prepare result data (including translation)
            info = disease_info.get(raw_name, disease_info['default'])
            lang = session.get('language','en')
            desc = get_translation(info.get('description', ''), lang)
            remedies = [get_translation(r, lang) for r in info.get('remedies', [])]

            if conf >= 85: conf_level = 'High'
            elif conf >= 60: conf_level = 'Medium'
            else: conf_level = 'Low'

            return render_template("result.html", prediction=pred_name, confidence=round(conf,2),
                                    image_path=current_image_url, description=desc, remedies=remedies,
                                    confidence_level=conf_level)
        except Exception as e:
            flash(tr_route(f"Prediction error: {e}"))
            return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

# -------------------
# 8) Tailwind CSS Templates (GLASS MORPHISM & DROPDOWN/CHART FIXES)
# -------------------
template_content = {
    "layout.html": """<!doctype html><html lang='{{ session.get("language", "en") }}'><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><script src="https://cdn.jsdelivr.net/gh/alpinejs/alpine@v2.x.x/dist/alpine.min.js" defer></script><title>{{ _(title or 'AgroScan AI') }}</title><style>
        body { min-height: 100vh; background-image: linear-gradient(to top right, #dcfce7, #ecfeff); }
        main { flex: 1; }
        .glass { background-color: rgba(255, 255, 255, 0.3); backdrop-filter: blur(10px); border: 1px solid rgba(255, 255, 255, 0.2); box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); border-radius: 1rem; }
        .glass-nav { background-color: rgba(255, 255, 255, 0.6); backdrop-filter: blur(8px); }
    </style></head><body class="text-gray-800 flex flex-col"><nav class="glass-nav fixed w-full z-30 shadow-md"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="currentColor"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 17.93c-2.82-.44-5.02-2.31-6.12-4.94L13 14h-2c-2.76 0-5 2.24-5 5H4c0-3.92 2.68-7.23 6.33-8.15C10.12 11.23 11.06 12 12 12s1.88-.77 2.67-1.15c.16-.08.31-.17.46-.26 3.65.92 6.33 4.23 6.33 8.15h-2c0-2.76-2.24-5-5-5h-2l6.12 4.94z"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="flex items-center space-x-4"><div class="hidden md:flex items-center space-x-4">{% if current_user.is_authenticated %}<span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span><a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600'>{{ _('Dashboard') }}</a><a href='{{ url_for("predict_page") }}' class='text-gray-600 hover:text-green-600'>{{ _('Analyze') }}</a><a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600'>{{ _('History') }}</a><a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600'>{{ _('Metrics') }}</a><a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold'>{{ _('Logout') }}</a>{% else %}<a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600'>{{ _('Login') }}</a><a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600'>{{ _('Register') }}</a>{% endif %}</div><div x-data="{ open: false }" class="relative"><button @click="open = !open" class="text-gray-600 hover:text-green-600 flex items-center p-2 rounded-lg hover:bg-gray-100 transition duration-150"><svg class="w-5 h-5 mr-1" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M7.75 2.75a.75.75 0 00-1.5 0v1.258a32.987 32.987 0 00-3.599.278.75.75 0 10.198 1.487A31.545 31.545 0 018.7 5.545 19.38 19.38 0 017 9.762V11.5a.75.75 0 001.5 0V9.762c0-.683.085-1.35.25-1.994.169-.65.4-1.28.68-1.86A7.94 7.94 0 0112 6.561v4.939a.75.75 0 101.5 0V6.561a7.94 7.94 0 012.07 1.407c.28.58.512 1.21.68 1.86.165.643.25 1.311.25 1.994V11.5a.75.75 0 001.5 0V9.762a19.38 19.38 0 01-1.7 4.217 31.545 31.545 0 014.101-.987.75.75 0 00.198-1.487 32.987 32.987 0 00-3.599-.278V2.75a.75.75 0 00-1.5 0v1.439a7.94 7.94 0 01-3.12 1.548A7.94 7.94 0 019.25 4.19V2.75zM8.5 13.25a.75.75 0 00-1.5 0v2a.75.75 0 001.5 0v-2z" /></svg>{{ languages[session.get('language', 'en')] or 'Language' }}</button><div x-show="open" @click.away="open = false" class="absolute right-0 mt-2 w-36 bg-white border border-gray-200 rounded-md shadow-lg z-50 overflow-hidden" style="display: none;">{% for lang_code, lang_name in languages.items() %}<a href='{{ url_for("set_language", lang=lang_code) }}' class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100 transition duration-100">{{ lang_name }}</a>{% endfor %}</div></div></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6 flex-grow'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6 border-t mt-auto w-full'>© 2025 AgroScan AI - {{ _('Model') }}: {{ ACTIVE_MODEL_KEY or 'N/A' }}</footer></body></html>"""
    , "home.html": """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-8 items-center glass p-10 rounded-xl'><div class='order-2 md:order-1'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900'>{{ _('Revolutionize Your Farming with AI') }}</h1><p class='mt-4 text-lg text-gray-700'>{{ _('Instantly detect crop diseases with a simple photo. Get actionable advice to protect your harvest and increase your yield.') }}</p><div class='mt-8 flex space-x-4'>{% if current_user.is_authenticated %}<a href='{{ url_for("predict_page") }}' class='bg-green-600 hover:bg-green-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Start Analysis') }}</a><a href='{{ url_for("dashboard") }}' class='bg-white hover:bg-gray-100 text-gray-700 px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('View Dashboard') }}</a>{% else %}<a href='{{ url_for("login") }}' class='bg-blue-600 hover:bg-blue-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Login to Start') }}</a>{% endif %}</div></div><div class='order-1 md:order-2'><img src='{{ url_for("static", filename="uploads/sample_leaf_ai.png") }}' class='rounded-lg shadow-xl w-full border border-gray-200' alt='Sample Leaf Image'></div></div>{% endblock %}"""
    , "login.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Login') }}</h2><form method='post' action='{{ url_for("login") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Sign in') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Not a member?') }} <a href='{{ url_for("register") }}' class='text-green-600 hover:text-green-700'>{{ _('Register here') }}</a></p></div>{% endblock %}"""
    , "register.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Register') }}</h2><form method='post' action='{{ url_for("register") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Register') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Already a member?') }} <a href='{{ url_for("login") }}' class='text-green-600 hover:text-green-700'>{{ _('Login here') }}</a></p></div>{% endblock %}"""
    , "predict.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Predict Disease') }}</h2>{% if model_loading %}<div class='text-center p-8 bg-yellow-50 rounded-lg'><div class='animate-spin w-8 h-8 border-4 border-yellow-600 border-t-transparent rounded-full mx-auto'></div><p class='mt-4 text-lg text-yellow-800 font-semibold'>{{ _('Model is loading...') }}</p><p class='text-sm text-yellow-700'>{{ _('This may take up to 60 seconds. Please wait on this page.') }}</p></div>{% else %}<form id='upload-form' method='post' action='{{ url_for("predict_page") }}' enctype='multipart/form-data'><input type="hidden" name="source" id="source-input" value="upload"><label for='file-upload' class='block border-2 border-dashed border-gray-300 p-10 rounded-xl cursor-pointer hover:border-green-500 transition duration-200'><input id='file-upload' name='file' type='file' accept='image/jpeg, image/png, image/jpg' class='hidden' onchange="document.getElementById('source-input').value = 'upload'; document.getElementById('file-info').textContent = 'File: ' + this.files[0].name;"><div class='text-center'><svg class='mx-auto w-12 h-12 text-gray-400' fill='none' stroke='currentColor' viewBox='0 0 24 24'><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M7 16a4 4 0 01-4-4V7a4 4 0 014-4h10a4 4 0 014 4v5a4 4 0 01-4 4h-3'></path><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M16 12l-4-4m0 0L8 12m4-4v11'></path></svg><p class='mt-2 text-gray-600 font-semibold'>{{ _('Click to upload leaf image') }}</p><p id='file-info' class='mt-1 text-sm text-gray-500'>{{ _('JPG or PNG recommended.') }}</p></div></label><div class='mt-6'><button id='submit-button' class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Detect Disease') }}</button><div id='loader' class='hidden w-full flex items-center justify-center p-3 bg-gray-100 rounded-lg mt-6'><div class='animate-spin w-6 h-6 border-4 border-green-600 border-t-transparent rounded-full mr-3'></div><span>{{ _('Analyzing...') }}</span></div></div></form><div class="mt-8 pt-6 border-t border-gray-200"><h3 class="text-xl font-bold mb-4">{{ _('Quick Analysis') }}</h3><div class="space-y-3">{% if last_uploaded_image_url %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="reuse"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-150 border border-gray-200 shadow-sm"><svg class="w-5 h-5 mr-3 text-blue-600" fill="currentColor" viewBox="0 0 20 20"><path d="M10 12a2 2 0 100-4 2 2 0 000 4z"/><path fill-rule="evenodd" d="M.458 10C1.732 5.943 5.522 3 10 3s8.268 2.943 9.542 7c-1.274 4.057-5.064 7-9.542 7S1.732 14.057.458 10zM14 10a4 4 0 11-8 0 4 4 0 018 0z" clip-rule="evenodd"/></svg>{{ _('Reuse last uploaded image') }}</button></form>{% endif %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="sample"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-150 border border-gray-200 shadow-sm"><svg class="w-5 h-5 mr-3 text-green-600" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M16.707 5.293a1 1 0 010 1.414l-8 8a1 1 0 01-1.414 0l-4-4a1 1 0 011.414-1.414L8 12.586l7.293-7.293a1 1 0 011.414 0z" clip-rule="evenodd"/></svg>{{ _('Use internal sample image') }}</button></form></div></div></div><script>const form=document.getElementById('upload-form'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); form.addEventListener('submit', e=>{ submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endif %}{% endblock %}"""
    , "result.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Detection Result') }}</h2><div class='grid md:grid-cols-2 gap-8 items-start'><div class='md:w-full'><img src='{{ image_path }}' class='rounded-lg w-full shadow-md' alt='Uploaded Leaf'></div><div class='md:w-full space-y-5'><div class='glass p-5 rounded-lg border-l-4 border-green-600'><p class='text-lg text-gray-700 mb-1'>{{ _('Predicted Condition') }}:</p><h3 class='text-2xl font-extrabold text-green-700'>{{ prediction }}</h3><p class='mt-2 text-gray-700'>{{ _('Confidence') }}: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-4 mt-3 overflow-hidden' title='{{ confidence }}%'>{% if confidence_level == 'High' %}<div class='h-4 bg-green-500' style='width: {{ confidence }}%;'></div>{% elif confidence_level == 'Medium' %}<div class='h-4 bg-yellow-500' style='width: {{ confidence }}%;'></div>{% else %}<div class='h-4 bg-red-500' style='width: {{ confidence }}%;'></div>{% endif %}</div></div><div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('About this Condition') }}</h4><p class='text-gray-700'>{{ description }}</p></div>{% if remedies %}<div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('Suggested Actions') }}</h4><ul class='list-disc pl-5 text-gray-700'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}</div></div><div class='mt-8 text-center'><a href='{{ url_for("predict_page") }}' class='bg-gray-600 hover:bg-gray-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Analyze Another Image') }}</a></div></div>{% endblock %}"""
    , "dashboard.html": """{% extends 'layout.html' %}{% block content %}<script src="https://cdn.jsdelivr.net/npm/chart.js@3.7.1/dist/chart.min.js"></script><h2 class='text-3xl font-bold mb-6'>{{ _('Dashboard') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><h3 class='text-xl font-bold mb-4'>{{ _('Model Configuration') }}</h3><div class='mt-4'><form method='post' action='{{ url_for("switch_model") }}'><label class='block text-gray-700'>{{ _('Select active model') }}:<select name='model' class='mt-2 p-2 border border-gray-300 rounded-lg w-full md:w-1/3 bg-white/70'> <option value='best' {% if active_model=='best' %}selected{% endif %}>{{ _('Best model') }} (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>{{ _('Fallback model') }} (crop_disease_model.h5)</option></select></label><div class='mt-4'><button class='bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold shadow-md'>{{ _('Switch Model') }}</button></div></form></div><h3 class='text-xl font-bold mt-8 mb-4'>{{ _('Your Prediction Statistics') }}</h3>{% if disease_data.labels %}<div class='grid grid-cols-1 lg:grid-cols-2 gap-8 mt-4'><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3'>{{ _('Detection Distribution') }}</h4><div class='h-64'><canvas id='diseaseChart'></canvas></div></div><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3'>{{ _('Activity Over Time') }}</h4><div class='h-64'><canvas id='monthlyChart'></canvas></div></div></div>{% else %}<p class='text-gray-500'>{{ _('No detections yet. Analyze to populate your dashboard.') }}</p>{% endif %}</div><script>const diseaseData={{ disease_data | tojson }}, monthlyData={{ monthly_data | tojson }}; if(diseaseData.labels.length>0){const ctx=document.getElementById('diseaseChart').getContext('2d'); new Chart(ctx, {type:'pie',data:{labels:diseaseData.labels,datasets:[{data:diseaseData.values,backgroundColor:['#10B981','#F59E0B','#EF4444','#6366F1','#06B6D4','#F472B6']}]},options:{responsive:true, maintainAspectRatio: false, plugins:{legend:{position:'right', labels: {color: '#374151'}}}}});} if(monthlyData.labels.length>0){const ctx=document.getElementById('monthlyChart').getContext('2d'); new Chart(ctx, {type:'bar',data:{labels:monthlyData.labels,datasets:[{label:'{{ _("Number of Scans") }}',data:monthlyData.values,backgroundColor:'#059669',borderColor:'#047857',borderWidth:1}]},options:{responsive:true, maintainAspectRatio: false, scales:{y:{beginAtZero:true, ticks: {stepSize: 1, color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}, x: {ticks: {color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}}}});}</script>{% endblock %}"""
    , "history.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Prediction History') }}</h2>{% if history %}<div class='grid md:grid-cols-3 gap-6'>{% for p in history %}<div class='glass p-0 rounded-xl shadow-md overflow-hidden hover:shadow-lg transition duration-200'><img src='{{ p.image_path }}' class='w-full h-40 object-cover' alt='Uploaded Leaf'><div class='p-4'><div class='text-sm text-gray-500'>{{ p.timestamp }}</div><div class='font-bold text-lg mt-1'>{{ p.prediction }}</div><div class='text-sm text-gray-600'>{{ _('Confidence') }}: {{ p.confidence | int }}%</div><div class='text-xs text-gray-500'>{{ _('Model') }}: {{ p.model_used }}</div></div></div>{% endfor %}</div>{% else %}<p class='text-gray-500'>{{ _('No saved predictions yet.') }}</p>{% endif %}{% endblock %}"""
    , "metrics.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Training Metrics') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><p class='text-gray-700 mb-4'>{{ _('The files below are loaded from your Colab root directory.') }}</p>{% if training_results %}<h3 class='text-xl font-bold mt-4'>{{ _('Final Training Results') }}</h3><pre class='whitespace-pre-wrap bg-white/70 p-4 rounded-lg text-sm border border-gray-200'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}{% if confusion_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Confusion Matrix') }}</h3><img src='{{ url_for("static", filename="confusion_matrix.png") }}' class='max-w-full rounded-lg shadow-md mt-2 border border-gray-200' alt='Confusion Matrix'>{% endif %}{% if report_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Classification Report') }}</h3><pre class='bg-white/70 p-4 rounded-lg text-sm border border-gray-200'>{{ report_text }}</pre>{% endif %}{% if not training_results and not confusion_exists and not report_exists %}<p class='text-gray-500'>{{ _('No training metrics files found (training_results.json, confusion_matrix.png, classification_report.txt).') }}</p>{% endif %}</div>{% endblock %}"""
}

# Write HTML templates to files
for name, content in template_content.items():
    with open(os.path.join(TEMPLATES_DIR, name), "w", encoding="utf-8") as f:
        f.write(content)

# -------------------
# 9) Start Ngrok and Flask
# -------------------
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    # Start background model loading immediately
    threading.Thread(target=_background_preload, daemon=True).start()

    print("Colab Flask app (dual-model, Tailwind UI) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("\nPaste your ngrok authtoken below. DO NOT re-run if model is still loading.")

    # Setup Ngrok
    try:
        token = getpass("ngrok authtoken (hidden): ")
        if token and token.strip():
            ngrok.set_auth_token(token.strip())
            public_url = ngrok.connect(5000)
            print(f"🔗 ngrok tunnel: {public_url}")
        else:
            print("Ngrok token not provided. App will only be accessible via local Colab link.")
    except Exception as e:
        print(f"Ngrok setup failed: {e}")

    # Run Flask
    app.run(host="0.0.0.0", port=5000, debug=False)

Colab Flask app (dual-model, Tailwind UI) starting...[model] Loading 'best' from best_crop_model.h5 ...

Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)

Paste your ngrok authtoken below. DO NOT re-run if model is still loading.
[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ALL models preloaded and ready.
ngrok authtoken (hidden): ··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:04] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:04] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:10] "POST /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:12] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:18] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:19] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:25] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:26] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:34] "GET /predict HTTP/1.1" 200 -

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


/tmp/ipython-input-4102151390.py:375: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat()
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:51] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:52] "GET /static/uploads/1763677600_images_5.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:26:57] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:00] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:08] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:09] "GET /static/uploads/1763677628_download_6.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:16] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:19] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:26] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:27] "GET /static/uploads/1763677646_download_5.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:35] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:45] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:46] "GET /static/uploads/1763677628_download_6.jpeg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:46] "GET /static/uploads/1763677646_download_5.jpeg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:46] "GET /static/uploads/1763677600_images_5.jpeg HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:50] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:51] "GET /static/confusion_matrix.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:27:54] "GET /dashboard HTTP/1.1" 200 -


In [30]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINAL PRODUCTION SCRIPT
# =========================================================================
#  -- FEATURES: Glass UI, Team Page, New Logged-In Home Page, Charts --
# =========================================================================

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, time, threading, sqlite3, json, atexit
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

# -------------------
# 2) Config, File System, and Constants
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
IMAGE_SIZE = (128, 128)
ALLOWED_EXT = {"png","jpg","jpeg"}
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# Model paths and global state
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
MODEL_LOADING = True
SAMPLE_FILENAME = "sample_leaf_ai.png"

# -------------------
# 3) Class names, Formatting, and Languages
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# -------------------
# 4) Database Setup
# -------------------
if os.path.exists(DB_PATH): os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     time TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Sample image generation for UI and Quick Analysis feature
sample_img = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# -------------------
# 5) Disease Information
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.','remedies':[]},
    'Potato___Early_blight': {'description':'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies':['Use certified seed','Apply fungicides containing mancozeb or chlorothalonil','Rotate crops']},
    'Potato___Late_blight': {'description':'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside.', 'remedies':['Remove infected plants','Protective fungicides proactively','Ensure good drainage and air circulation']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description':'Causes small, water-soaked spots on tomato leaves and fruit.','remedies':['Avoid working when wet','Use copper-based bactericides','Mulch around plants']},
    'Tomato_Early_blight': {'description':'Fungal disease, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies':['Ensure proper spacing','Apply preventative fungicides','Water at the base of the plant']},
    'Tomato_Late_blight': {'description':'A serious fungal disease characterized by large, greasy, grey-green spots on leaves.','remedies':['Apply preventative fungicides like chlorothalonil or copper','Remove and destroy infected plants immediately']},
    'Tomato_Leaf_Mold': {'description':'Fungal disease causing pale spots on the upper leaf surface and a velvety, olive-green mold on the underside.','remedies':['Improve air circulation and reduce humidity','Prune lower leaves']},
    'Tomato_Septoria_leaf_spot': {'description':'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.','remedies':['Remove infected leaves','Improve air circulation','Apply fungicides']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description':'Tiny pests causing stippling (tiny yellow or white dots) on leaves, leading to webbing.', 'remedies':['Spray plants with water','Apply insecticidal soap or horticultural oil','Introduce natural predators']},
    'Tomato__Target_Spot': {'description':'A fungal disease causing necrotic spots with concentric rings.', 'remedies':['Apply fungicides effective against Corynespora cassiicola','Improve air circulation and reduce leaf wetness']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description':'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies':['Control whitefly populations','Remove and destroy infected plants immediately','Use virus-resistant varieties']},
    'Tomato__Tomato_mosaic_virus': {'description':'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation.', 'remedies':['There is no cure; remove and destroy infected plants','Wash hands and tools thoroughly']},
    'Tomato_healthy': {'description':'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.','remedies':[]},
    'default': {'description':'Information for this condition is not available.','remedies':['Consult local extension services']}
}

# -------------------
# 6) Model Management & Translation Setup
# -------------------
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# Route-safe translation helper
def tr_route(text):
    lang = session.get('language', 'en')
    return get_translation(text, lang)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_globals():
    def tr(text):
        return tr_route(text)
    return dict(_=tr, languages=LANGUAGES, ACTIVE_MODEL_KEY=ACTIVE_MODEL_KEY)

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = tf.keras.models.load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

def _background_preload():
    global MODEL_LOADING
    MODEL_LOADING = True
    for k in ["best", "fallback"]:
        load_model_lazy(k)
    MODEL_LOADING = False
    print("ALL models preloaded and ready.")

# -------------------
# 7) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

@app.route("/")
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    # Load the new, detailed home page after login
    return render_template("home_logged_in.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("home")) # Redirect to the new home page
        flash(tr_route("Invalid username or password."))
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash(tr_route("Registration successful! Please login.")); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash(tr_route("Username already taken."))
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    global ACTIVE_MODEL_KEY
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()

    # Chart 1: Prediction Distribution (Pie Chart Data)
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    disease_rows = cur.fetchall()
    disease_data = {'labels':[r[0] for r in disease_rows], 'values':[r[1] for r in disease_rows]}

    # Chart 2: Activity Over Time (Bar Chart Data)
    cur.execute("SELECT strftime('%Y-%m', time) as month, COUNT(*) as count FROM predictions WHERE user_id = ? GROUP BY month ORDER BY month", (current_user.id,))
    monthly_rows = cur.fetchall()
    monthly_data = {
        "labels": [datetime.strptime(row[0], '%Y-%m').strftime('%b %Y') for row in monthly_rows],
        "values": [row[1] for row in monthly_rows]
    }

    conn.close()
    return render_template("dashboard.html", disease_data=disease_data, monthly_data=monthly_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(tr_route(f"Active model switched to: {m}"))
    else:
        flash(tr_route("Unknown model key"))
    return redirect(url_for('dashboard'))

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time, model_used FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    ist_offset = timedelta(hours=5, minutes=30)
    for img_url,pred,conf,ts,model_used in rows:
        try: ts_dt = datetime.fromisoformat(ts)
        except: ts_dt = datetime.strptime(ts.split('.')[0], "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + ist_offset
        hist.append({
            "image_path":img_url,"prediction":pred,
            "confidence":int(round(conf)), "timestamp":ist.strftime("%b %d, %Y %I:%M %p"),
            "model_used": model_used
        })
    return render_template("history.html", history=hist)

@app.route("/metrics")
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

@app.route("/team")
def team():
    # Define the team data to be passed to the template
    team_members = [
        {"name": "Umar Iqbal", "role": "Project Leader & Lead Developer", "contributions": tr_route("Led the entire project lifecycle, developed the core prediction pipeline, implemented the Flask backend, and designed the database structure.")},
        {"name": "Shruti Bajpai", "role": "Research and Documentation Lead", "contributions": tr_route("Authored the research paper, handled documentation, and assisted with model evaluation metrics.")},
        {"name": "Sudhanshu Tiwari", "role": "Presentation and Support", "contributions": tr_route("Developed the project presentation slides, assisted in data preprocessing steps, and ensured overall project coherence.")},
        {"name": "Vaishnavi Singh", "role": "Presentation and Support", "contributions": tr_route("Contributed to the project presentation design, provided general technical support, and assisted with final testing and deployment.")}
    ]
    project_summary = tr_route("This project, AgroScan AI, uses a deep learning model (EfficientNetB0) trained on the PlantVillage dataset to instantly diagnose common crop diseases in potatoes, tomatoes, and bell peppers. The goal is to provide farmers with fast, actionable, and language-accessible intelligence to protect their yield.")

    return render_template('team.html', team_members=team_members, project_summary=project_summary, title='Our Team')

def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global MODEL_LOADING, ACTIVE_MODEL_KEY
    model = MODEL_REGISTRY.get(ACTIVE_MODEL_KEY)

    last_uploaded_image_url = session.get('last_uploaded_image_url')

    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)
    if model is None:
        flash(tr_route(f"Model {ACTIVE_MODEL_KEY} failed to load. Please check files and re-run."))
        return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    if request.method == "POST":
        file_to_analyze = None
        current_image_url = None

        source_type = request.form.get('source', 'upload')

        if source_type == 'reuse' and last_uploaded_image_url:
            filename = os.path.basename(last_uploaded_image_url).split('?')[0]
            file_to_analyze = os.path.join(UPLOAD_DIR, filename)
            current_image_url = last_uploaded_image_url
            if not os.path.exists(file_to_analyze):
                flash(tr_route("Last uploaded image not found on server."))
                session.pop('last_uploaded_image_url', None)
                return redirect(url_for('predict_page'))

        elif source_type == 'sample':
            file_to_analyze = sample_img
            current_image_url = url_for('static', filename=f'uploads/{SAMPLE_FILENAME}')

        elif source_type == 'upload':
            if "file" not in request.files or request.files["file"].filename == "":
                flash(tr_route("No image uploaded."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            file = request.files["file"]
            if not allowed_file(file.filename):
                flash(tr_route("Invalid file type. Please upload JPG or PNG."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            fname = secure_filename(file.filename)
            unique_fname = f"{int(time.time())}_{fname}"
            file_to_analyze = os.path.join(UPLOAD_DIR, unique_fname)
            file.save(file_to_analyze)
            current_image_url = url_for('static', filename=f'uploads/{unique_fname}')

        else:
             flash(tr_route("Invalid analysis source."))
             return redirect(url_for('predict_page'))


        try:
            # === CRITICAL EFFICIENTNET PREPROCESSING ===
            img = Image.open(file_to_analyze).convert("RGB").resize(IMAGE_SIZE)
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            processed_arr = preprocess_input(arr)

            # Prediction
            preds = model.predict(processed_arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            raw_name = CLASS_NAMES[idx]
            pred_name = format_class_name(raw_name)

            # Update session with the current image path
            session['last_uploaded_image_url'] = current_image_url

            # Save to user history
            timestamp = datetime.utcnow().isoformat()
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time, model_used) VALUES (?,?,?,?,?,?)",
                        (current_user.id, current_image_url, pred_name, conf, timestamp, ACTIVE_MODEL_KEY))
            conn.commit(); conn.close()

            # Prepare result data (including translation)
            info = disease_info.get(raw_name, disease_info['default'])
            lang = session.get('language','en')
            desc = get_translation(info.get('description', ''), lang)
            remedies = [get_translation(r, lang) for r in info.get('remedies', [])]

            if conf >= 85: conf_level = 'High'
            elif conf >= 60: conf_level = 'Medium'
            else: conf_level = 'Low'

            return render_template("result.html", prediction=pred_name, confidence=round(conf,2),
                                    image_path=current_image_url, description=desc, remedies=remedies,
                                    confidence_level=conf_level)
        except Exception as e:
            flash(tr_route(f"Prediction error: {e}"))
            return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

# -------------------
# 8) Tailwind CSS Templates (FIXED & NEW PAGES)
# -------------------
template_content = {
    "layout.html": """<!doctype html><html lang='{{ session.get("language", "en") }}'><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><script src="https://cdn.jsdelivr.net/gh/alpinejs/alpine@v2.x.x/dist/alpine.min.js" defer></script><title>{{ _(title or 'AgroScan AI') }}</title><style>
        body { min-height: 100vh; background-image: linear-gradient(to top right, #dcfce7, #ecfeff); }
        main { flex: 1; }
        .glass { background-color: rgba(255, 255, 255, 0.3); backdrop-filter: blur(10px); border: 1px solid rgba(255, 255, 255, 0.2); box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); border-radius: 1rem; }
        .glass-nav { background-color: rgba(255, 255, 255, 0.6); backdrop-filter: blur(8px); }
    </style></head><body class="text-gray-800 flex flex-col"><nav class="glass-nav fixed w-full z-30 shadow-md"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="currentColor"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 17.93c-2.82-.44-5.02-2.31-6.12-4.94L13 14h-2c-2.76 0-5 2.24-5 5H4c0-3.92 2.68-7.23 6.33-8.15C10.12 11.23 11.06 12 12 12s1.88-.77 2.67-1.15c.16-.08.31-.17.46-.26 3.65.92 6.33 4.23 6.33 8.15h-2c0-2.76-2.24-5-5-5h-2l6.12 4.94z"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="flex items-center space-x-4"><div class="hidden md:flex items-center space-x-4">{% if current_user.is_authenticated %}<span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span><a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600'>{{ _('Dashboard') }}</a><a href='{{ url_for("predict_page") }}' class='text-gray-600 hover:text-green-600'>{{ _('Analyze') }}</a><a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600'>{{ _('History') }}</a><a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600'>{{ _('Metrics') }}</a><a href='{{ url_for("team") }}' class='text-gray-600 hover:text-green-600'>{{ _('Team') }}</a><a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold'>{{ _('Logout') }}</a>{% else %}<a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600'>{{ _('Login') }}</a><a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600'>{{ _('Register') }}</a>{% endif %}</div><div x-data="{ open: false }" class="relative"><button @click="open = !open" class="text-gray-600 hover:text-green-600 flex items-center p-2 rounded-lg hover:bg-gray-100 transition duration-150"><svg class="w-5 h-5 mr-1" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M7.75 2.75a.75.75 0 00-1.5 0v1.258a32.987 32.987 0 00-3.599.278.75.75 0 10.198 1.487A31.545 31.545 0 018.7 5.545 19.38 19.38 0 017 9.762V11.5a.75.75 0 001.5 0V9.762c0-.683.085-1.35.25-1.994.169-.65.4-1.28.68-1.86A7.94 7.94 0 0112 6.561v4.939a.75.75 0 101.5 0V6.561a7.94 7.94 0 012.07 1.407c.28.58.512 1.21.68 1.86.165.643.25 1.311.25 1.994V11.5a.75.75 0 001.5 0V9.762a19.38 19.38 0 01-1.7 4.217 31.545 31.545 0 014.101-.987.75.75 0 00.198-1.487 32.987 32.987 0 00-3.599-.278V2.75a.75.75 0 00-1.5 0v1.439a7.94 7.94 0 01-3.12 1.548A7.94 7.94 0 019.25 4.19V2.75zM8.5 13.25a.75.75 0 00-1.5 0v2a.75.75 0 001.5 0v-2z" /></svg>{{ languages[session.get('language', 'en')] or 'Language' }}</button><div x-show="open" @click.away="open = false" class="absolute right-0 mt-2 w-36 bg-white border border-gray-200 rounded-md shadow-lg z-50 overflow-hidden" style="display: none;">{% for lang_code, lang_name in languages.items() %}<a href='{{ url_for("set_language", lang=lang_code) }}' class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100 transition duration-100">{{ lang_name }}</a>{% endfor %}</div></div></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6 flex-grow'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6 border-t mt-auto w-full'>© 2025 AgroScan AI - {{ _('Model') }}: {{ ACTIVE_MODEL_KEY or 'N/A' }}</footer></body></html>"""
    , "home.html": """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-8 items-center glass p-10 rounded-xl'><div class='order-2 md:order-1'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900'>{{ _('Revolutionize Your Farming with AI') }}</h1><p class='mt-4 text-lg text-gray-700'>{{ _('Instantly detect crop diseases with a simple photo. Get actionable advice to protect your harvest and increase your yield.') }}</p><div class='mt-8 flex space-x-4'><a href='{{ url_for("login") }}' class='bg-blue-600 hover:bg-blue-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Login to Start') }}</a><a href='{{ url_for("register") }}' class='bg-white hover:bg-gray-100 text-gray-700 px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Register') }}</a></div></div><div class='order-1 md:order-2'><img src='{{ url_for("static", filename="uploads/sample_leaf_ai.png") }}' class='rounded-lg shadow-xl w-full border border-gray-200' alt='Sample Leaf Image'></div></div>{% endblock %}"""
    , "home_logged_in.html": """{% extends 'layout.html' %}{% block content %}<div class='glass p-10 rounded-xl mb-8'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900 mb-2'>{{ _('Welcome,') }} {{ current_user.username }}!</h1><p class='text-lg text-gray-700'>{{ _('Your AI-powered crop management dashboard is ready.') }}</p></div><div class='grid lg:grid-cols-3 gap-6'><div class='lg:col-span-2 glass p-6 rounded-xl space-y-6'><h3 class='text-2xl font-bold text-gray-800 mb-4'>{{ _('What to do next?') }}</h3><div class='grid sm:grid-cols-3 gap-4'><a href='{{ url_for("predict_page") }}' class='bg-green-500 hover:bg-green-600 text-white p-4 rounded-lg font-bold text-center shadow-md transition duration-200'><div class='text-3xl mb-1'>🔍</div>{{ _('Start Analysis') }}</a><a href='{{ url_for("dashboard") }}' class='bg-blue-500 hover:bg-blue-600 text-white p-4 rounded-lg font-bold text-center shadow-md transition duration-200'><div class='text-3xl mb-1'>📊</div>{{ _('Check Dashboard') }}</a><a href='{{ url_for("history") }}' class='bg-gray-500 hover:bg-gray-600 text-white p-4 rounded-lg font-bold text-center shadow-md transition duration-200'><div class='text-3xl mb-1'>📜</div>{{ _('View History') }}</a></div><h3 class='text-2xl font-bold text-gray-800 mt-6 mb-3'>{{ _('Quick Guide') }}</h3><div class='space-y-4'><div class='p-4 bg-white/70 rounded-lg border border-green-200'><h4 class='font-bold'>1. {{ _('Analyze') }} ({{ url_for("predict_page") }})</h4><p class='text-sm text-gray-700'>{{ _('Upload a clear image of a symptomatic plant leaf. Our AI will return a diagnosis and suggested remedies.') }}</p></div><div class='p-4 bg-white/70 rounded-lg border border-blue-200'><h4 class='font-bold'>2. {{ _('Dashboard') }} ({{ url_for("dashboard") }})</h4><p class='text-sm text-gray-700'>{{ _('Review your past activity and see the distribution of diseases you have detected over time.') }}</p></div><div class='p-4 bg-white/70 rounded-lg border border-gray-200'><h4 class='font-bold'>3. {{ _('Metrics') }} ({{ url_for("metrics") }})</h4><p class='text-sm text-gray-700'>{{ _('View the scientific performance data (Confusion Matrix, Classification Report) of the core AI model.') }}</p></div></div></div><div class='lg:col-span-1 glass p-6 rounded-xl'><h3 class='text-2xl font-bold text-gray-800 mb-4'>{{ _('System Status') }}</h3><div class='space-y-3'><div class='p-3 bg-white/70 rounded-lg border border-green-200'><span class='font-semibold'>{{ _('Active Model') }}:</span> {{ ACTIVE_MODEL_KEY }}</div><div class='p-3 bg-white/70 rounded-lg border border-blue-200'><span class='font-semibold'>{{ _('Input Size') }}:</span> 128x128 pixels</div><div class='p-3 bg-white/70 rounded-lg border border-gray-200'><span class='font-semibold'>{{ _('Language') }}:</span> {{ languages[session.get('language', 'en')] or 'English' }}</div><a href='{{ url_for("predict_page") }}' class='w-full block text-center bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold shadow-md mt-6'>{{ _('Jump to Analysis') }}</a></div></div></div>{% endblock %}"""
    , "login.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Login') }}</h2><form method='post' action='{{ url_for("login") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Sign in') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Not a member?') }} <a href='{{ url_for("register") }}' class='text-green-600 hover:text-green-700'>{{ _('Register here') }}</a></p></div>{% endblock %}"""
    , "register.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Register') }}</h2><form method='post' action='{{ url_for("register") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Register') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Already a member?') }} <a href='{{ url_for("login") }}' class='text-green-600 hover:text-green-700'>{{ _('Login here') }}</a></p></div>{% endblock %}"""
    , "predict.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Predict Disease') }}</h2>{% if model_loading %}<div class='text-center p-8 bg-yellow-50 rounded-lg'><div class='animate-spin w-8 h-8 border-4 border-yellow-600 border-t-transparent rounded-full mx-auto'></div><p class='mt-4 text-lg text-yellow-800 font-semibold'>{{ _('Model is loading...') }}</p><p class='text-sm text-yellow-700'>{{ _('This may take up to 60 seconds. Please wait on this page.') }}</p></div>{% else %}<form id='upload-form' method='post' action='{{ url_for("predict_page") }}' enctype='multipart/form-data'><input type="hidden" name="source" id="source-input" value="upload"><label for='file-upload' class='block border-2 border-dashed border-gray-300 p-10 rounded-xl cursor-pointer hover:border-green-500 transition duration-200'><input id='file-upload' name='file' type='file' accept='image/jpeg, image/png, image/jpg' class='hidden' onchange="document.getElementById('source-input').value = 'upload'; document.getElementById('file-info').textContent = 'File: ' + this.files[0].name;"><div class='text-center'><svg class='mx-auto w-12 h-12 text-gray-400' fill='none' stroke='currentColor' viewBox='0 0 24 24'><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M7 16a4 4 0 01-4-4V7a4 4 0 014-4h10a4 4 0 014 4v5a4 4 0 01-4 4h-3'></path><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M16 12l-4-4m0 0L8 12m4-4v11'></path></svg><p class='mt-2 text-gray-600 font-semibold'>{{ _('Click to upload leaf image') }}</p><p id='file-info' class='mt-1 text-sm text-gray-500'>{{ _('JPG or PNG recommended.') }}</p></div></label><div class='mt-6'><button id='submit-button' class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Detect Disease') }}</button><div id='loader' class='hidden w-full flex items-center justify-center p-3 bg-gray-100 rounded-lg mt-6'><div class='animate-spin w-6 h-6 border-4 border-green-600 border-t-transparent rounded-full mr-3'></div><span>{{ _('Analyzing...') }}</span></div></div></form><div class="mt-8 pt-6 border-t border-gray-200"><h3 class="text-xl font-bold mb-4">{{ _('Quick Analysis') }}</h3><div class="space-y-3">{% if last_uploaded_image_url %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="reuse"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-150 border border-gray-200 shadow-sm"><svg class="w-5 h-5 mr-3 text-blue-600" fill="currentColor" viewBox="0 0 20 20"><path d="M10 12a2 2 0 100-4 2 2 0 000 4z"/><path fill-rule="evenodd" d="M.458 10C1.732 5.943 5.522 3 10 3s8.268 2.943 9.542 7c-1.274 4.057-5.064 7-9.542 7S1.732 14.057.458 10zM14 10a4 4 0 11-8 0 4 4 0 018 0z" clip-rule="evenodd"/></svg>{{ _('Reuse last uploaded image') }}</button></form>{% endif %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="sample"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-150 border border-gray-200 shadow-sm"><svg class="w-5 h-5 mr-3 text-green-600" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M16.707 5.293a1 1 0 010 1.414l-8 8a1 1 0 01-1.414 0l-4-4a1 1 0 011.414-1.414L8 12.586l7.293-7.293a1 1 0 011.414 0z" clip-rule="evenodd"/></svg>{{ _('Use internal sample image') }}</button></form></div></div></div><script>const form=document.getElementById('upload-form'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); form.addEventListener('submit', e=>{ submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endif %}{% endblock %}"""
    , "result.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Detection Result') }}</h2><div class='grid md:grid-cols-2 gap-8 items-start'><div class='md:w-full'><img src='{{ image_path }}' class='rounded-lg w-full shadow-md' alt='Uploaded Leaf'></div><div class='md:w-full space-y-5'><div class='glass p-5 rounded-lg border-l-4 border-green-600'><p class='text-lg text-gray-700 mb-1'>{{ _('Predicted Condition') }}:</p><h3 class='text-2xl font-extrabold text-green-700'>{{ prediction }}</h3><p class='mt-2 text-gray-700'>{{ _('Confidence') }}: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-4 mt-3 overflow-hidden' title='{{ confidence }}%'>{% if confidence_level == 'High' %}<div class='h-4 bg-green-500' style='width: {{ confidence }}%;'></div>{% elif confidence_level == 'Medium' %}<div class='h-4 bg-yellow-500' style='width: {{ confidence }}%;'></div>{% else %}<div class='h-4 bg-red-500' style='width: {{ confidence }}%;'></div>{% endif %}</div></div><div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('About this Condition') }}</h4><p class='text-gray-700'>{{ description }}</p></div>{% if remedies %}<div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('Suggested Actions') }}</h4><ul class='list-disc pl-5 text-gray-700'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}</div></div><div class='mt-8 text-center'><a href='{{ url_for("predict_page") }}' class='bg-gray-600 hover:bg-gray-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-md'>{{ _('Analyze Another Image') }}</a></div></div>{% endblock %}"""
    , "dashboard.html": """{% extends 'layout.html' %}{% block content %}<script src="https://cdn.jsdelivr.net/npm/chart.js@3.7.1/dist/chart.min.js"></script><h2 class='text-3xl font-bold mb-6'>{{ _('Dashboard') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><h3 class='text-xl font-bold mb-4'>{{ _('Model Configuration') }}</h3><div class='mt-4'><form method='post' action='{{ url_for("switch_model") }}'><label class='block text-gray-700'>{{ _('Select active model') }}:<select name='model' class='mt-2 p-2 border border-gray-300 rounded-lg w-full md:w-1/3 bg-white/70'> <option value='best' {% if active_model=='best' %}selected{% endif %}>{{ _('Best model') }} (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>{{ _('Fallback model') }} (crop_disease_model.h5)</option></select></label><div class='mt-4'><button class='bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold shadow-md'>{{ _('Switch Model') }}</button></div></form></div><h3 class='text-xl font-bold mt-8 mb-4'>{{ _('Your Prediction Statistics') }}</h3>{% if disease_data.labels %}<div class='grid grid-cols-1 lg:grid-cols-2 gap-8 mt-4'><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3'>{{ _('Detection Distribution') }}</h4><div class='h-64'><canvas id='diseaseChart'></canvas></div></div><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3'>{{ _('Activity Over Time') }}</h4><div class='h-64'><canvas id='monthlyChart'></canvas></div></div></div>{% else %}<p class='text-gray-500'>{{ _('No detections yet. Analyze to populate your dashboard.') }}</p>{% endif %}</div><script>const diseaseData={{ disease_data | tojson }}, monthlyData={{ monthly_data | tojson }}; if(diseaseData.labels.length>0){const ctx=document.getElementById('diseaseChart').getContext('2d'); new Chart(ctx, {type:'pie',data:{labels:diseaseData.labels,datasets:[{data:diseaseData.values,backgroundColor:['#10B981','#F59E0B','#EF4444','#6366F1','#06B6D4','#F472B6']}]},options:{responsive:true, maintainAspectRatio: false, plugins:{legend:{position:'right', labels: {color: '#374151'}}}}});} if(monthlyData.labels.length>0){const ctx=document.getElementById('monthlyChart').getContext('2d'); new Chart(ctx, {type:'bar',data:{labels:monthlyData.labels,datasets:[{label:'{{ _("Number of Scans") }}',data:monthlyData.values,backgroundColor:'#059669',borderColor:'#047857',borderWidth:1}]},options:{responsive:true, maintainAspectRatio: false, scales:{y:{beginAtZero:true, ticks: {stepSize: 1, color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}, x: {ticks: {color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}}}});}</script>{% endblock %}"""
    , "history.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Prediction History') }}</h2>{% if history %}<div class='grid md:grid-cols-3 gap-6'>{% for p in history %}<div class='glass p-0 rounded-xl shadow-md overflow-hidden hover:shadow-lg transition duration-200'><img src='{{ p.image_path }}' class='w-full h-40 object-cover' alt='Uploaded Leaf'><div class='p-4'><div class='text-sm text-gray-500'>{{ p.timestamp }}</div><div class='font-bold text-lg mt-1'>{{ p.prediction }}</div><div class='text-sm text-gray-600'>{{ _('Confidence') }}: {{ p.confidence | int }}%</div><div class='text-xs text-gray-500'>{{ _('Model') }}: {{ p.model_used }}</div></div></div>{% endfor %}</div>{% else %}<p class='text-gray-500'>{{ _('No saved predictions yet.') }}</p>{% endif %}{% endblock %}"""
    , "metrics.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Training Metrics') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><p class='text-gray-700 mb-4'>{{ _('The files below are loaded from your Colab root directory.') }}</p>{% if training_results %}<h3 class='text-xl font-bold mt-4'>{{ _('Final Training Results') }}</h3><pre class='whitespace-pre-wrap bg-white/70 p-4 rounded-lg text-sm border border-gray-200'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}{% if confusion_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Confusion Matrix') }}</h3><img src='{{ url_for("static", filename="confusion_matrix.png") }}' class='max-w-full rounded-lg shadow-md mt-2 border border-gray-200' alt='Confusion Matrix'>{% endif %}{% if report_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Classification Report') }}</h3><pre class='bg-white/70 p-4 rounded-lg text-sm border border-gray-200'>{{ report_text }}</pre>{% endif %}{% if not training_results and not confusion_exists and not report_exists %}<p class='text-gray-500'>{{ _('No training metrics files found (training_results.json, confusion_matrix.png, classification_report.txt).') }}</p>{% endif %}</div>{% endblock %}"""
    , "team.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-8 text-center'>{{ _('Meet the Team') }}</h2><div class='glass p-8 rounded-xl shadow-xl'><h3 class='text-xl font-semibold mb-4'>{{ _('Project Overview') }}</h3><p class='text-gray-700 mb-8'>{{ project_summary }}</p><h3 class='text-xl font-semibold mb-6'>{{ _('The Contributors') }}</h3><div class='space-y-6'>{% for member in team_members %}<div class='p-4 bg-white/70 rounded-lg border border-gray-200 shadow-sm'><div class='flex items-center space-x-4'><div class='text-3xl font-extrabold text-green-600'>{{ member.name[0] }}</div><div><h4 class='text-lg font-bold text-gray-800'>{{ member.name }} <span class='text-sm text-gray-500'>({{ member.role }})</span></h4><p class='text-sm text-gray-700 mt-1'>{{ _('Contribution') }}: {{ member.contributions }}</p></div></div></div>{% endfor %}</div></div>{% endblock %}"""
}

# Write HTML templates to files
for name, content in template_content.items():
    with open(os.path.join(TEMPLATES_DIR, name), "w", encoding="utf-8") as f:
        f.write(content)

# -------------------
# 9) Start Ngrok and Flask
# -------------------
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    # Start background model loading immediately
    threading.Thread(target=_background_preload, daemon=True).start()

    print("Colab Flask app (dual-model, Tailwind UI) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("\nPaste your ngrok authtoken below. DO NOT re-run if model is still loading.")

    # Setup Ngrok
    try:
        token = getpass("ngrok authtoken (hidden): ")
        if token and token.strip():
            ngrok.set_auth_token(token.strip())
            public_url = ngrok.connect(5000)
            print(f"🔗 ngrok tunnel: {public_url}")
        else:
            print("Ngrok token not provided. App will only be accessible via local Colab link.")
    except Exception as e:
        print(f"Ngrok setup failed: {e}")

    # Run Flask
    app.run(host="0.0.0.0", port=5000, debug=False)

[model] Loading 'best' from best_crop_model.h5 ...
Colab Flask app (dual-model, Tailwind UI) starting...
Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)

Paste your ngrok authtoken below. DO NOT re-run if model is still loading.
[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ALL models preloaded and ready.
ngrok authtoken (hidden): ··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:35:49] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:35:50] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:35:54] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:36:00] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:36:00] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:36:08] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:36:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:36:23] "GET /team HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:37:03] "GET /team HTTP/1.1" 200 -
INFO:werkzeug

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step


/tmp/ipython-input-2863738654.py:389: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().isoformat()
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:37:29] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:37:30] "GET /static/uploads/1763678238_images_4.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:37:34] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:37:39] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:37:50] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:38:13] "GET /history HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:38:13] "GET /static/uploads/1763678238_images_4.jpeg HTTP/1.1" 304 -


In [31]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINAL PRODUCTION SCRIPT
# =========================================================================
#  -- NEW UI: Modern, Animated, Color-Enhanced Glass UI --
# =========================================================================

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, time, threading, sqlite3, json, atexit
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

# -------------------
# 2) Config, File System, and Constants
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
IMAGE_SIZE = (128, 128)
ALLOWED_EXT = {"png","jpg","jpeg"}
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# Model paths and global state
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
MODEL_LOADING = True
SAMPLE_FILENAME = "sample_leaf_ai.png"

# -------------------
# 3) Class names, Formatting, and Languages
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# -------------------
# 4) Database Setup
# -------------------
if os.path.exists(DB_PATH): os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     time TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Sample image generation for UI and Quick Analysis feature
sample_img = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# -------------------
# 5) Disease Information
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.','remedies':[]},
    'Potato___Early_blight': {'description':'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies':['Use certified seed','Apply fungicides containing mancozeb or chlorothalonil','Rotate crops']},
    'Potato___Late_blight': {'description':'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside.', 'remedies':['Remove infected plants','Protective fungicides proactively','Ensure good drainage and air circulation']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description':'Causes small, water-soaked spots on tomato leaves and fruit.','remedies':['Avoid working when wet','Use copper-based bactericides','Mulch around plants']},
    'Tomato_Early_blight': {'description':'Fungal disease, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies':['Ensure proper spacing','Apply preventative fungicides','Water at the base of the plant']},
    'Tomato_Late_blight': {'description':'A serious fungal disease characterized by large, greasy, grey-green spots on leaves.','remedies':['Apply preventative fungicides like chlorothalonil or copper','Remove and destroy infected plants immediately']},
    'Tomato_Leaf_Mold': {'description':'Fungal disease causing pale spots on the upper leaf surface and a velvety, olive-green mold on the underside.','remedies':['Improve air circulation and reduce humidity','Prune lower leaves']},
    'Tomato_Septoria_leaf_spot': {'description':'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.','remedies':['Remove infected leaves','Improve air circulation','Apply fungicides']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description':'Tiny pests causing stippling (tiny yellow or white dots) on leaves, leading to webbing.', 'remedies':['Spray plants with water','Apply insecticidal soap or horticultural oil','Introduce natural predators']},
    'Tomato__Target_Spot': {'description':'A fungal disease causing necrotic spots with concentric rings.', 'remedies':['Apply fungicides effective against Corynespora cassiicola','Improve air circulation and reduce leaf wetness']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description':'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies':['Control whitefly populations','Remove and destroy infected plants immediately','Use virus-resistant varieties']},
    'Tomato__Tomato_mosaic_virus': {'description':'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation.', 'remedies':['There is no cure; remove and destroy infected plants','Wash hands and tools thoroughly']},
    'Tomato_healthy': {'description':'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.','remedies':[]},
    'default': {'description':'Information for this condition is not available.','remedies':['Consult local extension services']}
}

# -------------------
# 6) Model Management & Translation Setup
# -------------------
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# Route-safe translation helper
def tr_route(text):
    lang = session.get('language', 'en')
    return get_translation(text, lang)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_globals():
    def tr(text):
        return tr_route(text)
    return dict(_=tr, languages=LANGUAGES, ACTIVE_MODEL_KEY=ACTIVE_MODEL_KEY)

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = tf.keras.models.load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

def _background_preload():
    global MODEL_LOADING
    MODEL_LOADING = True
    for k in ["best", "fallback"]:
        load_model_lazy(k)
    MODEL_LOADING = False
    print("ALL models preloaded and ready.")

# -------------------
# 7) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

@app.route("/")
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    # Load the new, detailed home page after login
    return render_template("home_logged_in.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("home")) # Redirect to the new home page
        flash(tr_route("Invalid username or password."))
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash(tr_route("Registration successful! Please login.")); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash(tr_route("Username already taken."))
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    global ACTIVE_MODEL_KEY
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()

    # Chart 1: Prediction Distribution (Pie Chart Data)
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    disease_rows = cur.fetchall()
    disease_data = {'labels':[r[0] for r in disease_rows], 'values':[r[1] for r in disease_rows]}

    # Chart 2: Activity Over Time (Bar Chart Data)
    cur.execute("SELECT strftime('%Y-%m', time) as month, COUNT(*) as count FROM predictions WHERE user_id = ? GROUP BY month ORDER BY month", (current_user.id,))
    monthly_rows = cur.fetchall()
    monthly_data = {
        "labels": [datetime.strptime(row[0], '%Y-%m').strftime('%b %Y') for row in monthly_rows],
        "values": [row[1] for row in monthly_rows]
    }

    conn.close()
    return render_template("dashboard.html", disease_data=disease_data, monthly_data=monthly_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(tr_route(f"Active model switched to: {m}"))
    else:
        flash(tr_route("Unknown model key"))
    return redirect(url_for('dashboard'))

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time, model_used FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    ist_offset = timedelta(hours=5, minutes=30)
    for img_url,pred,conf,ts,model_used in rows:
        try: ts_dt = datetime.fromisoformat(ts)
        except: ts_dt = datetime.strptime(ts.split('.')[0], "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + ist_offset
        hist.append({
            "image_path":img_url,"prediction":pred,
            "confidence":int(round(conf)), "timestamp":ist.strftime("%b %d, %Y %I:%M %p"),
            "model_used": model_used
        })
    return render_template("history.html", history=hist)

@app.route("/metrics")
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

@app.route("/team")
def team():
    # Define the team data to be passed to the template
    team_members = [
        {"name": "Umar Iqbal", "role": "Project Leader & Lead Developer", "contributions": tr_route("Led the entire project lifecycle, developed the core prediction pipeline, implemented the Flask backend, and designed the database structure.")},
        {"name": "Shruti Bajpai", "role": "Research and Documentation Lead", "contributions": tr_route("Authored the research paper, handled documentation, and assisted with model evaluation metrics.")},
        {"name": "Sudhanshu Tiwari", "role": "Presentation and Support", "contributions": tr_route("Developed the project presentation slides, assisted in data preprocessing steps, and ensured overall project coherence.")},
        {"name": "Vaishnavi Singh", "role": "Presentation and Support", "contributions": tr_route("Contributed to the project presentation design, provided general technical support, and assisted with final testing and deployment.")}
    ]
    project_summary = tr_route("This project, AgroScan AI, uses a deep learning model (EfficientNetB0) trained on the PlantVillage dataset to instantly diagnose common crop diseases in potatoes, tomatoes, and bell peppers. The goal is to provide farmers with fast, actionable, and language-accessible intelligence to protect their yield.")

    return render_template('team.html', team_members=team_members, project_summary=project_summary, title='Our Team')

def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global MODEL_LOADING, ACTIVE_MODEL_KEY
    model = MODEL_REGISTRY.get(ACTIVE_MODEL_KEY)

    last_uploaded_image_url = session.get('last_uploaded_image_url')

    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)
    if model is None:
        flash(tr_route(f"Model {ACTIVE_MODEL_KEY} failed to load. Please check files and re-run."))
        return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    if request.method == "POST":
        file_to_analyze = None
        current_image_url = None

        source_type = request.form.get('source', 'upload')

        if source_type == 'reuse' and last_uploaded_image_url:
            filename = os.path.basename(last_uploaded_image_url).split('?')[0]
            file_to_analyze = os.path.join(UPLOAD_DIR, filename)
            current_image_url = last_uploaded_image_url
            if not os.path.exists(file_to_analyze):
                flash(tr_route("Last uploaded image not found on server."))
                session.pop('last_uploaded_image_url', None)
                return redirect(url_for('predict_page'))

        elif source_type == 'sample':
            file_to_analyze = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
            current_image_url = url_for('static', filename=f'uploads/{SAMPLE_FILENAME}')

        elif source_type == 'upload':
            if "file" not in request.files or request.files["file"].filename == "":
                flash(tr_route("No image uploaded."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            file = request.files["file"]
            if not allowed_file(file.filename):
                flash(tr_route("Invalid file type. Please upload JPG or PNG."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            fname = secure_filename(file.filename)
            unique_fname = f"{int(time.time())}_{fname}"
            file_to_analyze = os.path.join(UPLOAD_DIR, unique_fname)
            file.save(file_to_analyze)
            current_image_url = url_for('static', filename=f'uploads/{unique_fname}')

        else:
             flash(tr_route("Invalid analysis source."))
             return redirect(url_for('predict_page'))


        try:
            # === CRITICAL EFFICIENTNET PREPROCESSING ===
            img = Image.open(file_to_analyze).convert("RGB").resize(IMAGE_SIZE)
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            processed_arr = preprocess_input(arr)

            # Prediction
            preds = model.predict(processed_arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            raw_name = CLASS_NAMES[idx]
            pred_name = format_class_name(raw_name)

            # Update session with the current image path
            session['last_uploaded_image_url'] = current_image_url

            # Save to user history
            timestamp = datetime.utcnow().isoformat()
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time, model_used) VALUES (?,?,?,?,?,?)",
                        (current_user.id, current_image_url, pred_name, conf, timestamp, ACTIVE_MODEL_KEY))
            conn.commit(); conn.close()

            # Prepare result data (including translation)
            info = disease_info.get(raw_name, disease_info['default'])
            lang = session.get('language','en')
            desc = get_translation(info.get('description', ''), lang)
            remedies = [get_translation(r, lang) for r in info.get('remedies', [])]

            if conf >= 85: conf_level = 'High'
            elif conf >= 60: conf_level = 'Medium'
            else: conf_level = 'Low'

            return render_template("result.html", prediction=pred_name, confidence=round(conf,2),
                                    image_path=current_image_url, description=desc, remedies=remedies,
                                    confidence_level=conf_level)
        except Exception as e:
            flash(tr_route(f"Prediction error: {e}"))
            return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

# -------------------
# 8) Tailwind CSS Templates (FIXED & NEW PAGES)
# -------------------
template_content = {
    # --- Layout Template ---
    "layout.html": """<!doctype html><html lang='{{ session.get("language", "en") }}'><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><script src="https://cdn.jsdelivr.net/gh/alpinejs/alpine@v2.x.x/dist/alpine.min.js" defer></script><title>{{ _(title or 'AgroScan AI') }}</title><style>
        body { min-height: 100vh; background-image: linear-gradient(to top right, #dcfce7, #ecfeff); }
        main { flex: 1; }
        .glass { background-color: rgba(255, 255, 255, 0.3); backdrop-filter: blur(10px); border: 1px solid rgba(255, 255, 255, 0.2); box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); border-radius: 1rem; transition: all 0.3s ease-in-out;}
        .glass-nav { background-color: rgba(255, 255, 255, 0.6); backdrop-filter: blur(8px); }
    </style></head><body class="text-gray-800 flex flex-col"><nav class="glass-nav fixed w-full z-30 shadow-md"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="currentColor"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 17.93c-2.82-.44-5.02-2.31-6.12-4.94L13 14h-2c-2.76 0-5 2.24-5 5H4c0-3.92 2.68-7.23 6.33-8.15C10.12 11.23 11.06 12 12 12s1.88-.77 2.67-1.15c.16-.08.31-.17.46-.26 3.65.92 6.33 4.23 6.33 8.15h-2c0-2.76-2.24-5-5-5h-2l6.12 4.94z"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="flex items-center space-x-4"><div class="hidden md:flex items-center space-x-4">{% if current_user.is_authenticated %}<span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span><a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Dashboard') }}</a><a href='{{ url_for("predict_page") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Analyze') }}</a><a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('History') }}</a><a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Metrics') }}</a><a href='{{ url_for("team") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Team') }}</a><a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold hover:text-blue-800 transition duration-150'>{{ _('Logout') }}</a>{% else %}<a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Login') }}</a><a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Register') }}</a>{% endif %}</div><div x-data="{ open: false }" class="relative"><button @click="open = !open" class="text-gray-600 hover:text-green-600 flex items-center p-2 rounded-lg hover:bg-gray-100 transition duration-150"><svg class="w-5 h-5 mr-1" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M7.75 2.75a.75.75 0 00-1.5 0v1.258a32.987 32.987 0 00-3.599.278.75.75 0 10.198 1.487A31.545 31.545 0 018.7 5.545 19.38 19.38 0 017 9.762V11.5a.75.75 0 001.5 0V9.762c0-.683.085-1.35.25-1.994.169-.65.4-1.28.68-1.86A7.94 7.94 0 0112 6.561v4.939a.75.75 0 101.5 0V6.561a7.94 7.94 0 012.07 1.407c.28.58.512 1.21.68 1.86.165.643.25 1.311.25 1.994V11.5a.75.75 0 001.5 0V9.762a19.38 19.38 0 01-1.7 4.217 31.545 31.545 0 014.101-.987.75.75 0 00.198-1.487 32.987 32.987 0 00-3.599-.278V2.75a.75.75 0 00-1.5 0v1.439a7.94 7.94 0 01-3.12 1.548A7.94 7.94 0 019.25 4.19V2.75zM8.5 13.25a.75.75 0 00-1.5 0v2a.75.75 0 001.5 0v-2z" /></svg>{{ languages[session.get('language', 'en')] or 'Language' }}</button><div x-show="open" @click.away="open = false" class="absolute right-0 mt-2 w-36 bg-white border border-gray-200 rounded-md shadow-lg z-50 overflow-hidden" style="display: none;">{% for lang_code, lang_name in languages.items() %}<a href='{{ url_for("set_language", lang=lang_code) }}' class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100 transition duration-100">{{ lang_name }}</a>{% endfor %}</div></div></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6 flex-grow'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6 border-t mt-auto w-full'>© 2025 AgroScan AI - {{ _('Model') }}: {{ ACTIVE_MODEL_KEY or 'N/A' }}</footer></body></html>"""
    , "home.html": """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-8 items-center glass p-10 rounded-xl'><div class='order-2 md:order-1'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900'>{{ _('Revolutionize Your Farming with AI') }}</h1><p class='mt-4 text-lg text-gray-700'>{{ _('Instantly detect crop diseases with a simple photo. Get actionable advice to protect your harvest and increase your yield.') }}</p><div class='mt-8 flex space-x-4'><a href='{{ url_for("login") }}' class='bg-blue-600 hover:bg-blue-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-xl hover:scale-[1.03]'>{{ _('Login to Start') }}</a><a href='{{ url_for("register") }}' class='bg-white hover:bg-gray-100 text-gray-700 px-6 py-3 rounded-lg font-bold transition duration-200 shadow-xl hover:scale-[1.03]'>{{ _('Register') }}</a></div></div><div class='order-1 md:order-2'><img src='{{ url_for("static", filename="uploads/sample_leaf_ai.png") }}' class='rounded-lg shadow-xl w-full border border-gray-200' alt='Sample Leaf Image'></div></div>{% endblock %}"""
    , "home_logged_in.html": """{% extends 'layout.html' %}{% block content %}<div class='glass p-10 rounded-xl mb-8 transition duration-300 shadow-xl'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900 mb-2'>{{ _('Welcome,') }} <span class='text-green-600'>{{ current_user.username }}!</span></h1><p class='text-lg text-gray-700'>{{ _('Your AI-powered crop management dashboard is ready.') }}</p></div><div class='grid lg:grid-cols-3 gap-6'><div class='lg:col-span-2 glass p-6 rounded-xl space-y-6 transition duration-300 shadow-xl'><h3 class='text-2xl font-bold text-gray-800 mb-4'>{{ _('What to do next?') }}</h3><div class='grid sm:grid-cols-3 gap-4'><a href='{{ url_for("predict_page") }}' class='bg-green-500 hover:bg-green-600 text-white p-4 rounded-lg font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>🔍</div>{{ _('Start Analysis') }}</a><a href='{{ url_for("dashboard") }}' class='bg-blue-500 hover:bg-blue-600 text-white p-4 rounded-lg font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>📊</div>{{ _('Check Dashboard') }}</a><a href='{{ url_for("history") }}' class='bg-gray-500 hover:bg-gray-600 text-white p-4 rounded-lg font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>📜</div>{{ _('View History') }}</a></div><h3 class='text-2xl font-bold text-gray-800 mt-8 mb-3 border-t pt-4'>{{ _('Quick Guide') }}</h3><div class='space-y-4'><div class='p-4 bg-white/70 rounded-lg border border-green-200 shadow-md'><h4 class='font-bold'>1. {{ _('Analyze') }} ({{ url_for("predict_page") }})</h4><p class='text-sm text-gray-700'>{{ _('Upload a clear image of a symptomatic plant leaf. Our AI will return a diagnosis and suggested remedies.') }}</p></div><div class='p-4 bg-white/70 rounded-lg border border-blue-200 shadow-md'><h4 class='font-bold'>2. {{ _('Dashboard') }} ({{ url_for("dashboard") }})</h4><p class='text-sm text-gray-700'>{{ _('Review your past activity and see the distribution of diseases you have detected over time.') }}</p></div><div class='p-4 bg-white/70 rounded-lg border border-gray-200 shadow-md'><h4 class='font-bold'>3. {{ _('Metrics') }} ({{ url_for("metrics") }})</h4><p class='text-sm text-gray-700'>{{ _('View the scientific performance data (Confusion Matrix, Classification Report) of the core AI model.') }}</p></div></div></div><div class='lg:col-span-1 glass p-6 rounded-xl transition duration-300 shadow-xl'><h3 class='text-2xl font-bold text-gray-800 mb-4'>{{ _('System Status') }}</h3><div class='space-y-3'><div class='p-3 bg-white/70 rounded-lg border border-green-200'><span class='font-semibold'>{{ _('Active Model') }}:</span> {{ ACTIVE_MODEL_KEY }}</div><div class='p-3 bg-white/70 rounded-lg border border-blue-200'><span class='font-semibold'>{{ _('Input Size') }}:</span> 128x128 pixels</div><div class='p-3 bg-white/70 rounded-lg border border-gray-200'><span class='font-semibold'>{{ _('Language') }}:</span> {{ languages[session.get('language', 'en')] or 'English' }}</div><a href='{{ url_for("predict_page") }}' class='w-full block text-center bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold shadow-md mt-6 transition duration-300 hover:scale-[1.03]'>{{ _('Jump to Analysis') }}</a></div></div></div>{% endblock %}"""
    , "login.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Login') }}</h2><form method='post' action='{{ url_for("login") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Sign in') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Not a member?') }} <a href='{{ url_for("register") }}' class='text-green-600 hover:text-green-700'>{{ _('Register here') }}</a></p></div>{% endblock %}"""
    , "register.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Register') }}</h2><form method='post' action='{{ url_for("register") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Register') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Already a member?') }} <a href='{{ url_for("login") }}' class='text-green-600 hover:text-green-700'>{{ _('Login here') }}</a></p></div>{% endblock %}"""
    , "predict.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Predict Disease') }}</h2>{% if model_loading %}<div class='text-center p-8 bg-yellow-50 rounded-lg'><div class='animate-spin w-8 h-8 border-4 border-yellow-600 border-t-transparent rounded-full mx-auto'></div><p class='mt-4 text-lg text-yellow-800 font-semibold'>{{ _('Model is loading...') }}</p><p class='text-sm text-yellow-700'>{{ _('This may take up to 60 seconds. Please wait on this page.') }}</p></div>{% else %}<form id='upload-form' method='post' action='{{ url_for("predict_page") }}' enctype='multipart/form-data'><input type="hidden" name="source" id="source-input" value="upload"><label for='file-upload' class='block border-2 border-dashed border-gray-300 p-10 rounded-xl cursor-pointer hover:border-green-500 transition duration-200'><input id='file-upload' name='file' type='file' accept='image/jpeg, image/png, image/jpg' class='hidden' onchange="document.getElementById('source-input').value = 'upload'; document.getElementById('file-info').textContent = 'File: ' + this.files[0].name;"><div class='text-center'><svg class='mx-auto w-12 h-12 text-gray-400' fill='none' stroke='currentColor' viewBox='0 0 24 24'><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M7 16a4 4 0 01-4-4V7a4 4 0 014-4h10a4 4 0 014 4v5a4 4 0 01-4 4h-3'></path><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M16 12l-4-4m0 0L8 12m4-4v11'></path></svg><p class='mt-2 text-gray-600 font-semibold'>{{ _('Click to upload leaf image') }}</p><p id='file-info' class='mt-1 text-sm text-gray-500'>{{ _('JPG or PNG recommended.') }}</p></div></label><div class='mt-6'><button id='submit-button' class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.01]'>{{ _('Detect Disease') }}</button><div id='loader' class='hidden w-full flex items-center justify-center p-3 bg-gray-100 rounded-lg mt-6'><div class='animate-spin w-6 h-6 border-4 border-green-600 border-t-transparent rounded-full mr-3'></div><span>{{ _('Analyzing...') }}</span></div></div></form><div class="mt-8 pt-6 border-t border-gray-200"><h3 class="text-xl font-bold mb-4">{{ _('Quick Analysis') }}</h3><div class="space-y-3">{% if last_uploaded_image_url %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="reuse"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-300 border border-gray-200 shadow-sm hover:scale-[1.01]"><svg class="w-5 h-5 mr-3 text-blue-600" fill="currentColor" viewBox="0 0 20 20"><path d="M10 12a2 2 0 100-4 2 2 0 000 4z"/><path fill-rule="evenodd" d="M.458 10C1.732 5.943 5.522 3 10 3s8.268 2.943 9.542 7c-1.274 4.057-5.064 7-9.542 7S1.732 14.057.458 10zM14 10a4 4 0 11-8 0 4 4 0 018 0z" clip-rule="evenodd"/></svg>{{ _('Reuse last uploaded image') }}</button></form>{% endif %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="sample"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-300 border border-gray-200 shadow-sm hover:scale-[1.01]"><svg class="w-5 h-5 mr-3 text-green-600" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M16.707 5.293a1 1 0 010 1.414l-8 8a1 1 0 01-1.414 0l-4-4a1 1 0 011.414-1.414L8 12.586l7.293-7.293a1 1 0 011.414 0z" clip-rule="evenodd"/></svg>{{ _('Use internal sample image') }}</button></form></div></div></div><script>const form=document.getElementById('upload-form'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); form.addEventListener('submit', e=>{ submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endif %}{% endblock %}"""
    , "result.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Detection Result') }}</h2><div class='grid md:grid-cols-2 gap-8 items-start'><div class='md:w-full'><img src='{{ image_path }}' class='rounded-lg w-full shadow-md' alt='Uploaded Leaf'></div><div class='md:w-full space-y-5'><div class='glass p-5 rounded-lg border-l-4 border-green-600'><p class='text-lg text-gray-700 mb-1'>{{ _('Predicted Condition') }}:</p><h3 class='text-2xl font-extrabold text-green-700'>{{ prediction }}</h3><p class='mt-2 text-gray-700'>{{ _('Confidence') }}: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-4 mt-3 overflow-hidden' title='{{ confidence }}%'>{% if confidence_level == 'High' %}<div class='h-4 bg-green-500' style='width: {{ confidence }}%;'></div>{% elif confidence_level == 'Medium' %}<div class='h-4 bg-yellow-500' style='width: {{ confidence }}%;'></div>{% else %}<div class='h-4 bg-red-500' style='width: {{ confidence }}%;'></div>{% endif %}</div></div><div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('About this Condition') }}</h4><p class='text-gray-700'>{{ description }}</p></div>{% if remedies %}<div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('Suggested Actions') }}</h4><ul class='list-disc pl-5 text-gray-700'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}</div></div><div class='mt-8 text-center'><a href='{{ url_for("predict_page") }}' class='bg-gray-600 hover:bg-gray-700 text-white px-6 py-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Analyze Another Image') }}</a></div></div>{% endblock %}"""
    , "dashboard.html": """{% extends 'layout.html' %}{% block content %}<script src="https://cdn.jsdelivr.net/npm/chart.js@3.7.1/dist/chart.min.js"></script><h2 class='text-3xl font-bold mb-6'>{{ _('Dashboard') }}</h2><div class='glass p-6 rounded-xl shadow-xl transition duration-300'><h3 class='text-xl font-bold mb-4'>{{ _('Model Configuration') }}</h3><div class='mt-4'><form method='post' action='{{ url_for("switch_model") }}'><label class='block text-gray-700'>{{ _('Select active model') }}:<select name='model' class='mt-2 p-2 border border-gray-300 rounded-lg w-full md:w-1/3 bg-white/70'> <option value='best' {% if active_model=='best' %}selected{% endif %}>{{ _('Best model') }} (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>{{ _('Fallback model') }} (crop_disease_model.h5)</option></select></label><div class='mt-4'><button class='bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold transition duration-300 shadow-md hover:scale-[1.03]'>{{ _('Switch Model') }}</button></div></form></div><h3 class='text-xl font-bold mt-8 mb-4 border-t pt-4'>{{ _('Your Prediction Statistics') }}</h3>{% if disease_data.labels %}<div class='grid grid-cols-1 lg:grid-cols-2 gap-8 mt-4'><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3 text-center'>{{ _('Detection Distribution') }}</h4><div class='h-64'><canvas id='diseaseChart'></canvas></div></div><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3 text-center'>{{ _('Activity Over Time') }}</h4><div class='h-64'><canvas id='monthlyChart'></canvas></div></div></div>{% else %}<p class='text-gray-500'>{{ _('No detections yet. Analyze to populate your dashboard.') }}</p>{% endif %}</div><script>const diseaseData={{ disease_data | tojson }}, monthlyData={{ monthly_data | tojson }}; if(diseaseData.labels.length>0){const ctx=document.getElementById('diseaseChart').getContext('2d'); new Chart(ctx, {type:'pie',data:{labels:diseaseData.labels,datasets:[{data:diseaseData.values,backgroundColor:['#059669','#F59E0B','#EF4444','#6366F1','#06B6D4','#F472B6']}]},options:{responsive:true, maintainAspectRatio: false, plugins:{legend:{position:'right', labels: {color: '#374151'}}}}});} if(monthlyData.labels.length>0){const ctx=document.getElementById('monthlyChart').getContext('2d'); new Chart(ctx, {type:'bar',data:{labels:monthlyData.labels,datasets:[{label:'{{ _("Number of Scans") }}',data:monthlyData.values,backgroundColor:'#059669',borderColor:'#047857',borderWidth:1}]},options:{responsive:true, maintainAspectRatio: false, scales:{y:{beginAtZero:true, ticks: {stepSize: 1, color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}, x: {ticks: {color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}}}});}</script>{% endblock %}"""
    , "history.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Prediction History') }}</h2>{% if history %}<div class='grid md:grid-cols-3 gap-6'>{% for p in history %}<div class='glass p-0 rounded-xl shadow-md overflow-hidden hover:shadow-xl transition duration-300 hover:scale-[1.01]'><img src='{{ p.image_path }}' class='w-full h-40 object-cover' alt='Uploaded Leaf'><div class='p-4'><div class='text-sm text-gray-500'>{{ p.timestamp }}</div><div class='font-bold text-lg mt-1'>{{ p.prediction }}</div><div class='text-sm text-gray-600'>{{ _('Confidence') }}: {{ p.confidence | int }}%</div><div class='text-xs text-gray-500'>{{ _('Model') }}: {{ p.model_used }}</div></div></div>{% endfor %}</div>{% else %}<p class='text-gray-500'>{{ _('No saved predictions yet.') }}</p>{% endif %}{% endblock %}"""
    , "metrics.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Training Metrics') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><p class='text-gray-700 mb-4'>{{ _('The files below are loaded from your Colab root directory.') }}</p>{% if training_results %}<h3 class='text-xl font-bold mt-4'>{{ _('Final Training Results') }}</h3><pre class='whitespace-pre-wrap bg-white/70 p-4 rounded-lg text-sm border border-gray-200 shadow-inner'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}{% if confusion_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Confusion Matrix') }}</h3><img src='{{ url_for("static", filename="confusion_matrix.png") }}' class='max-w-full rounded-lg shadow-md mt-2 border border-gray-200' alt='Confusion Matrix'>{% endif %}{% if report_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Classification Report') }}</h3><pre class='bg-white/70 p-4 rounded-lg text-sm border border-gray-200 shadow-inner'>{{ report_text }}</pre>{% endif %}{% if not training_results and not confusion_exists and not report_exists %}<p class='text-gray-500'>{{ _('No training metrics files found (training_results.json, confusion_matrix.png, classification_report.txt).') }}</p>{% endif %}</div>{% endblock %}"""
    , "team.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-8 text-center'>{{ _('Meet the Team') }}</h2><div class='glass p-8 rounded-xl shadow-xl'><h3 class='text-xl font-semibold mb-4'>{{ _('Project Overview') }}</h3><p class='text-gray-700 mb-8'>{{ project_summary }}</p><h3 class='text-xl font-semibold mb-6'>{{ _('The Contributors') }}</h3><div class='space-y-6'>{% for member in team_members %}<div class='p-4 bg-white/70 rounded-lg border border-gray-200 shadow-md transition duration-300 hover:scale-[1.01] hover:shadow-xl'><div class='flex items-center space-x-4'><div class='w-10 h-10 flex items-center justify-center bg-green-500 text-white rounded-full text-lg font-extrabold shadow-lg'>{{ member.name[0] }}</div><div><h4 class='text-lg font-bold text-gray-800'>{{ member.name }} <span class='text-sm text-gray-500'>({{ member.role }})</span></h4><p class='text-sm text-gray-700 mt-1'>{{ _('Contribution') }}: {{ member.contributions }}</p></div></div></div>{% endfor %}</div></div>{% endblock %}"""
}

# Write HTML templates to files
for name, content in template_content.items():
    with open(os.path.join(TEMPLATES_DIR, name), "w", encoding="utf-8") as f:
        f.write(content)

# -------------------
# 9) Start Ngrok and Flask
# -------------------
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    # Start background model loading immediately
    threading.Thread(target=_background_preload, daemon=True).start()

    print("Colab Flask app (dual-model, Tailwind UI) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("\nPaste your ngrok authtoken below. DO NOT re-run if model is still loading.")

    # Setup Ngrok
    try:
        token = getpass("ngrok authtoken (hidden): ")
        if token and token.strip():
            ngrok.set_auth_token(token.strip())
            public_url = ngrok.connect(5000)
            print(f"🔗 ngrok tunnel: {public_url}")
        else:
            print("Ngrok token not provided. App will only be accessible via local Colab link.")
    except Exception as e:
        print(f"Ngrok setup failed: {e}")

    # Run Flask
    app.run(host="0.0.0.0", port=5000, debug=False)

[model] Loading 'best' from best_crop_model.h5 ...Colab Flask app (dual-model, Tailwind UI) starting...
Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)

Paste your ngrok authtoken below. DO NOT re-run if model is still loading.

[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ALL models preloaded and ready.
ngrok authtoken (hidden): ··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:00] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:01] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:03] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:09] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:09] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:15] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:41] "GET /dashboard HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:43:42] "GET /predict HTTP/1.1" 200 -
INFO:

In [32]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINAL PRODUCTION SCRIPT
# =========================================================================
#  -- NEW UI: Modern Team Profile Card Grid (Mimics User's Image) --
# =========================================================================

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, time, threading, sqlite3, json, atexit
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

# -------------------
# 2) Config, File System, and Constants
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
IMAGE_SIZE = (128, 128)
ALLOWED_EXT = {"png","jpg","jpeg"}
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# Model paths and global state
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
MODEL_LOADING = True
SAMPLE_FILENAME = "sample_leaf_ai.png"

# -------------------
# 3) Class names, Formatting, and Languages
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# -------------------
# 4) Database Setup
# -------------------
if os.path.exists(DB_PATH): os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     time TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Sample image generation for UI and Quick Analysis feature
sample_img = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# -------------------
# 5) Disease Information
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.','remedies':[]},
    'Potato___Early_blight': {'description':'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies':['Use certified seed','Apply fungicides containing mancozeb or chlorothalonil','Rotate crops']},
    'Potato___Late_blight': {'description':'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside.', 'remedies':['Remove infected plants','Protective fungicides proactively','Ensure good drainage and air circulation']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description':'Causes small, water-soaked spots on tomato leaves and fruit.','remedies':['Avoid working when wet','Use copper-based bactericides','Mulch around plants']},
    'Tomato_Early_blight': {'description':'Fungal disease, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies':['Ensure proper spacing','Apply preventative fungicides','Water at the base of the plant']},
    'Tomato_Late_blight': {'description':'A serious fungal disease characterized by large, greasy, grey-green spots on leaves.','remedies':['Apply preventative fungicides like chlorothalonil or copper','Remove and destroy infected plants immediately']},
    'Tomato_Leaf_Mold': {'description':'Fungal disease causing pale spots on the upper leaf surface and a velvety, olive-green mold on the underside.','remedies':['Improve air circulation and reduce humidity','Prune lower leaves']},
    'Tomato_Septoria_leaf_spot': {'description':'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.','remedies':['Remove infected leaves','Improve air circulation','Apply fungicides']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description':'Tiny pests causing stippling (tiny yellow or white dots) on leaves, leading to webbing.', 'remedies':['Spray plants with water','Apply insecticidal soap or horticultural oil','Introduce natural predators']},
    'Tomato__Target_Spot': {'description':'A fungal disease causing necrotic spots with concentric rings.', 'remedies':['Apply fungicides effective against Corynespora cassiicola','Improve air circulation and reduce leaf wetness']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description':'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies':['Control whitefly populations','Remove and destroy infected plants immediately','Use virus-resistant varieties']},
    'Tomato__Tomato_mosaic_virus': {'description':'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation.', 'remedies':['There is no cure; remove and destroy infected plants','Wash hands and tools thoroughly']},
    'Tomato_healthy': {'description':'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.','remedies':[]},
    'default': {'description':'Information for this condition is not available.','remedies':['Consult local extension services']}
}

# -------------------
# 6) Model Management & Translation Setup
# -------------------
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# Route-safe translation helper
def tr_route(text):
    lang = session.get('language', 'en')
    return get_translation(text, lang)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_globals():
    def tr(text):
        return tr_route(text)
    return dict(_=tr, languages=LANGUAGES, ACTIVE_MODEL_KEY=ACTIVE_MODEL_KEY)

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = tf.keras.models.load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

def _background_preload():
    global MODEL_LOADING
    MODEL_LOADING = True
    for k in ["best", "fallback"]:
        load_model_lazy(k)
    MODEL_LOADING = False
    print("ALL models preloaded and ready.")

# -------------------
# 7) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

@app.route("/")
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template("home_logged_in.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("home"))
        flash(tr_route("Invalid username or password."))
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash(tr_route("Registration successful! Please login.")); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash(tr_route("Username already taken."))
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    global ACTIVE_MODEL_KEY
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()

    # Chart 1: Prediction Distribution (Pie Chart Data)
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    disease_rows = cur.fetchall()
    disease_data = {'labels':[r[0] for r in disease_rows], 'values':[r[1] for r in disease_rows]}

    # Chart 2: Activity Over Time (Bar Chart Data)
    cur.execute("SELECT strftime('%Y-%m', time) as month, COUNT(*) as count FROM predictions WHERE user_id = ? GROUP BY month ORDER BY month", (current_user.id,))
    monthly_rows = cur.fetchall()
    monthly_data = {
        "labels": [datetime.strptime(row[0], '%Y-%m').strftime('%b %Y') for row in monthly_rows],
        "values": [row[1] for row in monthly_rows]
    }

    conn.close()
    return render_template("dashboard.html", disease_data=disease_data, monthly_data=monthly_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(tr_route(f"Active model switched to: {m}"))
    else:
        flash(tr_route("Unknown model key"))
    return redirect(url_for('dashboard'))

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time, model_used FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    ist_offset = timedelta(hours=5, minutes=30)
    for img_url,pred,conf,ts,model_used in rows:
        try: ts_dt = datetime.fromisoformat(ts)
        except: ts_dt = datetime.strptime(ts.split('.')[0], "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + ist_offset
        hist.append({
            "image_path":img_url,"prediction":pred,
            "confidence":int(round(conf)), "timestamp":ist.strftime("%b %d, %Y %I:%M %p"),
            "model_used": model_used
        })
    return render_template("history.html", history=hist)

@app.route("/metrics")
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

@app.route("/team")
def team():
    # Define the team data to be passed to the template
    team_members = [
        {"name": "Umar Iqbal", "role": "Project Leader & Lead Developer", "contributions": tr_route("Led the entire project lifecycle, developed the core prediction pipeline, implemented the Flask backend, and designed the database structure."), "img": url_for('static', filename='uploads/umar.png')},
        {"name": "Shruti Bajpai", "role": "Research and Documentation Lead", "contributions": tr_route("Authored the research paper, handled documentation, and assisted with model evaluation metrics."), "img": url_for('static', filename='uploads/shruti.png')},
        {"name": "Sudhanshu Tiwari", "role": "Presentation and Support", "contributions": tr_route("Developed the project presentation slides, assisted in data preprocessing steps, and ensured overall project coherence."), "img": url_for('static', filename='uploads/sudhanshu.png')},
        {"name": "Vaishnavi Singh", "role": "Presentation and Support", "contributions": tr_route("Contributed to the project presentation design, provided general technical support, and assisted with final testing and deployment."), "img": url_for('static', filename='uploads/vaishnavi.png')}
    ]
    project_summary = tr_route("This project, AgroScan AI, uses a deep learning model (EfficientNetB0) trained on the PlantVillage dataset to instantly diagnose common crop diseases in potatoes, tomatoes, and bell peppers. The goal is to provide farmers with fast, actionable, and language-accessible intelligence to protect their yield.")

    return render_template('team.html', team_members=team_members, project_summary=project_summary, title='Our Team')

def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global MODEL_LOADING, ACTIVE_MODEL_KEY
    model = MODEL_REGISTRY.get(ACTIVE_MODEL_KEY)

    last_uploaded_image_url = session.get('last_uploaded_image_url')

    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)
    if model is None:
        flash(tr_route(f"Model {ACTIVE_MODEL_KEY} failed to load. Please check files and re-run."))
        return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    if request.method == "POST":
        file_to_analyze = None
        current_image_url = None

        source_type = request.form.get('source', 'upload')

        if source_type == 'reuse' and last_uploaded_image_url:
            filename = os.path.basename(last_uploaded_image_url).split('?')[0]
            file_to_analyze = os.path.join(UPLOAD_DIR, filename)
            current_image_url = last_uploaded_image_url
            if not os.path.exists(file_to_analyze):
                flash(tr_route("Last uploaded image not found on server."))
                session.pop('last_uploaded_image_url', None)
                return redirect(url_for('predict_page'))

        elif source_type == 'sample':
            file_to_analyze = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
            current_image_url = url_for('static', filename=f'uploads/{SAMPLE_FILENAME}')

        elif source_type == 'upload':
            if "file" not in request.files or request.files["file"].filename == "":
                flash(tr_route("No image uploaded."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            file = request.files["file"]
            if not allowed_file(file.filename):
                flash(tr_route("Invalid file type. Please upload JPG or PNG."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            fname = secure_filename(file.filename)
            unique_fname = f"{int(time.time())}_{fname}"
            file_to_analyze = os.path.join(UPLOAD_DIR, unique_fname)
            file.save(file_to_analyze)
            current_image_url = url_for('static', filename=f'uploads/{unique_fname}')

        else:
             flash(tr_route("Invalid analysis source."))
             return redirect(url_for('predict_page'))


        try:
            # === CRITICAL EFFICIENTNET PREPROCESSING ===
            img = Image.open(file_to_analyze).convert("RGB").resize(IMAGE_SIZE)
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            processed_arr = preprocess_input(arr)

            # Prediction
            preds = model.predict(processed_arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            raw_name = CLASS_NAMES[idx]
            pred_name = format_class_name(raw_name)

            # Update session with the current image path
            session['last_uploaded_image_url'] = current_image_url

            # Save to user history
            timestamp = datetime.utcnow().isoformat()
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time, model_used) VALUES (?,?,?,?,?,?)",
                        (current_user.id, current_image_url, pred_name, conf, timestamp, ACTIVE_MODEL_KEY))
            conn.commit(); conn.close()

            # Prepare result data (including translation)
            info = disease_info.get(raw_name, disease_info['default'])
            lang = session.get('language','en')
            desc = get_translation(info.get('description', ''), lang)
            remedies = [get_translation(r, lang) for r in info.get('remedies', [])]

            if conf >= 85: conf_level = 'High'
            elif conf >= 60: conf_level = 'Medium'
            else: conf_level = 'Low'

            return render_template("result.html", prediction=pred_name, confidence=round(conf,2),
                                    image_path=current_image_url, description=desc, remedies=remedies,
                                    confidence_level=conf_level)
        except Exception as e:
            flash(tr_route(f"Prediction error: {e}"))
            return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

# -------------------
# 8) Tailwind CSS Templates (FIXED & NEW PAGES)
# -------------------
template_content = {
    # --- Layout Template ---
    "layout.html": """<!doctype html><html lang='{{ session.get("language", "en") }}'><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><script src="https://cdn.jsdelivr.net/gh/alpinejs/alpine@v2.x.x/dist/alpine.min.js" defer></script><title>{{ _(title or 'AgroScan AI') }}</title><style>
        body { min-height: 100vh; background-image: linear-gradient(to top right, #dcfce7, #ecfeff); }
        main { flex: 1; }
        .glass { background-color: rgba(255, 255, 255, 0.3); backdrop-filter: blur(10px); border: 1px solid rgba(255, 255, 255, 0.2); box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); border-radius: 1rem; transition: all 0.3s ease-in-out;}
        .glass-nav { background-color: rgba(255, 255, 255, 0.6); backdrop-filter: blur(8px); }
    </style></head><body class="text-gray-800 flex flex-col"><nav class="glass-nav fixed w-full z-30 shadow-md"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="currentColor"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 17.93c-2.82-.44-5.02-2.31-6.12-4.94L13 14h-2c-2.76 0-5 2.24-5 5H4c0-3.92 2.68-7.23 6.33-8.15C10.12 11.23 11.06 12 12 12s1.88-.77 2.67-1.15c.16-.08.31-.17.46-.26 3.65.92 6.33 4.23 6.33 8.15h-2c0-2.76-2.24-5-5-5h-2l6.12 4.94z"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="flex items-center space-x-4"><div class="hidden md:flex items-center space-x-4">{% if current_user.is_authenticated %}<span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span><a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Dashboard') }}</a><a href='{{ url_for("predict_page") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Analyze') }}</a><a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('History') }}</a><a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Metrics') }}</a><a href='{{ url_for("team") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Team') }}</a><a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold hover:text-blue-800 transition duration-150'>{{ _('Logout') }}</a>{% else %}<a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Login') }}</a><a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Register') }}</a>{% endif %}</div><div x-data="{ open: false }" class="relative"><button @click="open = !open" class="text-gray-600 hover:text-green-600 flex items-center p-2 rounded-lg hover:bg-gray-100 transition duration-150"><svg class="w-5 h-5 mr-1" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M7.75 2.75a.75.75 0 00-1.5 0v1.258a32.987 32.987 0 00-3.599.278.75.75 0 10.198 1.487A31.545 31.545 0 018.7 5.545 19.38 19.38 0 017 9.762V11.5a.75.75 0 001.5 0V9.762c0-.683.085-1.35.25-1.994.169-.65.4-1.28.68-1.86A7.94 7.94 0 0112 6.561v4.939a.75.75 0 101.5 0V6.561a7.94 7.94 0 012.07 1.407c.28.58.512 1.21.68 1.86.165.643.25 1.311.25 1.994V11.5a.75.75 0 001.5 0V9.762a19.38 19.38 0 01-1.7 4.217 31.545 31.545 0 014.101-.987.75.75 0 00.198-1.487 32.987 32.987 0 00-3.599-.278V2.75a.75.75 0 00-1.5 0v1.439a7.94 7.94 0 01-3.12 1.548A7.94 7.94 0 019.25 4.19V2.75zM8.5 13.25a.75.75 0 00-1.5 0v2a.75.75 0 001.5 0v-2z" /></svg>{{ languages[session.get('language', 'en')] or 'Language' }}</button><div x-show="open" @click.away="open = false" class="absolute right-0 mt-2 w-36 bg-white border border-gray-200 rounded-md shadow-lg z-50 overflow-hidden" style="display: none;">{% for lang_code, lang_name in languages.items() %}<a href='{{ url_for("set_language", lang=lang_code) }}' class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100 transition duration-100">{{ lang_name }}</a>{% endfor %}</div></div></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6 flex-grow'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6 border-t mt-auto w-full'>© 2025 AgroScan AI - {{ _('Model') }}: {{ ACTIVE_MODEL_KEY or 'N/A' }}</footer></body></html>"""
    , "home.html": """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-8 items-center glass p-10 rounded-xl'><div class='order-2 md:order-1'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900'>{{ _('Revolutionize Your Farming with AI') }}</h1><p class='mt-4 text-lg text-gray-700'>{{ _('Instantly detect crop diseases with a simple photo. Get actionable advice to protect your harvest and increase your yield.') }}</p><div class='mt-8 flex space-x-4'><a href='{{ url_for("login") }}' class='bg-blue-600 hover:bg-blue-700 text-white px-6 py-3 rounded-lg font-bold transition duration-200 shadow-xl hover:scale-[1.03]'>{{ _('Login to Start') }}</a><a href='{{ url_for("register") }}' class='bg-white hover:bg-gray-100 text-gray-700 px-6 py-3 rounded-lg font-bold transition duration-200 shadow-xl hover:scale-[1.03]'>{{ _('Register') }}</a></div></div><div class='order-1 md:order-2'><img src='{{ url_for("static", filename="uploads/sample_leaf_ai.png") }}' class='rounded-lg shadow-xl w-full border border-gray-200' alt='Sample Leaf Image'></div></div>{% endblock %}"""
    , "home_logged_in.html": """{% extends 'layout.html' %}{% block content %}<div class='glass p-10 rounded-xl mb-8 transition duration-300 shadow-xl'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900 mb-2'>{{ _('Welcome,') }} <span class='text-green-600'>{{ current_user.username }}!</span></h1><p class='text-lg text-gray-700'>{{ _('Your AI-powered crop management dashboard is ready.') }}</p></div><div class='grid lg:grid-cols-3 gap-6'><div class='lg:col-span-2 glass p-6 rounded-xl space-y-6 transition duration-300 shadow-xl'><h3 class='text-2xl font-bold text-gray-800 mb-4'>{{ _('What to do next?') }}</h3><div class='grid sm:grid-cols-3 gap-4'><a href='{{ url_for("predict_page") }}' class='bg-green-500 hover:bg-green-600 text-white p-4 rounded-lg font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>🔍</div>{{ _('Start Analysis') }}</a><a href='{{ url_for("dashboard") }}' class='bg-blue-500 hover:bg-blue-600 text-white p-4 rounded-lg font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>📊</div>{{ _('Check Dashboard') }}</a><a href='{{ url_for("history") }}' class='bg-gray-500 hover:bg-gray-600 text-white p-4 rounded-lg font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>📜</div>{{ _('View History') }}</a></div><h3 class='text-2xl font-bold text-gray-800 mt-8 mb-3 border-t pt-4'>{{ _('Quick Guide') }}</h3><div class='space-y-4'><div class='p-4 bg-white/70 rounded-lg border border-green-200 shadow-md'><h4 class='font-bold'>1. {{ _('Analyze') }} ({{ url_for("predict_page") }})</h4><p class='text-sm text-gray-700'>{{ _('Upload a clear image of a symptomatic plant leaf. Our AI will return a diagnosis and suggested remedies.') }}</p></div><div class='p-4 bg-white/70 rounded-lg border border-blue-200 shadow-md'><h4 class='font-bold'>2. {{ _('Dashboard') }} ({{ url_for("dashboard") }})</h4><p class='text-sm text-gray-700'>{{ _('Review your past activity and see the distribution of diseases you have detected over time.') }}</p></div><div class='p-4 bg-white/70 rounded-lg border border-gray-200 shadow-md'><h4 class='font-bold'>3. {{ _('Metrics') }} ({{ url_for("metrics") }})</h4><p class='text-sm text-gray-700'>{{ _('View the scientific performance data (Confusion Matrix, Classification Report) of the core AI model.') }}</p></div></div></div><div class='lg:col-span-1 glass p-6 rounded-xl transition duration-300 shadow-xl'><h3 class='text-2xl font-bold text-gray-800 mb-4'>{{ _('System Status') }}</h3><div class='space-y-3'><div class='p-3 bg-white/70 rounded-lg border border-green-200'><span class='font-semibold'>{{ _('Active Model') }}:</span> {{ ACTIVE_MODEL_KEY }}</div><div class='p-3 bg-white/70 rounded-lg border border-blue-200'><span class='font-semibold'>{{ _('Input Size') }}:</span> 128x128 pixels</div><div class='p-3 bg-white/70 rounded-lg border border-gray-200'><span class='font-semibold'>{{ _('Language') }}:</span> {{ languages[session.get('language', 'en')] or 'English' }}</div><a href='{{ url_for("predict_page") }}' class='w-full block text-center bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold shadow-md mt-6 transition duration-300 hover:scale-[1.03]'>{{ _('Jump to Analysis') }}</a></div></div></div>{% endblock %}"""
    , "login.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Login') }}</h2><form method='post' action='{{ url_for("login") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Sign in') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Not a member?') }} <a href='{{ url_for("register") }}' class='text-green-600 hover:text-green-700'>{{ _('Register here') }}</a></p></div>{% endblock %}"""
    , "register.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Register') }}</h2><form method='post' action='{{ url_for("register") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Register') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Already a member?') }} <a href='{{ url_for("login") }}' class='text-green-600 hover:text-green-700'>{{ _('Login here') }}</a></p></div>{% endblock %}"""
    , "predict.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Predict Disease') }}</h2>{% if model_loading %}<div class='text-center p-8 bg-yellow-50 rounded-lg'><div class='animate-spin w-8 h-8 border-4 border-yellow-600 border-t-transparent rounded-full mx-auto'></div><p class='mt-4 text-lg text-yellow-800 font-semibold'>{{ _('Model is loading...') }}</p><p class='text-sm text-yellow-700'>{{ _('This may take up to 60 seconds. Please wait on this page.') }}</p></div>{% else %}<form id='upload-form' method='post' action='{{ url_for("predict_page") }}' enctype='multipart/form-data'><input type="hidden" name="source" id="source-input" value="upload"><label for='file-upload' class='block border-2 border-dashed border-gray-300 p-10 rounded-xl cursor-pointer hover:border-green-500 transition duration-200'><input id='file-upload' name='file' type='file' accept='image/jpeg, image/png, image/jpg' class='hidden' onchange="document.getElementById('source-input').value = 'upload'; document.getElementById('file-info').textContent = 'File: ' + this.files[0].name;"><div class='text-center'><svg class='mx-auto w-12 h-12 text-gray-400' fill='none' stroke='currentColor' viewBox='0 0 24 24'><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M7 16a4 4 0 01-4-4V7a4 4 0 014-4h10a4 4 0 014 4v5a4 4 0 01-4 4h-3'></path><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M16 12l-4-4m0 0L8 12m4-4v11'></path></svg><p class='mt-2 text-gray-600 font-semibold'>{{ _('Click to upload leaf image') }}</p><p id='file-info' class='mt-1 text-sm text-gray-500'>{{ _('JPG or PNG recommended.') }}</p></div></label><div class='mt-6'><button id='submit-button' class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.01]'>{{ _('Detect Disease') }}</button><div id='loader' class='hidden w-full flex items-center justify-center p-3 bg-gray-100 rounded-lg mt-6'><div class='animate-spin w-6 h-6 border-4 border-green-600 border-t-transparent rounded-full mr-3'></div><span>{{ _('Analyzing...') }}</span></div></div></form><div class="mt-8 pt-6 border-t border-gray-200"><h3 class="text-xl font-bold mb-4">{{ _('Quick Analysis') }}</h3><div class="space-y-3">{% if last_uploaded_image_url %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="reuse"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-300 border border-gray-200 shadow-sm hover:scale-[1.01]"><svg class="w-5 h-5 mr-3 text-blue-600" fill="currentColor" viewBox="0 0 20 20"><path d="M10 12a2 2 0 100-4 2 2 0 000 4z"/><path fill-rule="evenodd" d="M.458 10C1.732 5.943 5.522 3 10 3s8.268 2.943 9.542 7c-1.274 4.057-5.064 7-9.542 7S1.732 14.057.458 10zM14 10a4 4 0 11-8 0 4 4 0 018 0z" clip-rule="evenodd"/></svg>{{ _('Reuse last uploaded image') }}</button></form>{% endif %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="sample"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-300 border border-gray-200 shadow-sm hover:scale-[1.01]"><svg class="w-5 h-5 mr-3 text-green-600" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M16.707 5.293a1 1 0 010 1.414l-8 8a1 1 0 01-1.414 0l-4-4a1 1 0 011.414-1.414L8 12.586l7.293-7.293a1 1 0 011.414 0z" clip-rule="evenodd"/></svg>{{ _('Use internal sample image') }}</button></form></div></div></div><script>const form=document.getElementById('upload-form'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); form.addEventListener('submit', e=>{ submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endif %}{% endblock %}"""
    , "result.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Detection Result') }}</h2><div class='grid md:grid-cols-2 gap-8 items-start'><div class='md:w-full'><img src='{{ image_path }}' class='rounded-lg w-full shadow-md' alt='Uploaded Leaf'></div><div class='md:w-full space-y-5'><div class='glass p-5 rounded-lg border-l-4 border-green-600'><p class='text-lg text-gray-700 mb-1'>{{ _('Predicted Condition') }}:</p><h3 class='text-2xl font-extrabold text-green-700'>{{ prediction }}</h3><p class='mt-2 text-gray-700'>{{ _('Confidence') }}: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-4 mt-3 overflow-hidden' title='{{ confidence }}%'>{% if confidence_level == 'High' %}<div class='h-4 bg-green-500' style='width: {{ confidence }}%;'></div>{% elif confidence_level == 'Medium' %}<div class='h-4 bg-yellow-500' style='width: {{ confidence }}%;'></div>{% else %}<div class='h-4 bg-red-500' style='width: {{ confidence }}%;'></div>{% endif %}</div></div><div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('About this Condition') }}</h4><p class='text-gray-700'>{{ description }}</p></div>{% if remedies %}<div class='glass p-5 rounded-lg'><h4 class='font-bold text-lg mb-2'>{{ _('Suggested Actions') }}</h4><ul class='list-disc pl-5 text-gray-700'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}</div></div><div class='mt-8 text-center'><a href='{{ url_for("predict_page") }}' class='bg-gray-600 hover:bg-gray-700 text-white px-6 py-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Analyze Another Image') }}</a></div></div>{% endblock %}"""
    , "dashboard.html": """{% extends 'layout.html' %}{% block content %}<script src="https://cdn.jsdelivr.net/npm/chart.js@3.7.1/dist/chart.min.js"></script><h2 class='text-3xl font-bold mb-6'>{{ _('Dashboard') }}</h2><div class='glass p-6 rounded-xl shadow-xl transition duration-300'><h3 class='text-xl font-bold mb-4'>{{ _('Model Configuration') }}</h3><div class='mt-4'><form method='post' action='{{ url_for("switch_model") }}'><label class='block text-gray-700'>{{ _('Select active model') }}:<select name='model' class='mt-2 p-2 border border-gray-300 rounded-lg w-full md:w-1/3 bg-white/70'> <option value='best' {% if active_model=='best' %}selected{% endif %}>{{ _('Best model') }} (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>{{ _('Fallback model') }} (crop_disease_model.h5)</option></select></label><div class='mt-4'><button class='bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold transition duration-300 shadow-md hover:scale-[1.03]'>{{ _('Switch Model') }}</button></div></form></div><h3 class='text-xl font-bold mt-8 mb-4 border-t pt-4'>{{ _('Your Prediction Statistics') }}</h3>{% if disease_data.labels %}<div class='grid grid-cols-1 lg:grid-cols-2 gap-8 mt-4'><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3 text-center'>{{ _('Detection Distribution') }}</h4><div class='h-64'><canvas id='diseaseChart'></canvas></div></div><div class='glass p-5 rounded-xl'><h4 class='font-bold mb-3 text-center'>{{ _('Activity Over Time') }}</h4><div class='h-64'><canvas id='monthlyChart'></canvas></div></div></div>{% else %}<p class='text-gray-500'>{{ _('No detections yet. Analyze to populate your dashboard.') }}</p>{% endif %}</div><script>const diseaseData={{ disease_data | tojson }}, monthlyData={{ monthly_data | tojson }}; if(diseaseData.labels.length>0){const ctx=document.getElementById('diseaseChart').getContext('2d'); new Chart(ctx, {type:'pie',data:{labels:diseaseData.labels,datasets:[{data:diseaseData.values,backgroundColor:['#059669','#F59E0B','#EF4444','#6366F1','#06B6D4','#F472B6']}]},options:{responsive:true, maintainAspectRatio: false, plugins:{legend:{position:'right', labels: {color: '#374151'}}}}});} if(monthlyData.labels.length>0){const ctx=document.getElementById('monthlyChart').getContext('2d'); new Chart(ctx, {type:'bar',data:{labels:monthlyData.labels,datasets:[{label:'{{ _("Number of Scans") }}',data:monthlyData.values,backgroundColor:'#059669',borderColor:'#047857',borderWidth:1}]},options:{responsive:true, maintainAspectRatio: false, scales:{y:{beginAtZero:true, ticks: {stepSize: 1, color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}, x: {ticks: {color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}}}});}</script>{% endblock %}"""
    , "history.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Prediction History') }}</h2>{% if history %}<div class='grid md:grid-cols-3 gap-6'>{% for p in history %}<div class='glass p-0 rounded-xl shadow-md overflow-hidden hover:shadow-xl transition duration-300 hover:scale-[1.01]'><img src='{{ p.image_path }}' class='w-full h-40 object-cover' alt='Uploaded Leaf'><div class='p-4'><div class='text-sm text-gray-500'>{{ p.timestamp }}</div><div class='font-bold text-lg mt-1'>{{ p.prediction }}</div><div class='text-sm text-gray-600'>{{ _('Confidence') }}: {{ p.confidence | int }}%</div><div class='text-xs text-gray-500'>{{ _('Model') }}: {{ p.model_used }}</div></div></div>{% endfor %}</div>{% else %}<p class='text-gray-500'>{{ _('No saved predictions yet.') }}</p>{% endif %}{% endblock %}"""
    , "metrics.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Training Metrics') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><p class='text-gray-700 mb-4'>{{ _('The files below are loaded from your Colab root directory.') }}</p>{% if training_results %}<h3 class='text-xl font-bold mt-4'>{{ _('Final Training Results') }}</h3><pre class='whitespace-pre-wrap bg-white/70 p-4 rounded-lg text-sm border border-gray-200 shadow-inner'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}<h3 class='text-xl font-bold mt-6'>{{ _('Confusion Matrix') }}</h3><img src='{{ url_for("static", filename="confusion_matrix.png") }}' class='max-w-full rounded-lg shadow-md mt-2 border border-gray-200' alt='Confusion Matrix'>{% if report_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Classification Report') }}</h3><pre class='bg-white/70 p-4 rounded-lg text-sm border border-gray-200 shadow-inner'>{{ report_text }}</pre>{% endif %}{% if not training_results and not confusion_exists and not report_exists %}<p class='text-gray-500'>{{ _('No training metrics files found (training_results.json, confusion_matrix.png, classification_report.txt).') }}</p>{% endif %}</div>{% endblock %}"""
    # --- New Team Template (Profile Card Grid) ---
    , "team.html": """{% extends 'layout.html' %}{% block content %}<div class='flex flex-col items-center'><h2 class='text-4xl font-extrabold mb-4 text-gray-900'>{{ _('Meet Our Team') }}</h2><p class='text-lg text-gray-600 mb-10 text-center max-w-2xl'>{{ project_summary }}</p><div class='grid md:grid-cols-4 sm:grid-cols-2 gap-8 w-full justify-center'>{% for member in team_members %}<div class='glass team-card p-6 rounded-xl shadow-lg flex flex-col items-center text-center transition duration-300 hover:scale-[1.03] hover:shadow-2xl'><div class='w-32 h-32 mb-4 bg-white rounded-full overflow-hidden border-4 border-green-300/50 flex items-center justify-center'><img src='{{ member.img }}' alt='{{ member.name }} Bitmoji' class='w-full h-full object-cover'></div><h4 class='text-xl font-bold text-gray-800'>{{ member.name }}</h4><p class='text-sm font-semibold text-green-600 mb-3'>{{ member.role }}</p><p class='text-sm text-gray-600 mt-2'>{{ _('Contribution') }}: {{ member.contributions }}</p><div class='mt-4 flex space-x-3'>{% set initials = member.name.split(' ') | map('first') | join('') %}<div class='w-6 h-6 bg-blue-500 text-white text-xs rounded-full flex items-center justify-center'>F</div><div class='w-6 h-6 bg-red-500 text-white text-xs rounded-full flex items-center justify-center'>T</div></div></div>{% endfor %}</div></div>{% endblock %}"""
}

# Write HTML templates to files
for name, content in template_content.items():
    with open(os.path.join(TEMPLATES_DIR, name), "w", encoding="utf-8") as f:
        f.write(content)

# -------------------
# 9) Start Ngrok and Flask
# -------------------
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    # Start background model loading immediately
    threading.Thread(target=_background_preload, daemon=True).start()

    print("Colab Flask app (dual-model, Tailwind UI) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("\nPaste your ngrok authtoken below. DO NOT re-run if model is still loading.")

    # Setup Ngrok
    try:
        token = getpass("ngrok authtoken (hidden): ")
        if token and token.strip():
            ngrok.set_auth_token(token.strip())
            public_url = ngrok.connect(5000)
            print(f"🔗 ngrok tunnel: {public_url}")
        else:
            print("Ngrok token not provided. App will only be accessible via local Colab link.")
    except Exception as e:
        print(f"Ngrok setup failed: {e}")

    # Run Flask
    app.run(host="0.0.0.0", port=5000, debug=False)

[model] Loading 'best' from best_crop_model.h5 ...Colab Flask app (dual-model, Tailwind UI) starting...
Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)

Paste your ngrok authtoken below. DO NOT re-run if model is still loading.

[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ALL models preloaded and ready.
ngrok authtoken (hidden): ··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:46:43] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:46:44] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:46:50] "POST /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:46:52] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:46:57] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:46:58] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:47:03] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:47:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 22:47:04] "POST /login HTTP/1.1" 302 -
INFO:werk

In [37]:
# =========================================================================
#  AI-Based Crop Disease Detection - FINAL STABLE & FIXED SCRIPT
# =========================================================================
#  -- FIXES: Dashboard SUM Error & History Template Syntax Error (FINAL FIX) --
# =========================================================================

# -------------------
# 0) Install dependencies
# -------------------
!pip install -q flask flask-login pyngrok tensorflow pillow deep-translator

# -------------------
# 1) Imports & config
# -------------------
import os, time, threading, sqlite3, json, atexit
from getpass import getpass
from datetime import datetime, timedelta
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_login import LoginManager, UserMixin, login_user, logout_user, current_user, login_required
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image, ImageDraw
from deep_translator import GoogleTranslator
from tensorflow.keras.applications.efficientnet import preprocess_input

# -------------------
# 2) Config, File System, and Constants
# -------------------
BASE_DIR = os.getcwd()
TEMPLATES_DIR = os.path.join(BASE_DIR, "templates")
STATIC_DIR = os.path.join(BASE_DIR, "static")
UPLOAD_DIR = os.path.join(STATIC_DIR, "uploads")
os.makedirs(TEMPLATES_DIR, exist_ok=True)
os.makedirs(UPLOAD_DIR, exist_ok=True)
DB_PATH = os.path.join(BASE_DIR, "database.db")
IMAGE_SIZE = (128, 128)
ALLOWED_EXT = {"png","jpg","jpeg"}
os.environ["PYNGROK_DISABLE_UPDATE_CHECK"] = "true"

# Model paths and global state
MODEL_A_PATH = "best_crop_model.h5"
MODEL_B_PATH = "crop_disease_model.h5"
MODEL_PATHS = {"best": MODEL_A_PATH, "fallback": MODEL_B_PATH}
MODEL_REGISTRY = {"best": None, "fallback": None}
ACTIVE_MODEL_KEY = "best"
MODEL_LOADING = True
SAMPLE_FILENAME = "sample_leaf_ai.png"
FARMER_HERO_IMG = "farmer_hero.png"

# -------------------
# 3) Class names, Formatting, and Languages
# -------------------
CLASS_NAMES = [
    'Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot',
    'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold',
    'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite',
    'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus',
    'Tomato__Tomato_mosaic_virus', 'Tomato_healthy'
]
CLEAN_NAMES = [format_class_name(n) for n in CLASS_NAMES]
def format_class_name(name): return name.replace('___', ': ').replace('__',' ').replace('_',' ')

LANGUAGES = {
    'en':"English", 'hi':"हिन्दी", 'ur':"اردو", 'bn':"বাংলা",
    'te':"తెలుగు", 'ta':"தமிழ்", 'mr':"मराठी", 'gu':"ગુજરાતી",
    'pa':"ਪੰਜਾਬੀ", 'ml':"മലയാളം", 'kn':"ಕನ್ನಡ"
}

# -------------------
# 4) Database Setup
# -------------------
if os.path.exists(DB_PATH): os.remove(DB_PATH)

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute('''CREATE TABLE IF NOT EXISTS users (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     username TEXT UNIQUE NOT NULL,
                     password_hash TEXT NOT NULL)''')
    cur.execute('''CREATE TABLE IF NOT EXISTS predictions (
                     id INTEGER PRIMARY KEY AUTOINCREMENT,
                     user_id INTEGER NOT NULL,
                     image_path TEXT NOT NULL,
                     model_used TEXT NOT NULL,
                     prediction TEXT NOT NULL,
                     confidence REAL NOT NULL,
                     time TEXT NOT NULL,
                     FOREIGN KEY (user_id) REFERENCES users(id))''')
    conn.commit(); conn.close()
init_db()

# Sample image generation for UI and Quick Analysis feature
sample_img = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
if not os.path.exists(sample_img):
    img = Image.new("RGB", (400,260), (210,240,200))
    d = ImageDraw.Draw(img)
    d.text((30,30), "AgroScan Sample Leaf", fill=(20,90,30))
    img.save(sample_img)

# Placeholder image for Farmer Hero section
if not os.path.exists(os.path.join(UPLOAD_DIR, FARMER_HERO_IMG)):
    img = Image.new("RGB", (1000, 600), (45, 128, 100))
    d = ImageDraw.Draw(img)
    d.text((50, 50), "Upload farmer_hero.png here", fill=(255, 255, 255))
    img.save(os.path.join(UPLOAD_DIR, FARMER_HERO_IMG))

# Placeholder Bitmoji paths (User must upload these)
def get_bitmoji_url(name):
    filename = f"{name.lower().split()[0]}.png"
    if not os.path.exists(os.path.join(UPLOAD_DIR, filename)):
        return url_for('static', filename='uploads/sample_leaf_ai.png')
    return url_for('static', filename=f'uploads/{filename}')


# -------------------
# 5) Disease Information
# -------------------
disease_info = {
    'Pepper__bell___Bacterial_spot': {'description':'A common bacterial disease that causes dark, water-soaked spots on leaves and fruit, leading to reduced yield and quality.', 'remedies':['Apply copper-based sprays','Improve air circulation','Remove infected debris']},
    'Pepper__bell___healthy': {'description':'The plant appears to be in excellent health. Leaves are well-formed without signs of pests or disease.','remedies':[]},
    'Potato___Early_blight': {'description':'A fungal disease causing dark, concentric rings on leaves, often described as "target spots". It typically affects lower, older leaves first.', 'remedies':['Use certified seed','Apply fungicides containing mancozeb or chlorothalonil','Rotate crops']},
    'Potato___Late_blight': {'description':'A devastating fungal-like disease that causes large, dark lesions on leaves and stems, often with a white moldy growth on the underside.', 'remedies':['Remove infected plants','Protective fungicides proactively','Ensure good drainage and air circulation']},
    'Potato___healthy': {'description':'The potato plant looks healthy and vigorous. No signs of blight or other common diseases are visible.','remedies':[]},
    'Tomato_Bacterial_spot': {'description':'Causes small, water-soaked spots on tomato leaves and fruit.','remedies':['Avoid working when wet','Use copper-based bactericides','Mulch around plants']},
    'Tomato_Early_blight': {'description':'Fungal disease, creating "target spot" lesions on lower leaves, which can lead to defoliation.', 'remedies':['Ensure proper spacing','Apply preventative fungicides','Water at the base of the plant']},
    'Tomato_Late_blight': {'description':'A serious fungal disease characterized by large, greasy, grey-green spots on leaves.','remedies':['Apply preventative fungicides like chlorothalonil or copper','Remove and destroy infected plants immediately']},
    'Tomato_Leaf_Mold': {'description':'Fungal disease causing pale spots on the upper leaf surface and a velvety, olive-green mold on the underside.','remedies':['Improve air circulation and reduce humidity','Prune lower leaves']},
    'Tomato_Septoria_leaf_spot': {'description':'A fungal disease that appears as numerous small, circular spots with dark borders and tan centers on older, lower leaves.','remedies':['Remove infected leaves','Improve air circulation','Apply fungicides']},
    'Tomato_Spider_mites_Two_spotted_spider_mite': {'description':'Tiny pests causing stippling (tiny yellow or white dots) on leaves, leading to webbing.', 'remedies':['Spray plants with water','Apply insecticidal soap or horticultural oil','Introduce natural predators']},
    'Tomato__Target_Spot': {'description':'A fungal disease causing necrotic spots with concentric rings.', 'remedies':['Apply fungicides effective against Corynespora cassiicola','Improve air circulation and reduce leaf wetness']},
    'Tomato__Tomato_YellowLeaf__Curl_Virus': {'description':'A viral disease transmitted by whiteflies. Symptoms include severe stunting, upward curling of leaves, and yellowing of leaf margins.', 'remedies':['Control whitefly populations','Remove and destroy infected plants immediately','Use virus-resistant varieties']},
    'Tomato__Tomato_mosaic_virus': {'description':'A viral disease that causes mottled light and dark green patterns on leaves, along with stunting and malformation.', 'remedies':['There is no cure; remove and destroy infected plants','Wash hands and tools thoroughly']},
    'Tomato_healthy': {'description':'The tomato plant is healthy, showing vibrant green leaves and no signs of spots, pests, or viral infection.','remedies':[]},
    'default': {'description':'Information for this condition is not available.','remedies':['Consult local extension services']}
}

# -------------------
# 6) Model Management & Translation Setup
# -------------------
translation_cache = {}
def get_translation(text, target_language):
    if not text or target_language == 'en': return text
    key = (text, target_language)
    if key in translation_cache: return translation_cache[key]
    try:
        out = GoogleTranslator(source='auto', target=target_language).translate(text)
        translation_cache[key] = out; return out
    except Exception: return text

# Route-safe translation helper
def tr_route(text):
    lang = session.get('language', 'en')
    return get_translation(text, lang)

app = Flask(__name__, template_folder=TEMPLATES_DIR, static_folder=STATIC_DIR)
app.secret_key = os.environ.get("FLASK_SECRET", os.urandom(24))
login_manager = LoginManager(); login_manager.init_app(app); login_manager.login_view = 'login'
class User(UserMixin): pass
@login_manager.user_loader
def load_user(user_id):
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT id, username FROM users WHERE id=?", (user_id,))
    r = cur.fetchone(); conn.close()
    if r:
        u = User(); u.id = r[0]; u.username = r[1]; return u
    return None

@app.context_processor
def inject_globals():
    def tr(text):
        return tr_route(text)
    return dict(_=tr, languages=LANGUAGES, ACTIVE_MODEL_KEY=ACTIVE_MODEL_KEY)

def load_model_lazy(key):
    path = MODEL_PATHS.get(key)
    if not path or not os.path.exists(path):
        print(f"[model] {key} path missing: {path}")
        return None
    if MODEL_REGISTRY.get(key) is None:
        try:
            print(f"[model] Loading '{key}' from {path} ...")
            MODEL_REGISTRY[key] = tf.keras.models.load_model(path, compile=False)
            print(f"[model] Loaded '{key}'.")
        except Exception as e:
            print(f"[model] Failed to load {key}: {e}")
            MODEL_REGISTRY[key] = None
    return MODEL_REGISTRY[key]

def _background_preload():
    global MODEL_LOADING
    MODEL_LOADING = True
    for k in ["best", "fallback"]:
        load_model_lazy(k)
    MODEL_LOADING = False
    print("ALL models preloaded and ready.")

# -------------------
# 7) Routes
# -------------------
@app.route('/language/<lang>')
def set_language(lang):
    if lang in LANGUAGES:
        session['language']=lang
    return redirect(request.referrer or url_for('home'))

@app.route("/")
def home():
    if not current_user.is_authenticated:
        return redirect(url_for('login'))
    return render_template("home_logged_in.html")

@app.route("/login", methods=["GET","POST"])
def login():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        cur.execute("SELECT id, username, password_hash FROM users WHERE username=?", (username,))
        row = cur.fetchone(); conn.close()
        if row and check_password_hash(row[2], password):
            u = User(); u.id = row[0]; u.username = row[1]; login_user(u)
            return redirect(url_for("home"))
        flash(tr_route("Invalid username or password."))
    return render_template("login.html")

@app.route("/register", methods=["GET","POST"])
def register():
    if current_user.is_authenticated: return redirect(url_for('home'))
    if request.method=="POST":
        username = request.form["username"].strip()
        password = request.form["password"]
        conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
        try:
            cur.execute("INSERT INTO users (username, password_hash) VALUES (?,?)", (username, generate_password_hash(password)))
            conn.commit(); conn.close(); flash(tr_route("Registration successful! Please login.")); return redirect(url_for('login'))
        except sqlite3.IntegrityError:
            conn.close(); flash(tr_route("Username already taken."))
    return render_template("register.html")

@app.route("/logout")
@login_required
def logout():
    logout_user(); return redirect(url_for("home"))

@app.route("/dashboard")
@login_required
def dashboard():
    global ACTIVE_MODEL_KEY
    conn = sqlite3.connect(DB_PATH); cur=conn.cursor()

    # Chart 1: Prediction Distribution (Pie Chart Data)
    cur.execute("SELECT prediction, COUNT(*) FROM predictions WHERE user_id=? GROUP BY prediction", (current_user.id,))
    disease_rows = cur.fetchall()
    disease_data = {'labels':[r[0] for r in disease_rows], 'values':[r[1] for r in disease_rows]}

    # Chart 2: Activity Over Time (Bar Chart Data)
    cur.execute("SELECT strftime('%Y-%m', time) as month, COUNT(*) as count FROM predictions WHERE user_id = ? GROUP BY month ORDER BY month", (current_user.id,))
    monthly_rows = cur.fetchall()
    monthly_data = {
        "labels": [datetime.strptime(row[0], '%Y-%m').strftime('%b %Y') for row in monthly_rows],
        "values": [row[1] for row in monthly_rows]
    }

    conn.close()
    return render_template("dashboard.html", disease_data=disease_data, monthly_data=monthly_data, active_model=ACTIVE_MODEL_KEY)

@app.route('/switch_model', methods=['POST'])
@login_required
def switch_model():
    global ACTIVE_MODEL_KEY
    m = request.form.get('model','best')
    if m in MODEL_PATHS:
        ACTIVE_MODEL_KEY = m
        load_model_lazy(m)
        flash(tr_route(f"Active model switched to: {m}"))
    else:
        flash(tr_route("Unknown model key"))
    return redirect(url_for('dashboard'))

@app.route("/history")
@login_required
def history():
    conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
    cur.execute("SELECT image_path, prediction, confidence, time, model_used FROM predictions WHERE user_id=? ORDER BY time DESC",(current_user.id,))
    rows = cur.fetchall(); conn.close()
    hist=[]
    ist_offset = timedelta(hours=5, minutes=30)
    for img_url,pred,conf,ts,model_used in rows:
        try: ts_dt = datetime.fromisoformat(ts)
        except: ts_dt = datetime.strptime(ts.split('.')[0], "%Y-%m-%d %H:%M:%S")
        ist = ts_dt + ist_offset
        hist.append({
            "image_path":img_url,"prediction":pred,
            "confidence":int(round(conf)), "timestamp":ist.strftime("%b %d, %Y %I:%M %p"),
            "model_used": model_used
        })
    return render_template("history.html", history=hist)

@app.route("/metrics")
@login_required
def metrics():
    training_results = None; report_text=None
    confusion_exists = os.path.exists("confusion_matrix.png")
    report_exists = os.path.exists("classification_report.txt")
    if os.path.exists("training_results.json"):
        with open("training_results.json",'r') as f: training_results = json.load(f)
    if report_exists:
        with open("classification_report.txt",'r') as f: report_text=f.read()
    return render_template('metrics.html', training_results=training_results, confusion_exists=confusion_exists, report_exists=report_exists, report_text=report_text)

@app.route("/team")
def team():
    # Define the team data to be passed to the template
    team_members = [
        {"name": "Umar Iqbal", "role": "Project Leader & Lead Developer", "contributions": tr_route("Led the entire project lifecycle, developed the core prediction pipeline, implemented the Flask backend, and designed the database structure."), "img": get_bitmoji_url("Umar")},
        {"name": "Shruti Bajpai", "role": "Research and Documentation Lead", "contributions": tr_route("Authored the research paper, handled documentation, and assisted with model evaluation metrics."), "img": get_bitmoji_url("Shruti")},
        {"name": "Sudhanshu Tiwari", "role": "Presentation and Support", "contributions": tr_route("Developed the project presentation slides, assisted in data preprocessing steps, and ensured overall project coherence."), "img": get_bitmoji_url("Sudhanshu")},
        {"name": "Vaishnavi Singh", "role": "Presentation and Support", "contributions": tr_route("Contributed to the project presentation design, provided general technical support, and assisted with final testing and deployment."), "img": get_bitmoji_url("Vaishnavi")}
    ]
    project_summary = tr_route("This project, AgroScan AI, uses a deep learning model (EfficientNetB0) trained on the PlantVillage dataset to instantly diagnose common crop diseases in potatoes, tomatoes, and bell peppers. The goal is to provide farmers with fast, actionable, and language-accessible intelligence to protect their yield.")

    return render_template('team.html', team_members=team_members, project_summary=project_summary, title='Our Team')

def allowed_file(filename): return '.' in filename and filename.rsplit('.',1)[1].lower() in ALLOWED_EXT

@app.route("/predict", methods=["GET", "POST"])
@login_required
def predict_page():
    global MODEL_LOADING, ACTIVE_MODEL_KEY
    model = MODEL_REGISTRY.get(ACTIVE_MODEL_KEY)

    last_uploaded_image_url = session.get('last_uploaded_image_url')

    if model is None and MODEL_LOADING:
        return render_template("predict.html", model_loading=True, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)
    if model is None:
        flash(tr_route(f"Model {ACTIVE_MODEL_KEY} failed to load. Please check files and re-run."))
        return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    if request.method == "POST":
        file_to_analyze = None
        current_image_url = None

        source_type = request.form.get('source', 'upload')

        if source_type == 'reuse' and last_uploaded_image_url:
            filename = os.path.basename(last_uploaded_image_url).split('?')[0]
            file_to_analyze = os.path.join(UPLOAD_DIR, filename)
            current_image_url = last_uploaded_image_url
            if not os.path.exists(file_to_analyze):
                flash(tr_route("Last uploaded image not found on server."))
                session.pop('last_uploaded_image_url', None)
                return redirect(url_for('predict_page'))

        elif source_type == 'sample':
            file_to_analyze = os.path.join(UPLOAD_DIR, SAMPLE_FILENAME)
            current_image_url = url_for('static', filename=f'uploads/{SAMPLE_FILENAME}')

        elif source_type == 'upload':
            if "file" not in request.files or request.files["file"].filename == "":
                flash(tr_route("No image uploaded."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            file = request.files["file"]
            if not allowed_file(file.filename):
                flash(tr_route("Invalid file type. Please upload JPG or PNG."))
                return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

            fname = secure_filename(file.filename)
            unique_fname = f"{int(time.time())}_{fname}"
            file_to_analyze = os.path.join(UPLOAD_DIR, unique_fname)
            file.save(file_to_analyze)
            current_image_url = url_for('static', filename=f'uploads/{unique_fname}')

        else:
             flash(tr_route("Invalid analysis source."))
             return redirect(url_for('predict_page'))


        try:
            # === CRITICAL EFFICIENTNET PREPROCESSING ===
            img = Image.open(file_to_analyze).convert("RGB").resize(IMAGE_SIZE)
            arr = np.array(img).astype("float32")
            arr = np.expand_dims(arr, 0)
            processed_arr = preprocess_input(arr)

            # Prediction
            preds = model.predict(processed_arr)
            idx = int(np.argmax(preds))
            conf = float(np.max(preds)*100)
            raw_name = CLASS_NAMES[idx]
            pred_name = format_class_name(raw_name)

            # Update session with the current image path
            session['last_uploaded_image_url'] = current_image_url

            # Save to user history
            timestamp = datetime.utcnow().isoformat()
            conn = sqlite3.connect(DB_PATH); cur = conn.cursor()
            cur.execute("INSERT INTO predictions (user_id, image_path, prediction, confidence, time, model_used) VALUES (?,?,?,?,?,?)",
                        (current_user.id, current_image_url, pred_name, conf, timestamp, ACTIVE_MODEL_KEY))
            conn.commit(); conn.close()

            # Prepare result data (including translation)
            info = disease_info.get(raw_name, disease_info['default'])
            lang = session.get('language','en')
            desc = get_translation(info.get('description', ''), lang)
            remedies = [get_translation(r, lang) for r in info.get('remedies', [])]

            if conf >= 85: conf_level = 'High'
            elif conf >= 60: conf_level = 'Medium'
            else: conf_level = 'Low'

            return render_template("result.html", prediction=pred_name, confidence=round(conf,2),
                                    image_path=current_image_url, description=desc, remedies=remedies,
                                    confidence_level=conf_level)
        except Exception as e:
            flash(tr_route(f"Prediction error: {e}"))
            return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

    return render_template("predict.html", model_loading=False, last_uploaded_image_url=last_uploaded_image_url, sample_filename=SAMPLE_FILENAME)

# -------------------
# 8) Tailwind CSS Templates (ULTIMATE OVERHAUL)
# -------------------
template_content = {
    # --- Layout Template ---
    "layout.html": """<!doctype html><html lang='{{ session.get("language", "en") }}'><head><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1"><script src="https://cdn.tailwindcss.com"></script><script src="https://cdn.jsdelivr.net/gh/alpinejs/alpine@v2.x.x/dist/alpine.min.js" defer></script><title>{{ _(title or 'AgroScan AI') }}</title><style>
        body { min-height: 100vh; background-image: linear-gradient(to top right, #dcfce7, #ecfeff); }
        main { flex: 1; }
        .glass { background-color: rgba(255, 255, 255, 0.3); backdrop-filter: blur(10px); border: 1px solid rgba(255, 255, 255, 0.2); box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1); border-radius: 1rem; transition: all 0.3s ease-in-out;}
        .glass-nav { background-color: rgba(255, 255, 255, 0.6); backdrop-filter: blur(8px); }
    </style></head><body class="text-gray-800 flex flex-col"><nav class="glass-nav fixed w-full z-30 shadow-md"><div class="max-w-6xl mx-auto px-4 py-3 flex items-center justify-between"><a href="{{ url_for('home') }}" class="flex items-center space-x-3"><svg class="h-8 w-8 text-green-600" viewBox="0 0 24 24" fill="currentColor"><path d="M12 2C6.48 2 2 6.48 2 12s4.48 10 10 10 10-4.48 10-10S17.52 2 12 2zm1 17.93c-2.82-.44-5.02-2.31-6.12-4.94L13 14h-2c-2.76 0-5 2.24-5 5H4c0-3.92 2.68-7.23 6.33-8.15C10.12 11.23 11.06 12 12 12s1.88-.77 2.67-1.15c.16-.08.31-.17.46-.26 3.65.92 6.33 4.23 6.33 8.15h-2c0-2.76-2.24-5-5-5h-2l6.12 4.94z"/></svg><span class="font-bold text-xl">AgroScan AI</span></a><div class="flex items-center space-x-4"><div class="hidden md:flex items-center space-x-4">{% if current_user.is_authenticated %}<span class="text-gray-700">{{ _('Welcome') }}, {{ current_user.username }}!</span><a href='{{ url_for("dashboard") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Dashboard') }}</a><a href='{{ url_for("predict_page") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Analyze') }}</a><a href='{{ url_for("history") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('History') }}</a><a href='{{ url_for("metrics") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Metrics') }}</a><a href='{{ url_for("team") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Team') }}</a><a href='{{ url_for("logout") }}' class='text-blue-600 font-semibold hover:text-blue-800 transition duration-150'>{{ _('Logout') }}</a>{% else %}<a href='{{ url_for("login") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Login') }}</a><a href='{{ url_for("register") }}' class='text-gray-600 hover:text-green-600 transition duration-150'>{{ _('Register') }}</a>{% endif %}</div><div x-data="{ open: false }" class="relative"><button @click="open = !open" class="text-gray-600 hover:text-green-600 flex items-center p-2 rounded-lg hover:bg-gray-100 transition duration-150"><svg class="w-5 h-5 mr-1" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M7.75 2.75a.75.75 0 00-1.5 0v1.258a32.987 32.987 0 00-3.599.278.75.75 0 10.198 1.487A31.545 31.545 0 018.7 5.545 19.38 19.38 0 017 9.762V11.5a.75.75 0 001.5 0V9.762c0-.683.085-1.35.25-1.994.169-.65.4-1.28.68-1.86A7.94 7.94 0 0112 6.561v4.939a.75.75 0 101.5 0V6.561a7.94 7.94 0 012.07 1.407c.28.58.512 1.21.68 1.86.165.643.25 1.311.25 1.994V11.5a.75.75 0 001.5 0V9.762a19.38 19.38 0 01-1.7 4.217 31.545 31.545 0 014.101-.987.75.75 0 00.198-1.487 32.987 32.987 0 00-3.599-.278V2.75a.75.75 0 00-1.5 0v1.439a7.94 7.94 0 01-3.12 1.548A7.94 7.94 0 019.25 4.19V2.75zM8.5 13.25a.75.75 0 00-1.5 0v2a.75.75 0 001.5 0v-2z" /></svg>{{ languages[session.get('language', 'en')] or 'Language' }}</button><div x-show="open" @click.away="open = false" class="absolute right-0 mt-2 w-36 bg-white border border-gray-200 rounded-md shadow-lg z-50 overflow-hidden" style="display: none;">{% for lang_code, lang_name in languages.items() %}<a href='{{ url_for("set_language", lang=lang_code) }}' class="block px-4 py-2 text-sm text-gray-700 hover:bg-gray-100 transition duration-100">{{ lang_name }}</a>{% endfor %}</div></div></div></div></nav><main class='pt-20 max-w-6xl mx-auto p-6 flex-grow'>{% with messages = get_flashed_messages() %}{% if messages %}<div class='mb-4 p-3 bg-red-100 text-red-700 rounded'>{{ messages[0] }}</div>{% endif %}{% endwith %}{% block content %}{% endblock %}</main><footer class='max-w-6xl mx-auto text-center text-gray-500 py-6 border-t mt-auto w-full'>© 2025 AgroScan AI - {{ _('Model') }}: {{ ACTIVE_MODEL_KEY or 'N/A' }}</footer></body></html>"""

    # --- Home Page (Logged OUT) ---
    , "home.html": """{% extends 'layout.html' %}{% block content %}<div class='grid md:grid-cols-2 gap-12 items-center glass p-10 rounded-xl'><div class='order-2 md:order-1'><h1 class='text-5xl md:text-6xl font-extrabold text-gray-900 leading-tight'>{{ _('Empower Your Harvest with Intelligent Vision') }}</h1><p class='mt-4 text-xl text-gray-700 font-medium'>{{ _('Instantly detect crop diseases with a single photo. Get actionable advice to protect your yield and secure your future.') }}</p><div class='mt-8 flex space-x-4'><a href='{{ url_for("login") }}' class='bg-green-600 hover:bg-green-700 text-white px-8 py-4 rounded-xl font-bold transition duration-200 shadow-xl hover:scale-[1.03] text-lg'>{{ _('Get Started Now') }}</a><a href='{{ url_for("register") }}' class='bg-white hover:bg-gray-100 text-gray-700 px-8 py-4 rounded-xl font-bold transition duration-200 shadow-xl hover:scale-[1.03] text-lg'>{{ _('Register Free') }}</a></div></div><div class='order-1 md:order-2'><div class="relative w-full h-96 overflow-hidden rounded-xl shadow-2xl"><img src='{{ url_for("static", filename="uploads/farmer_hero.png") }}' class='w-full h-full object-cover transition duration-500 hover:opacity-90' alt='Farmer with crop'><div class="absolute inset-0 bg-black/20"></div></div></div></div>{% endblock %}"""

    # --- Home Page (Logged IN - Quick Guide) ---
    , "home_logged_in.html": """{% extends 'layout.html' %}{% block content %}<div class='glass p-10 rounded-xl mb-8 transition duration-300 shadow-xl'><h1 class='text-4xl md:text-5xl font-extrabold text-gray-900 mb-2'>{{ _('Welcome Back,') }} <span class='text-green-600'>{{ current_user.username }}!</span></h1><p class='text-lg text-gray-700'>{{ _('Your centralized crop health management dashboard.') }}</p></div><div class='grid lg:grid-cols-3 gap-6'><div class='lg:col-span-2 glass p-6 rounded-xl space-y-6 transition duration-300 shadow-xl'><h3 class='text-2xl font-bold text-gray-800 mb-4'>{{ _('What to do next?') }}</h3><div class='grid sm:grid-cols-3 gap-4'><a href='{{ url_for("predict_page") }}' class='bg-green-600 hover:bg-green-700 text-white p-5 rounded-xl font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>🔍</div>{{ _('Start Analysis') }}</a><a href='{{ url_for("dashboard") }}' class='bg-blue-600 hover:bg-blue-700 text-white p-5 rounded-xl font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>📊</div>{{ _('Check Dashboard') }}</a><a href='{{ url_for("history") }}' class='bg-gray-600 hover:bg-gray-700 text-white p-5 rounded-xl font-bold text-center shadow-lg transition duration-300 hover:scale-[1.03]'><div class='text-3xl mb-1'>📜</div>{{ _('View History') }}</a></div><h3 class='text-2xl font-bold text-gray-800 mt-8 mb-3 border-t pt-4'>{{ _('Quick Guide') }}</h3><div class='space-y-4'><div class='p-4 bg-white/70 rounded-lg border border-green-200 shadow-md'><h4 class='font-bold'>1. {{ _('Analyze') }} ({{ url_for("predict_page") }})</h4><p class='text-sm text-gray-700'>{{ _('Upload a clear image of a symptomatic plant leaf. Our AI will return a diagnosis and suggested remedies.') }}</p></div><div class='p-4 bg-white/70 rounded-lg border border-blue-200 shadow-md'><h4 class='font-bold'>2. {{ _('Dashboard') }} ({{ url_for("dashboard") }})</h4><p class='text-sm text-gray-700'>{{ _('Review your past activity and see the distribution of diseases you have detected over time.') }}</p></div><div class='p-4 bg-white/70 rounded-lg border border-gray-200 shadow-md'><h4 class='font-bold'>3. {{ _('Metrics') }} ({{ url_for("metrics") }})</h4><p class='text-sm text-gray-700'>{{ _('View the scientific performance data (Confusion Matrix, Classification Report) of the core AI model.') }}</p></div></div></div><div class='lg:col-span-1 glass p-6 rounded-xl transition duration-300 shadow-xl'><h3 class='text-2xl font-bold text-gray-800 mb-4'>{{ _('System Status') }}</h3><div class='space-y-3'><div class='p-3 bg-white/70 rounded-lg border border-green-200'><span class='font-semibold'>{{ _('Active Model') }}:</span> {{ ACTIVE_MODEL_KEY }}</div><div class='p-3 bg-white/70 rounded-lg border border-blue-200'><span class='font-semibold'>{{ _('Input Size') }}:</span> 128x128 pixels</div><div class='p-3 bg-white/70 rounded-lg border border-gray-200'><span class='font-semibold'>{{ _('Language') }}:</span> {{ languages[session.get('language', 'en')] or 'English' }}</div><a href='{{ url_for("predict_page") }}' class='w-full block text-center bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold shadow-md mt-6 transition duration-300 hover:scale-[1.03]'>{{ _('Jump to Analysis') }}</a></div></div></div>{% endblock %}"""

    # --- Login/Register Templates (Aesthetic Consistency) ---
    , "login.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Login') }}</h2><form method='post' action='{{ url_for("login") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Sign in') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Not a member?') }} <a href='{{ url_for("register") }}' class='text-green-600 hover:text-green-700'>{{ _('Register here') }}</a></p></div>{% endblock %}"""
    , "register.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-md mx-auto glass p-8 rounded-xl shadow-xl mt-12'><h2 class='text-3xl font-bold text-center mb-6'>{{ _('Register') }}</h2><form method='post' action='{{ url_for("register") }}'><label class='block mb-4'>{{ _('Username') }}<input name='username' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><label class='block mb-4'>{{ _('Password') }}<input name='password' type='password' class='w-full p-3 border border-gray-300 rounded-lg mt-1 focus:border-green-500 focus:ring-green-500 bg-white/70 transition duration-150' required></label><button class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Register') }}</button></form><p class='mt-4 text-center text-sm text-gray-600'>{{ _('Already a member?') }} <a href='{{ url_for("login") }}' class='text-green-600 hover:text-green-700'>{{ _('Login here') }}</a></p></div>{% endblock %}"""

    # --- Predict Template (Quick Analysis retained) ---
    , "predict.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-3xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Predict Disease') }}</h2>{% if model_loading %}<div class='text-center p-8 bg-yellow-50 rounded-lg'><div class='animate-spin w-8 h-8 border-4 border-yellow-600 border-t-transparent rounded-full mx-auto'></div><p class='mt-4 text-lg text-yellow-800 font-semibold'>{{ _('Model is loading...') }}</p><p class='text-sm text-yellow-700'>{{ _('This may take up to 60 seconds. Please wait on this page.') }}</p></div>{% else %}<form id='upload-form' method='post' action='{{ url_for("predict_page") }}' enctype='multipart/form-data'><input type="hidden" name="source" id="source-input" value="upload"><label for='file-upload' class='block border-2 border-dashed border-gray-300 p-10 rounded-xl cursor-pointer hover:border-green-500 transition duration-200'><input id='file-upload' name='file' type='file' accept='image/jpeg, image/png, image/jpg' class='hidden' onchange="document.getElementById('source-input').value = 'upload'; document.getElementById('file-info').textContent = 'File: ' + this.files[0].name;"><div class='text-center'><svg class='mx-auto w-12 h-12 text-gray-400' fill='none' stroke='currentColor' viewBox='0 0 24 24'><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M7 16a4 4 0 01-4-4V7a4 4 0 014-4h10a4 4 0 014 4v5a4 4 0 01-4 4h-3'></path><path stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M16 12l-4-4m0 0L8 12m4-4v11'></path></svg><p class='mt-2 text-gray-600 font-semibold'>{{ _('Click to upload leaf image') }}</p><p id='file-info' class='mt-1 text-sm text-gray-500'>{{ _('JPG or PNG recommended.') }}</p></div></label><div class='mt-6'><button id='submit-button' class='w-full bg-green-600 hover:bg-green-700 text-white p-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.01]'>{{ _('Detect Disease') }}</button><div id='loader' class='hidden w-full flex items-center justify-center p-3 bg-gray-100 rounded-lg mt-6'><div class='animate-spin w-6 h-6 border-4 border-green-600 border-t-transparent rounded-full mr-3'></div><span>{{ _('Analyzing...') }}</span></div></div></form><div class="mt-8 pt-6 border-t border-gray-200"><h3 class="text-xl font-bold mb-4">{{ _('Quick Analysis') }}</h3><div class="space-y-3">{% if last_uploaded_image_url %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="reuse"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-300 border border-gray-200 shadow-sm hover:scale-[1.01]"><svg class="w-5 h-5 mr-3 text-blue-600" fill="currentColor" viewBox="0 0 20 20"><path d="M10 12a2 2 0 100-4 2 2 0 000 4z"/><path fill-rule="evenodd" d="M.458 10C1.732 5.943 5.522 3 10 3s8.268 2.943 9.542 7c-1.274 4.057-5.064 7-9.542 7S1.732 14.057.458 10zM14 10a4 4 0 11-8 0 4 4 0 018 0z" clip-rule="evenodd"/></svg>{{ _('Reuse last uploaded image') }}</button></form>{% endif %}<form method="post" action="{{ url_for('predict_page') }}"><input type="hidden" name="source" value="sample"><button type="submit" class="w-full text-left bg-white/70 hover:bg-white/90 p-3 rounded-lg flex items-center transition duration-300 border border-gray-200 shadow-sm hover:scale-[1.01]"><svg class="w-5 h-5 mr-3 text-green-600" fill="currentColor" viewBox="0 0 20 20"><path fill-rule="evenodd" d="M16.707 5.293a1 1 0 010 1.414l-8 8a1 1 0 01-1.414 0l-4-4a1 1 0 011.414-1.414L8 12.586l7.293-7.293a1 1 0 011.414 0z" clip-rule="evenodd"/></svg>{{ _('Use internal sample image') }}</button></form></div></div></div><script>const form=document.getElementById('upload-form'), submitButton=document.getElementById('submit-button'), loader=document.getElementById('loader'); form.addEventListener('submit', e=>{ submitButton.classList.add('hidden'); loader.classList.remove('hidden'); });</script>{% endif %}{% endblock %}"""

    , "result.html": """{% extends 'layout.html' %}{% block content %}<div class='max-w-4xl mx-auto glass p-8 rounded-xl shadow-xl mt-4'><h2 class='text-3xl font-bold mb-6 text-center'>{{ _('Detection Result') }}</h2><div class='grid md:grid-cols-2 gap-8 items-start'><div class='md:w-full'><img src='{{ image_path }}' class='rounded-lg w-full shadow-2xl border border-gray-200 transition duration-300 hover:scale-[1.01]' alt='Uploaded Leaf'></div><div class='md:w-full space-y-5'><div class='glass p-6 rounded-xl border-l-4 border-green-600 shadow-xl transition duration-300 hover:shadow-2xl'><p class='text-lg text-gray-700 mb-1'>{{ _('Predicted Condition') }}:</p><h3 class='text-3xl font-extrabold text-green-700'>{{ prediction }}</h3><p class='mt-3 text-gray-700'>{{ _('Confidence') }}: <strong>{{ confidence }}%</strong></p><div class='w-full bg-gray-200 rounded-full h-4 mt-3 overflow-hidden' title='{{ confidence }}%'><div class='h-4 rounded-full' style='width: {{ confidence }}%; {% if confidence_level == 'High' %} background-color: #059669; {% elif confidence_level == 'Medium' %} background-color: #f59e0b; {% else %} background-color: #ef4444; {% endif %}'></div></div></div><div class='glass p-6 rounded-xl shadow-xl transition duration-300 hover:shadow-2xl'><h4 class='font-bold text-xl mb-2 text-blue-600'>{{ _('About this Condition') }}</h4><p class='text-gray-700'>{{ description }}</p></div>{% if remedies %}<div class='glass p-6 rounded-xl shadow-xl transition duration-300 hover:shadow-2xl'><h4 class='font-bold text-xl mb-2 text-green-600'>{{ _('Suggested Actions') }}</h4><ul class='list-disc pl-5 text-gray-700 space-y-1'>{% for r in remedies %}<li>{{ r }}</li>{% endfor %}</ul></div>{% endif %}</div></div><div class='mt-8 text-center'><a href='{{ url_for("predict_page") }}' class='bg-gray-600 hover:bg-gray-700 text-white px-6 py-3 rounded-lg font-bold transition duration-300 shadow-xl hover:scale-[1.03]'>{{ _('Analyze Another Image') }}</a></div></div>{% endblock %}"""

    # --- Dashboard Template (Redesigned Grid Cards) ---
    , "dashboard.html": """{% extends 'layout.html' %}{% block content %}<script src="https://cdn.jsdelivr.net/npm/chart.js@3.7.1/dist/chart.min.js"></script><h2 class='text-3xl font-bold mb-6'>{{ _('Dashboard') }}</h2><div class='glass p-6 rounded-xl shadow-xl transition duration-300'><h3 class='text-2xl font-bold mb-4'>{{ _('Progress and Analysis Overview') }}</h3><div class='grid lg:grid-cols-3 gap-6'><div class='p-5 bg-white/90 rounded-xl shadow-lg border border-gray-200 transition duration-300 hover:scale-[1.01]'><h4 class='font-bold mb-3 text-center text-lg text-blue-600'>{{ _('Detection Distribution') }}</h4><div class='h-64'><canvas id='diseaseChart'></canvas></div></div><div class='p-5 bg-white/90 rounded-xl shadow-lg border border-gray-200 transition duration-300 hover:scale-[1.01]'><h4 class='font-bold mb-3 text-center text-lg text-green-600'>{{ _('Activity Over Time') }}</h4><div class='h-64'><canvas id='monthlyChart'></canvas></div></div><div class='glass p-5 rounded-xl shadow-lg transition duration-300 hover:scale-[1.01] border border-gray-200'><h4 class='font-bold mb-3 text-lg text-gray-800'>{{ _('Model Configuration') }}</h4><form method='post' action='{{ url_for("switch_model") }}'><label class='block text-gray-700'>{{ _('Active Model') }}:<select name='model' class='mt-2 p-2 border border-gray-300 rounded-lg w-full bg-white/70'> <option value='best' {% if active_model=='best' %}selected{% endif %}>{{ _('Best model') }} (best_crop_model.h5)</option><option value='fallback' {% if active_model=='fallback' %}selected{% endif %}>{{ _('Fallback model') }} (crop_disease_model.h5)</option></select></label><div class='mt-4'><button class='w-full bg-blue-600 hover:bg-blue-700 text-white px-4 py-2 rounded-lg font-bold transition duration-300 shadow-md'>{{ _('Switch Model') }}</button></div></form><h4 class='font-bold mt-4 pt-3 border-t text-lg text-gray-800'>{{ _('Total Scans') }}: {{ disease_data.values | sum(start=0) }}</h4></div></div><h3 class='text-xl font-bold mt-8 mb-4 border-t pt-4'>{{ _('Detailed Prediction Counts') }}</h3>{% if disease_data.labels %}<ul class='space-y-2'>{% for i in range(disease_data.labels|length) %}<li class='flex justify-between p-3 bg-white/70 rounded-lg border border-gray-200 shadow-sm hover:bg-white/90 transition duration-150'><span class='font-semibold'>{{ disease_data.labels[i] }}</span><span>{{ disease_data.values[i] }} {{ _('times detected') }}</span></li>{% endfor %}</ul>{% else %}<p class='text-gray-500'>{{ _('No detections yet. Analyze to populate your dashboard.') }}</p>{% endif %}</div><script>const diseaseData={{ disease_data | tojson }}, monthlyData={{ monthly_data | tojson }}; if(diseaseData.labels.length>0){const ctx=document.getElementById('diseaseChart').getContext('2d'); new Chart(ctx, {type:'pie',data:{labels:diseaseData.labels,datasets:[{data:diseaseData.values,backgroundColor:['#059669','#F59E0B','#EF4444','#6366F1','#06B6D4','#F472B6']}]},options:{responsive:true, maintainAspectRatio: false, plugins:{legend:{position:'right', labels: {color: '#374151'}}}}});} if(monthlyData.labels.length>0){const ctx=document.getElementById('monthlyChart').getContext('2d'); new Chart(ctx, {type:'bar',data:{labels:monthlyData.labels,datasets:[{label:'{{ _("Number of Scans") }}',data:monthlyData.values,backgroundColor:'#059669',borderColor:'#047857',borderWidth:1}]},options:{responsive:true, maintainAspectRatio: false, scales:{y:{beginAtZero:true, ticks: {stepSize: 1, color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}, x: {ticks: {color: '#374151'}, grid: {color: 'rgba(209, 213, 219, 0.4)'}}}}});}</script>{% endblock %}"""

    # --- History Template (Redesigned List Cards) ---
    , "history.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Prediction History') }}</h2><div class='glass p-6 rounded-xl shadow-xl transition duration-300'><div class='grid md:grid-cols-3 gap-4'>{% for p in history %}{% set conf = p.confidence | int %}{% set bg_color = "red-100" if conf < 70 else ("blue-100" if conf < 85 else "green-100") %}{% set text_color = "red-700" if conf < 70 else ("blue-700" if conf < 85 else "green-700") %}{% set bar_color = "red-600" if conf < 70 else ("blue-600" if conf < 85 else "green-600") %}<div class='p-4 rounded-xl shadow-lg border border-gray-200 transition duration-300 hover:shadow-2xl hover:scale-[1.01]' style="background-color: var(--tw-colors-{{ bg_color }});"><div class='flex justify-between items-start mb-2'><div class='text-md font-semibold text-gray-800'>{{ p.prediction }}</div><div class='text-xs font-bold px-2 py-0.5 rounded-full text-white bg-gray-600'>{{ p.model_used }}</div></div><div class='text-xs text-gray-600 mb-2'><svg class="w-4 h-4 inline mr-1" fill="none" stroke="currentColor" viewBox="0 0 24 24"><path stroke-linecap="round" stroke-linejoin="round" stroke-width="2" d="M8 7V3m8 4V3m-9 9h.01M16 12h.01M21 16V6a2 2 0 00-2-2H5a2 2 0 00-2 2v14a2 2 0 002 2h14a2 2 0 002-2v-4"></path></svg> {{ p.timestamp }}</div><div class='w-full bg-gray-300 rounded-full h-2.5 mt-2'><div class='h-2.5 rounded-full' style='width: {{ p.confidence }}%; background-color: var(--tw-colors-{{ bar_color }});'></div></div><p class='text-xs mt-1 font-semibold' style='color: var(--tw-colors-{{ text_color }})'>{{ _('Confidence') }}: {{ p.confidence | int }}%</p><a href='{{ p.image_path }}' target='_blank' class='mt-2 inline-block text-blue-600 hover:text-blue-800 text-sm font-semibold'>{{ _('View Image') }}</a></div>{% endfor %}{% else %}<p class='text-gray-500'>{{ _('No saved predictions yet.') }}</p>{% endif %}</div></div>{% endblock %}"""

    , "metrics.html": """{% extends 'layout.html' %}{% block content %}<h2 class='text-3xl font-bold mb-6'>{{ _('Training Metrics') }}</h2><div class='glass p-6 rounded-xl shadow-xl'><p class='text-gray-700 mb-4'>{{ _('The files below are loaded from your Colab root directory.') }}</p>{% if training_results %}<h3 class='text-xl font-bold mt-4'>{{ _('Final Training Results') }}</h3><pre class='whitespace-pre-wrap bg-white/70 p-4 rounded-lg text-sm border border-gray-200 shadow-inner'>{{ training_results | tojson(indent=2) }}</pre>{% endif %}<h3 class='text-xl font-bold mt-6'>{{ _('Confusion Matrix') }}</h3><img src='{{ url_for("static", filename="confusion_matrix.png") }}' class='max-w-full rounded-lg shadow-md mt-2 border border-gray-200' alt='Confusion Matrix'>{% if report_exists %}<h3 class='text-xl font-bold mt-6'>{{ _('Classification Report') }}</h3><pre class='bg-white/70 p-4 rounded-lg text-sm border border-gray-200 shadow-inner'>{{ report_text }}</pre>{% endif %}{% if not training_results and not confusion_exists and not report_exists %}<p class='text-gray-500'>{{ _('No training metrics files found (training_results.json, confusion_matrix.png, classification_report.txt).') }}</p>{% endif %}</div>{% endblock %}"""

    , "team.html": """{% extends 'layout.html' %}{% block content %}<div class='flex flex-col items-center'><h2 class='text-4xl font-extrabold mb-4 text-center text-gray-900'>{{ _('Meet Our Team') }}</h2><p class='text-lg text-gray-600 mb-10 text-center max-w-2xl mx-auto'>{{ _('The dedicated contributors who brought AgroScan AI to life.') }}</p><div class='grid lg:grid-cols-4 md:grid-cols-2 gap-8 w-full justify-center'>{% for member in team_members %}<div class='team-card p-6 bg-white/95 rounded-2xl shadow-xl flex flex-col items-center text-center transition duration-300 hover:shadow-2xl hover:scale-[1.02] border border-gray-100'><div class='w-36 h-36 mb-4 mt-2 bg-gray-50 rounded-full overflow-hidden border-4 border-green-400/50 flex items-center justify-center'><img src='{{ member.img }}' alt='{{ member.name }} Bitmoji' class='w-full h-full object-cover'></div><h4 class='text-xl font-bold text-gray-800'>{{ member.name }}</h4><p class='text-md font-semibold text-blue-600 mb-3'>{{ member.role }}</p><div class='text-sm text-gray-700 mt-2 text-center'><p>{{ member.contributions }}</p></div><div class='mt-4 flex space-x-3'><div class='w-6 h-6 bg-blue-500/80 text-white text-xs rounded-full flex items-center justify-center'>F</div><div class='w-6 h-6 bg-red-500/80 text-white text-xs rounded-full flex items-center justify-center'>T</div></div></div>{% endfor %}</div><div class='mt-12 pt-8 border-t border-gray-200 glass p-6 rounded-xl w-full'><h3 class='text-xl font-semibold mb-4'>{{ _('Project Summary') }}</h3><p class='text-gray-700'>{{ project_summary }}</p></div></div>{% endblock %}"""
}

# Write HTML templates to files
for name, content in template_content.items():
    with open(os.path.join(TEMPLATES_DIR, name), "w", encoding="utf-8") as f:
        f.write(content)

# -------------------
# 9) Start Ngrok and Flask
# -------------------
def _cleanup():
    try: ngrok.kill()
    except: pass
atexit.register(_cleanup)

if __name__ == "__main__":
    # Start background model loading immediately
    threading.Thread(target=_background_preload, daemon=True).start()

    print("Colab Flask app (dual-model, Tailwind UI) starting...")
    print("Models discovered at:")
    for k,p in MODEL_PATHS.items():
        print(f" - {k}: {p} {'(FOUND)' if os.path.exists(p) else '(missing)'}")
    print("\nPaste your ngrok authtoken below. DO NOT re-run if model is still loading.")

    # Setup Ngrok
    try:
        token = getpass("ngrok authtoken (hidden): ")
        if token and token.strip():
            ngrok.set_auth_token(token.strip())
            public_url = ngrok.connect(5000)
            print(f"🔗 ngrok tunnel: {public_url}")
        else:
            print("Ngrok token not provided. App will only be accessible via local Colab link.")
    except Exception as e:
        print(f"Ngrok setup failed: {e}")

    # Run Flask
    app.run(host="0.0.0.0", port=5000, debug=False)

[model] Loading 'best' from best_crop_model.h5 ...
Colab Flask app (dual-model, Tailwind UI) starting...
Models discovered at:
 - best: best_crop_model.h5 (FOUND)
 - fallback: crop_disease_model.h5 (FOUND)

Paste your ngrok authtoken below. DO NOT re-run if model is still loading.
[model] Loaded 'best'.
[model] Loading 'fallback' from crop_disease_model.h5 ...
[model] Loaded 'fallback'.
ALL models preloaded and ready.
ngrok authtoken (hidden): ··········
🔗 ngrok tunnel: NgrokTunnel: "https://jared-choreographic-oversteadfastly.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 23:28:53] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 23:28:54] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 23:28:57] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 23:29:05] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 23:29:05] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 23:29:10] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 23:29:10] "GET / HTTP/1.1" 200 -
ERROR:__main__:Exception on /dashboard [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1511, in wsgi_app
   